# Setup

In [ ]:
base_path_kry = '/home/mmandelbrod/repositories/'
base_path_mac = '/Users/mmaandelbrod/workspace/'
import sys
import os

tmp = !hostname
if tmp[0].startswith('krylov'):
    on_kry = True
else: 
    on_kry = False
    
base_path = base_path_kry if on_kry else base_path_mac

print(base_path)

sys.path.append(os.path.join(base_path, 'signals/'))
# sys.path.append(os.path.join(base_path, 'signals/src'))
sys.path.append(os.path.join(base_path, 'generative-signals/'))
sys.path.append(os.path.join(base_path, 'generative-signals/src'))
sys.path.append(os.path.join(base_path, 'signals/workflows/pipelines/'))
sys.path.append(os.path.join(base_path, 'signals/workflows/global_utils/'))
sys.path.append(os.path.join(base_path, 'signals/workflows'))
sys.path.append(os.path.join(base_path, 'matans_sandbox/utils'))

os.chdir(os.path.join(base_path,'generative-signals'))
print(f"curr dir: {os.getcwd()}")
        

In [ ]:
import pyspark.sql.functions as F
import pandas as pd
import time

In [1]:
import warnings
warnings.filterwarnings("ignore")
import os
import sys
import base64
from bs4 import BeautifulSoup
import re
from pyspark.sql.functions import broadcast


In [2]:
from data_extraction.extraction_funcs import *

ModuleNotFoundError: No module named 'data_extraction'

In [ ]:
import data_extraction.extraction_funcs
from data_extraction.extraction_funcs import *

In [ ]:
bxkrylov.spark_notebook??

In [ ]:
import json
import os
from datetime import datetime
from functools import reduce
from typing import List

import bxkrylov
# import bx_signals.etl.signals_fetcher as fetcher
from pyspark.sql.functions import array, col, expr, lower, when, row_number, lit, first, get_json_object, from_json, explode, split, count, collect_list, concat_ws, array_distinct, map_values, aggregate, size, regexp_replace, isnull, filter
from pyspark.sql.types import StructType, StructField, StringType, ArrayType, MapType, IntegerType, LongType
from pyspark.sql.functions import array_intersect, array_contains, array_except, coalesce, map_keys
from pyspark.sql import DataFrame

directory = "./signals_sanity_reports"
if not os.path.exists(directory):
    os.makedirs(directory)
    
min_execs = 400
max_execs = 1000


# .config("spark.shuffle.io.maxRetries", "10") \
#     .config("spark.shuffle.io.retryWait", "5s") \
#     .config("spark.reducer.maxSizeInFlight", "96m") \
#     .config("spark.speculation", "true") \
#     .config("spark.speculation.quantile", "0.75") \
#     .config("spark.speculation.multiplier", "1.5") \


bxkrylov.spark_notebook(globals(),
                        spark_dynamicAllocation_minExecutors=min_execs,
                        spark_executor_instances=min_execs,
                        spark_dynamicAllocation_initialExecutors=min_execs,
                        spark_dynamicAllocation_maxExecutors=max_execs,
                        spark_executor_memory="10g",
                        spark_driver_maxResultSize="26g",
                        spark_executor_memoryOverhead="26g",
                        spark_driver_memory="80g",
                        spark_kryoserializer_buffer_max="1500m",
                        spark_executor_cores="5", 
                        spark_shuffle_io_maxRetries="10", 
                        spark_shuffle_io_retryWait="5s",
                        spark_reducer_maxSizeInFlight="96m",
                        spark_network_timeout="800s",
                        spark_executor_heartbeatInterval="60s"                        
                        )

directory = "./signals_sanity_reports"
if not os.path.exists(directory):
    os.makedirs(directory)
    
!python3 -V
spark.sparkContext.setLogLevel("OFF")
spark


bxkrylov - using spark config:  {
    "spark.driver.host": "10.207.16.31",
    "spark.rdd.compress": true,
    "spark.network.timeout": "800s",
    "spark.executor.heartbeatInterval": "60s",
    "spark.sql.broadcastTimeout": "12000s",
    "spark.yarn.queue": "hdlq-exprce-default",
    "spark.executor.extraJavaOptions": "-XX:MaxMetaspaceSize=512m -XX:ParallelGCThreads=3",
    "spark.driver.extraJavaOptions": "-XX:MaxMetaspaceSize=512m -XX:ParallelGCThreads=3",
    "spark.speculation": true,
    "spark.executor.cores": "5",
    "spark.driver.memory": "80g",
    "spark.executor.memory": "10g",
    "spark.driver.maxResultSize": "26g",
    "spark.dynamicAllocation.maxExecutors": 1000,
    "spark.dynamicAllocation.minExecutors": 400,
    "spark.executor.instances": 400,
    "spark.dynamicAllocation.initialExecutors": 400,
    "spark.executor.memoryOverhead": "26g",
    "spark.kryoserializer.buffer.max": "1500m",
    "spark.shuffle.io.maxRetries": "10",
    "spark.shuffle.io.retryWait": "5s",

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/apache/releases/spark-3.1.1.1.0.0-bin-ebay/jars/parquet-kms-client-0.2.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/apache/releases/spark-3.1.1.1.0.0-bin-ebay/jars/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/23 11:55:44 WARN HiveConf: DEPRECATED: hive.metastore.ds.retry.* no longer has any effect.  Use hive.hmshandler.retry.* instead
24/09/23 11:55:44 WARN HiveConf: HiveConf of name hive.metastore.local does not exist
24/09/23 11:55:44 WARN HiveConf: HiveConf of name hive.enforce.sorting does not exist
24/09/23 11:55:44 WARN HiveConf: HiveConf of name hive.ser

In [8]:
spark

In [8]:
sys.path

['/home/mmandelbrod/repositories/generative-signals/notebooks/data_extraction',
 '/tmp/spark-7d8c3c9a-2ec7-4985-b10c-1683515e2813/userFiles-676613bd-b8ab-4189-84eb-b498e54475c2',
 '/opt/clients/pykrylov/3.10/latest/install',
 '/home/mmandelbrod/repositories/generative-signals/notebooks/data_extraction',
 '/opt/conda/lib/python310.zip',
 '/opt/conda/lib/python3.10',
 '/opt/conda/lib/python3.10/lib-dynload',
 '',
 '/home/mmandelbrod/.local/lib/python3.10/site-packages',
 '/opt/conda/lib/python3.10/site-packages',
 '/opt/clients/propeller/3.10/latest/install',
 '/home/mmandelbrod/repositories/signals/',
 '/home/mmandelbrod/repositories/generative-signals/',
 '/home/mmandelbrod/repositories/generative-signals/src',
 '/home/mmandelbrod/repositories/signals/workflows/pipelines/',
 '/home/mmandelbrod/repositories/signals/workflows/global_utils/',
 '/home/mmandelbrod/repositories/signals/workflows',
 '/home/mmandelbrod/repositories/matans_sandbox/utils',
 '/opt/conda/lib/python3.10/site-packag

## importlib

In [51]:
import importlib

import data_extraction.extraction_funcs
importlib.reload(data_extraction.extraction_funcs)
import data_extraction.extraction_funcs
from data_extraction.extraction_funcs import *

# import dataset_utils
# importlib.reload(dataset_utils)
# import dataset_utils
# from dataset_utils import *

# # import xgb_ranker_classifier_pipeline
# # importlib.reload(xgb_ranker_classifier_pipeline)
# # from xgb_ranker_classifier_pipeline import *

# import src.metrics.auuc_funcs
# importlib.reload(src.metrics.auuc_funcs)
# import src.metrics.auuc_funcs
# from src.metrics.auuc_funcs import *

# import pipelines.xgb_predict_signal_pipeline
# importlib.reload(pipelines.xgb_predict_signal_pipeline)
# import pipelines.xgb_predict_signal_pipeline
# from pipelines.xgb_predict_signal_pipeline import *



# import src.metrics.xgb_metrics_funcs
# importlib.reload(src.metrics.xgb_metrics_funcs)
# import src.metrics.xgb_metrics_funcs
# from src.metrics.auuc_funcs import *


# # import signals_pipeline_base
# # importlib.reload(signals_pipeline_base)
# #
# import pipelines.causalml_decision_tree_pipeline
# importlib.reload(pipelines.causalml_decision_tree_pipeline)
# import pipelines.causalml_decision_tree_pipeline
# from pipelines.causalml_decision_tree_pipeline import *

# # import config
# # importlib.reload(config)
# # from config import *

# import causal.causalml_funcs
# importlib.reload(causal.causalml_funcs)
# import causal.causalml_funcs
# from causal.causalml_funcs import *

# import pipeline_objects
# importlib.reload(pipeline_objects)
# from pipeline_objects import *
# #%%


In [83]:
from  data_extraction.extraction_funcs import extract_item_data

In [84]:
extract_item_data??

Signature:
extract_item_data(
    spark,
    start_date,
    end_date,
    metacats=None,
    business_vertical_names=None,
    leafcats=None,
    site_ids=None,
    n_items=None,
    take_shortest_dup_value=True,
)
Docstring: <no docstring>
Source:   
def extract_item_data(spark, start_date, end_date, metacats=None, business_vertical_names = None,
                      leafcats=None, site_ids = None, n_items=None,
                      take_shortest_dup_value=True):
    items_df = spark.table("ACCESS_VIEWS.DW_LSTG_ITEM").filter((F.col('auct_end_dt') >= start_date) &\
                                                               (F.col('auct_end_dt') <= end_date))

    #TODO: there's no meta_categ_id in the table. Fix this
    # if metacats is not None:
    #     items_df = items_df.filter(F.col('meta_categ_id').isin(metacats))
    if leafcats is not None:
        items_df = items_df.filter(F.col('leaf_categ_id').isin(leafcats))

    if site_ids is not None:
        items_df = items_d

# Functions Definitions

In [12]:

# valid_types = ['df']
# invalid_aspect_names = ['mergednamespacenames', 'maturityscore',
#                         'isprplinkenabled', 'lhexposetotse', 'seller-selected-epid',
#                         'non-domestic product', 'modified item', 'upc',
#                         'producttitle', 'ean', 'savedautotagprodrefid', 'ebay product id (epid)',
#                         'gtin', 'california prop 65 warning', 'catrecoscore_1',
#                         'catrecoscore_2', 'catrecoscore_3', 'catrecoid_1', 'catrecoid_2',
#                         'catrecoid_3', 'miscatscore', 'p2sprobability', 'miscatscore_v1',
#                         'uom1', 'miscatscore_v1', 'uom2', 'mpn', 'uom3', 'productimagezoomguid',
#                         'manufacturer part number', 'isbn-13', 'isbn-10', 'other part number',
#                         'miscatscore_cf_v1', 'isclplinkenabled', 'oe/oem part number',
#                         'features', 'model', 'ks', 'number in pack',
#                         'item height', 'item length', 'item width', 'item weight',
#                         'number of items in set', 'food aisle', 'width', 'length', 'items included',
#                         'custom bundle', 'volume', 'period after opening (pao)', 'featured refinements',
#                         'set includes',
#                         ]

# def extract_item_data(spark, start_date, end_date, metacats=None, leafcats=None):
#     items_df = spark.table("ACCESS_VIEWS.DW_LSTG_ITEM").filter((F.col('auct_end_dt') >= start_date) &\
#                                                                (F.col('auct_end_dt') <= end_date))
#     #TODO: there's no meta_categ_id in the table. Fix this
#     # if metacats is not None:
#     #     items_df = items_df.filter(F.col('meta_categ_id').isin(metacats))
#     if leafcats is not None:
#         items_df = items_df.filter(F.col('leaf_categ_id').isin(leafcats))

#     items_df = items_df.select('ITEM_ID', 'AUCT_START_DT', 'AUCT_END_DT',  'leaf_categ_id' )#.cache()
    
#     aspects_df = spark.table("ACCESS_VIEWS.ITEM_ASPCT_CLSSFCTN_SAP").filter((F.col('auct_end_dt') >= start_date)&\
#                                                                 (F.col('auct_end_dt') <= end_date)&\
#                                                                 (F.col("NS_TYPE_CD").isin(valid_types))&\
#                                                                  (~F.col("PRDCT_ASPCT_NM").isin(invalid_aspect_names))
#                                                                 )
#     df_item_aspects = aspects_df.join(broadcast(items_df), on=['ITEM_ID', "AUCT_END_DT"])
    
#     # in case of multiple values per aspect, select the one with the shortest string value
#     df_item_aspects = df_item_aspects.withColumn('VALUE_LEN', F.length("ASPCT_VLU_NM").alias('VALUE_LEN'))
#     row_column = F.row_number().over(Window.partitionBy("ITEM_ID", "PRDCT_ASPCT_NM")
#                                      .orderBy(df_item_aspects['VALUE_LEN']))
#     df_item_aspects = df_item_aspects.withColumn("row_num", row_column.alias("row_num")) \
#         .where(F.col("row_num") == 1)
#     df_item_aspects = lowercase_string_columns(df_item_aspects)
#     return df_item_aspects




# def html_to_plain(html: str):
#     if html is None or len(html) == 0:
#         return ""
#     html = base64.b64decode(html).decode('utf-8')
#     try:
#         bs = BeautifulSoup(html, 'html.parser')
#         plain = bs.get_text(separator=' ', strip=True)
#         return re.sub(r"\s+", " ", plain)
#     except:
#         return ""

    
# def collect_item_aspects(session, df_items):
#     df_items = df_items.select('ITEM_ID', "AUCT_END_DT").dropDuplicates().cache()

#     max_auct_end_dt = df_items.agg({"AUCT_END_DT": "max"}).collect()[0][0]
#     min_auct_end_dt = df_items.agg({"AUCT_END_DT": "min"}).collect()[0][0]

#     # get aspects
#     df_item_aspects = session.table("ACCESS_VIEWS.ITEM_ASPCT_CLSSFCTN_SAP") \
#         .where((min_auct_end_dt < F.col('AUCT_END_DT')) & (F.col('AUCT_END_DT') < max_auct_end_dt)) \
#         .join(df_items, on=['ITEM_ID', "AUCT_END_DT"])

#     # in case of multiple values per aspect, select the one with the shortest string value
#     df_item_aspects = df_item_aspects.withColumn('VALUE_LEN', F.length("ASPCT_VLU_NM").alias('VALUE_LEN'))
#     row_column = F.row_number().over(Window.partitionBy("ITEM_ID", "PRDCT_ASPCT_NM")
#                                      .orderBy(df_item_aspects['VALUE_LEN']))
#     df_item_aspects = df_item_aspects.withColumn("row_num", row_column.alias("row_num")) \
#         .where(F.col("row_num") == 1)
#     df_item_aspects = lowercase_string_columns(df_item_aspects)
#     df_item_aspects = df_item_aspects.where((F.col("NS_TYPE_CD").isin(valid_types))
#                                             & (~F.col("PRDCT_ASPCT_NM").isin(invalid_aspect_names)))

#     # group all aspect name-value rows per item into a row with a dict column
#     df_item_aspects = df_item_aspects \
#         .withColumn("ASPECTS", F.create_map("PRDCT_ASPCT_NM", "ASPCT_VLU_NM").alias("ASPECTS")) \
#         .groupBy('ITEM_ID') \
#         .agg(F.collect_list('ASPECTS').alias('ASPECTS')) \
#         .select('ITEM_ID', combineMap('ASPECTS').alias('ASPECTS'))
#     return df_item_aspects

# def get_category_names(session, df_categories):
#     df_categories = df_categories.select("item_site_id", "LEAF_CATEG_ID").dropDuplicates()
#     df_categ_names = session.table("ACCESS_VIEWS.DW_CATEGORY_GROUPINGS") \
#         .where(F.col('LEAF_CATEG_ID') == F.col('MOVE_TO')) \
#         .join(df_categories, ["SITE_ID", "LEAF_CATEG_ID"]) \
#         .dropDuplicates(["SITE_ID", "LEAF_CATEG_ID"]) \
#         .select("SITE_ID", "LEAF_CATEG_ID", "LEAF_CATEG_NAME", "CATEG_LVL2_ID", "CATEG_LVL2_NAME", "META_CATEG_ID", "META_CATEG_NAME")
#     return df_categ_names

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import os
import sys
import base64
from bs4 import BeautifulSoup
import re
import pandas as pd

def html_to_plain(html: str):
    if html is None or len(html) == 0:
        return ""
    html = base64.b64decode(html).decode('utf-8')
    try:
        bs = BeautifulSoup(html, 'html.parser')
        plain = bs.get_text(separator=' ', strip=True)
        return re.sub(r"\s+", " ", plain)
    except:
        return ""

df = pd.read_parquet("home_app_item_desc")

df["desc"] = df["item_desc"].apply(lambda x: html_to_plain(x))

item_id_to_desc = dict()
for _, row in df.iterrows():
    item_id_to_desc[str(row["item_id"])] = row["desc"]

# Improving speed 

In [10]:
start_date = '2024-08-28'
end_date = '2024-08-28'
leafcats=[165193]

In [13]:
extract_item_data??

Signature:
extract_item_data(
    spark,
    start_date,
    end_date,
    metacats=None,
    leafcats=None,
    site_ids=None,
    n_items=None,
    take_shortest_dup_value=True,
)
Docstring: <no docstring>
Source:   
def extract_item_data(spark, start_date, end_date, metacats=None, leafcats=None, site_ids = None, n_items=None,
                      take_shortest_dup_value=True):
    items_df = spark.table("ACCESS_VIEWS.DW_LSTG_ITEM").filter((F.col('auct_end_dt') >= start_date) &\
                                                               (F.col('auct_end_dt') <= end_date))

    #TODO: there's no meta_categ_id in the table. Fix this
    # if metacats is not None:
    #     items_df = items_df.filter(F.col('meta_categ_id').isin(metacats))
    if leafcats is not None:
        items_df = items_df.filter(F.col('leaf_categ_id').isin(leafcats))

    if site_ids is not None:
        items_df = items_df.filter(F.col('site_id').isin(site_ids))

    if n_items is not None:
        # select 

In [36]:
df_item_aspects = extract_item_data(spark, start_date, end_date, metacats=None, leafcats=leafcats)

In [37]:
start = time.time()
pdf_item_aspects = df_item_aspects.toPandas()
end = time.time()
print(f"elapsed time (min): {(end - start)//60}")
print(f"num_aspects: {num_aspects}")

IOPub message rate exceeded.===========>                   (6034 + 300) / 10000]
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

24/08/21 14:55:11 ERROR YarnScheduler: Lost executor 200 on hdc42-mcc10-01-1110-4206-009-tess0029.stratus.rno.ebay.com: Executor decommission.
24/08/21 14:55:45 ERROR YarnScheduler: Lost executor 199 on hdc42-mcc10-01-0110-5103-015-tess0173.stratus.rno.ebay.com: Executor decommission.
24/08/21 14:55:57 ERROR YarnScheduler: Lost executor 129 on hdc42-mcc10-01-0710-3605-016-tess0035.stratus.rno.ebay.com: Executor decommission.
24/08/21 14:56:23 ERROR YarnScheduler: Lost executor 192 on hdc42-mcc10-01-0710-3402-025-tess0035.stratus.rno.ebay.com: Executor decommission.
24/08/21 14:56:27 ERROR YarnScheduler: Lost executor 167 o

elapsed time (min): 143.0
num_aspects: 1692994850


In [38]:
pdf_item_aspects

,item_id,auct_end_dt,prdct_aspct_nm,aspct_vlu_nm,ns_type_cd,aspct_src,load_ts,key_type,AUCT_START_DT,leaf_categ_id,VALUE_LEN,row_num
0,145950472855,2024-08-20,brand,john deere,df,30,2024-08-13 19:05:00,1,2024-08-13,165193,10,1
1,145950472855,2024-08-20,catrecoid_1,165193,df,6,2024-08-15 12:13:09,2,2024-08-13,165193,6,1
2,145950472855,2024-08-20,catrecoid_2,15687,df,6,2024-08-15 12:13:09,2,2024-08-13,165193,5,1
3,145950472855,2024-08-20,catrecoid_3,53159,df,6,2024-08-15 12:13:09,2,2024-08-13,165193,5,1
4,145950472855,2024-08-20,catrecoscore_1,52,df,6,2024-08-15 12:13:09,2,2024-08-13,165193,2,1
5,145950472855,2024-08-20,catrecoscore_2,21,df,6,2024-08-15 12:13:09,2,2024-08-13,165193,2,1
6,145950472855,2024-08-20,catrecoscore_3,12,df,6,2024-08-15 12:13:09,2,2024-08-13,165193,2,1
7,145950472855,2024-08-20,country/region of manufacture,jersey,df,3,2024-08-13 19:05:00,1,2024-08-13,165193,6,1
8,145950472855,2024-08-20,theme,agriculture,df,3,2024-08-13 19:05:00,1,2024-08-13,165193,11,1
9,145950472855,2024-08-20,core_product_type_v2,clothing: primary product,df,291,2024-08-15 12:13:09,2,2024-08-13,165193,25,1


In [29]:
aspects_df.select('auct_end_dt').groupby('auct_end_dt').count().show()

2024-08-21 12:33:04,622 ERROR MainProcess KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/opt/clients/pykrylov/3.10/latest/install/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/clients/pykrylov/3.10/latest/install/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/conda/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

# Debugging by caching 

In [48]:
items_df = spark.table("ACCESS_VIEWS.DW_LSTG_ITEM")

In [49]:
items_df.printSchema()

root
 |-- item_id: decimal(18,0) (nullable = true)
 |-- auct_end_dt: date (nullable = true)
 |-- auct_start_dt: date (nullable = true)
 |-- sale_sched_end_dt: date (nullable = true)
 |-- auct_type_code: decimal(4,0) (nullable = true)
 |-- item_site_id: decimal(4,0) (nullable = true)
 |-- item_cntry_id: decimal(4,0) (nullable = true)
 |-- lstg_curncy_id: decimal(4,0) (nullable = true)
 |-- lstg_status_id: byte (nullable = true)
 |-- icon_flags: string (nullable = true)
 |-- slr_id: decimal(18,0) (nullable = true)
 |-- slr_site_id: decimal(4,0) (nullable = true)
 |-- slr_cntry_id: decimal(4,0) (nullable = true)
 |-- high_bdr_id: decimal(18,0) (nullable = true)
 |-- leaf_categ_id: decimal(9,0) (nullable = true)
 |-- leaf_categ_id2: decimal(9,0) (nullable = true)
 |-- auct_start_exchng_rate: decimal(18,14) (nullable = true)
 |-- auct_end_exchng_rate: decimal(18,14) (nullable = true)
 |-- start_price_lstg_curncy: decimal(18,2) (nullable = true)
 |-- rsrv_price_list_crncy: decimal(18,2) (nul

In [39]:
items_df = spark.table("ACCESS_VIEWS.DW_LSTG_ITEM").filter((F.col('auct_end_dt') >= start_date) &\
                                                               (F.col('auct_end_dt') <= end_date)).cache()

In [50]:
items_df.count()

2024-08-21 15:21:46,148 ERROR MainProcess KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/opt/clients/pykrylov/3.10/latest/install/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/clients/pykrylov/3.10/latest/install/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/conda/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [11]:
aspects_df = spark.table("ACCESS_VIEWS.ITEM_ASPCT_CLSSFCTN_SAP").filter((F.col('auct_end_dt') >= start_date)&\
                                                                (F.col('auct_end_dt') <= end_date)&\
                                                                (F.col("NS_TYPE_CD").isin(valid_types))&\
                                                                 (~F.col("PRDCT_ASPCT_NM").isin(invalid_aspect_names))
                                                                ).cache()

In [12]:
aspects_df.count()

2024-08-21 16:36:07,907 ERROR MainProcess KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/opt/clients/pykrylov/3.10/latest/install/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/clients/pykrylov/3.10/latest/install/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/conda/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [44]:
aspects_df.rdd.getNumPartitions()

10000

In [45]:
df_item_aspects = aspects_df.join(items_df, on=['ITEM_ID', "AUCT_END_DT"])

In [46]:
df_item_aspects.rdd.getNumPartitions()

2024-08-21 15:11:32,716 ERROR MainProcess KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/opt/clients/pykrylov/3.10/latest/install/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/clients/pykrylov/3.10/latest/install/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/conda/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

# Full dataset generation

In [9]:
start_date = '2024-08-28'
end_date = '2024-08-28'
# leafcats = [261328] #Trading Card Singles (it's a leafcat)
leafcats = ['169291'] #Women's Bags & Handbags
n_items = 1000
take_shortest_dup_value = False

In [37]:
# extract_item_data??

In [98]:
start = time.time()
items_aspects_df_169291 = extract_item_data(spark, start_date, end_date, leafcats=leafcats, n_items = n_items, take_shortest_dup_value=take_shortest_dup_value )
items_aspects_pdf_169291 = items_aspects_df_169291.toPandas()
items_aspects_pdf_169291.to_parquet('data/items_aspects_pdf_169291.parquet')
# items_aspects_df.write.parquet('mmandelbrod/gen_sigs/24_08_20_gensig_data_leafcat_261328.parquet')
end = time.time()
print(f"elapsed time (min): {(end - start)//60}")



Adding aspects
Adding descriptions


AttributeError: 'DataFrame' object has no attribute 'rename'

In [43]:
items_aspects_pdf_169291

,ITEM_ID,item_desc,AUCT_TITL,SUBTITLE,AUCT_START_DT,AUCT_END_DT,leaf_categ_id,ITEM_SITE_ID,ASPECTS
0,176354257674,PHNwYW4gaWQ9IlBIZWFkZXJfbGJsIj4gCiA8dGFibGUgd2...,pu borsa a tracolla da donna borsa a secchiell...,None,2024-04-28,2024-08-28,169291,101,"{'marca': '- senza marca/generico -', 'colore ..."
1,315255191452,PCFET0NUWVBFIGh0bWw+CjxodG1sPgo8aGVhZD4KICAgID...,mandarina duck vichy tote vichy pink,None,2024-03-28,2024-08-28,169291,186,"{'color exterior': 'rosa', 'marca': 'mandarina..."
2,315603946287,PGRpdj48c3BhbiBzdHlsZT0iYmFja2dyb3VuZC1jb2xvcj...,brighton black leather croc women's shoulder b...,None,2024-07-28,2024-08-28,169291,0,"{'occasion': 'travel', 'country/region of manu..."
3,395559852922,PCFET0NUWVBFIGh0bWwgUFVCTElDICItLy9XM0MvL0RURC...,fashion forward pu leather handbag for women 2...,None,2024-07-28,2024-08-28,169291,15,"{'miscatscore': '26', 'lining material': 'lamb..."
4,405003403992,PHA+VGhpcyB3aGl0ZSBhbmQgc2lsdmVyIEF0aGxldGEgdG...,white & silver athleta tote bag long handles e...,None,2024-05-28,2024-08-28,169291,0,"{'occasion': 'tote', 'hardware color': 'white'..."
...,...,...,...,...,...,...,...,...,...
995,256556964671,PG1ldGEgaHR0cHMtZXF1aXY9IkNvbnRlbnQtVHlwZSIgY2...,lattice pattern crossbody bags large capacity ...,None,2024-06-28,2024-08-28,169291,3,"{'country/region of manufacture': 'china', 'ha..."
996,285982660756,IDwhLS0gYmVnaW45OTk5OTlfMSAtLT48ZGl2IGNsYXNzPS...,4 pcs strandschaufel kinder sandkasten spielz...,None,2024-07-28,2024-08-28,169291,77,"{'miscatscore': '29', 'catrecoscore_3': '17', ..."
997,285983598595,IDwhLS0gYmVnaW45OTk5OTlfMSAtLT48ZGl2IGNsYXNzPS...,2 pieces indonesian rattan pine cone satchel ...,None,2024-07-28,2024-08-28,169291,3,"{'miscatscore': '25', 'catrecoscore_2': '27', ..."
998,364237256721,RU1NQSBGT1ggYmVpZ2UgbGVhdGhlciBoYW5kYmFnLiA=,emma fox beige leather handbag,None,2023-04-28,2024-08-28,169291,0,"{'occasion': 'shoulder bag', 'catrecoscore_1':..."


# Extraction at 14.9.24

## Extract Collectibles for site 0

In [8]:
start_date = '2024-09-25'
end_date = '2024-09-25'
business_vertical_names = ['Collectibles']
site_ids = [0]
n_items = 10000
take_shortest_dup_value = False
leafcats=None
metacats=None

In [ ]:
start = time.time()
items_aspects_df_collectb_us = extract_item_data(spark, start_date, end_date, business_vertical_names=business_vertical_names,
                                            site_ids=site_ids,
                                            n_items = n_items, take_shortest_dup_value=take_shortest_dup_value )
items_aspects_pdf_collectb_us = items_aspects_df_collectb_us.toPandas()
items_aspects_pdf_collectb_us.to_parquet('data/Datasets/hag_pna_collct_09_24/RawExtracted/collct_us_09_27.parquet')
end = time.time()
print(f"elapsed time (min): {(end - start)//60}")

Adding aspects
Adding descriptions


24/09/15 05:47:47 ERROR YarnScheduler: Lost executor 1318 on hdc42-mcc10-01-0110-7501-016-tess0173.stratus.rno.ebay.com: Container from a bad node: container_e4098_1726112755069_631806_01_001718 on host: hdc42-mcc10-01-0110-7501-016-tess0173.stratus.rno.ebay.com. Exit status: 154. Diagnostics: [2024-09-14 22:47:43.517]Container exited with a non-zero exit code 154
[2024-09-14 22:47:43.517]Container exited with a non-zero exit code 154
.


In [10]:
items_aspects_pdf_collectb_us

,item_id,item_desc,title,SUBTITLE,AUCT_START_DT,AUCT_END_DT,leaf_categ_id,ITEM_SITE_ID,bsns_vrtcl_name,meta_categ_id,GALLERY_URL,aspects
0,126643930817,PGRpdiBzdHlsZT0iIndpZHRoOiAxMDAlOyB0ZXh0LWFsaW...,sasaki saki wixoss signature,None,2024-08-25,2024-09-25,69528,0,collectibles,1,https://i.ebayimg.com/00/s/ODEwWDEwODA=/z/VbYA...,"{'miscatscore': '42', 'catrecoscore_3': '6', '..."
1,144570721711,PHN0eWxlPmltZ3ttYXgtd2lkdGg6MTAwJX08L3N0eWxlPj...,1955 bowman baseball #129 hal naragon vg/ex *d2,None,2022-05-25,2024-09-25,261328,0,collectibles,64482,https://i.ebayimg.com/00/s/ODI4WDExNzI=/z/kLMA...,{'country/region of manufacture': 'united stat...
2,145854441123,CgoKCTxtZXRhIGNoYXJzZXQ9InV0Zi04Ij4KCTxtZXRhIG...,edw1949sell : canada 1928-29 scott #149-57 ver...,None,2024-06-25,2024-09-25,3480,0,collectibles,260,https://i.ebayimg.com/00/s/Nzk4WDc5Nw==/z/jI4A...,"{'country/region of manufacture': 'canada', 'p..."
3,145978761092,PHAgY2xhc3M9Ik1zb05vcm1hbCIgc3R5bGU9InRleHQtYW...,jim plunkett san francisco 49ers nfl football ...,None,2024-08-25,2024-09-25,112478,0,collectibles,1,https://i.ebayimg.com/00/s/OTY0WDExMDQ=/z/Kk4A...,"{'subject': 'san francisco', 'year of producti..."
4,156274109142,PHN0eWxlIHR5cGU9J3RleHQvY3NzJz4KaHRtbCwgYm9keS...,cabochon black scorpion pear shape (cut top) 3...,None,2024-06-25,2024-09-25,103931,0,collectibles,1,https://i.ebayimg.com/00/s/MTU4M1gxNjAw/z/MEcA...,"{'miscatscore': '15', 'catrecoscore_3': '0', '..."
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,326176436617,PCEtLSBWZXJzaW9uOiBTaXhCaXQgQWdlbnQgNC4wLjI0Ny...,"1953 us cover - sesquicentennial, tiffin, ohio c4",None,2024-06-25,2024-09-25,689,0,collectibles,260,https://i.ebayimg.com/00/s/OTAxWDE2MDA=/z/JScA...,"{'place of origin': 'united states', 'catrecos..."
9996,353432532874,PGZvbnQgcndyPScxJyBzaXplPSc0JyBzdHlsZT0nZm9udC...,nhl canada 2000 & 2001 all star games first da...,None,2021-03-25,2024-09-25,3486,0,collectibles,260,https://i.ebayimg.com/00/s/MTIwMFgxNjAw/z/LcgA...,{'country/region of manufacture': 'united stat...
9997,363119549857,PGZvbnQgc2l6ZT0iNCI+PGZvbnQgc3R5bGU9ImJhY2tncm...,1969 chevrolet cst fleetside pickup orange tru...,None,2020-09-25,2024-09-25,1316,0,collectibles,1,https://i.ebayimg.com/00/s/MTA4MFg4MTA=/z/h88A...,"{'color': 'orange', 'country/region': 'united ..."
9998,371992756516,PFRBQkxFIENFTExTUEFDSU5HPSIyOCIgQ0VMTFBBRERJTk...,agony and the ecstasy 1965 lobby card #8 rex h...,None,2017-06-25,2024-09-25,198,0,collectibles,45100,http://i.ebayimg.com/00/s/NDgwWDcyMA==/z/whEAA...,"{'miscatscore': '26', 'catrecoscore_3': '12', ..."


In [10]:
items_aspects_pdf_collectb_us = pd.read_parquet('data/ExtractedNoSig/items_aspects_pdf_collectb_us_oct.parquet')

In [14]:
items_aspects_pdf_collectb_us.loc[108, 'GALLERY_URL']

'https://i.ebayimg.com/00/s/mjk3wdeymda=/z/66qaaosw6xjlv6ny/$_1.jpg?set_id=880000500f'

In [52]:
description_df = spark.table("ACCESS_VIEWS.LSTG_ITEM_DESC_DIM").filter((F.col('auct_end_dt') >= start_date) &\
                                                                (F.col('auct_end_dt') <= end_date)).\
                                                                select('item_id', 'item_desc')

In [61]:
spark.table("ACCESS_VIEWS.LSTG_ITEM_DESC_DIM").filter((F.col('auct_end_dt') >= start_date) &\
                                                                (F.col('auct_end_dt') <= end_date)).count()

76387395

24/09/01 14:37:19 ERROR YarnScheduler: Lost executor 3041 on hdc42-mcc10-01-1110-3803-028-tess0029.stratus.rno.ebay.com: Executor decommission.
24/09/01 14:37:20 ERROR YarnScheduler: Lost executor 3489 on hdc42-mcc10-01-1110-3405-012-tess0029.stratus.rno.ebay.com: Executor decommission.
24/09/01 14:37:20 ERROR YarnScheduler: Lost executor 3078 on hdc42-mcc10-01-0710-2106-033-tess0035.stratus.rno.ebay.com: Executor decommission.
24/09/01 14:37:20 ERROR YarnScheduler: Lost executor 3515 on hdc42-mcc10-01-1110-3902-024-tess0029.stratus.rno.ebay.com: Executor decommission.
24/09/01 14:37:21 ERROR YarnScheduler: Lost executor 2594 on hdc42-mcc10-01-1110-2504-016-tess0029.stratus.rno.ebay.com: Executor decommission.
24/09/01 14:37:21 ERROR YarnScheduler: Lost executor 2361 on hdc42-mcc10-01-0110-7105-020-tess0173.stratus.rno.ebay.com: Executor decommission.
24/09/01 14:37:21 ERROR YarnScheduler: Lost executor 3415 on hdc42-mcc10-01-1110-3601-010-tess0029.stratus.rno.ebay.com: Executor decomm

In [51]:
items_aspects_pdf_collectb_us['item_desc'].unique()

array([None], dtype=object)

In [94]:
items_aspects_pdf_collectb_us.to_parquet('data/items_aspects_pdf_collectb_us.parquet')

In [100]:
items_aspects_pdf_collectb_us = pd.read_parquet('data/items_aspects_pdf_collectb_us.parquet')
items_aspects_pdf_collectb_us

,item_id,item_desc,title,SUBTITLE,AUCT_START_DT,AUCT_END_DT,leaf_categ_id,ITEM_SITE_ID,bsns_vrtcl_name,meta_categ_id,aspects,desc
0,114664125925,VGhpcyB2aW50YWdlIG11ZyBpcyBhIG11c3QtaGF2ZSBmb3...,vintage hardee's rise and shine made from scra...,None,2021-02-02,2024-09-02,850,0,collectibles,1,"{'4wd/2wd': None, 'age': None, 'age level': No...",This vintage mug is a must-have for collectors...
1,115755359434,PHA+VXAgZm9yIHNhbGUgaXMgYSAxOTgwcyBEb24gTGFjZW...,1980 don lacefield chevrolet license plate hil...,None,2023-04-02,2024-09-02,421,0,collectibles,1,"{'4wd/2wd': None, 'age': None, 'age level': No...",Up for sale is a 1980s Don Lacefield Chevrolet...
2,155236461306,PGRpdiBpZD0iZHNfZGl2Ij48Zm9udCByd3I9IjEiIHNpem...,2020 topps opening day fernando tatis jr. #189...,None,2022-11-02,2024-09-02,261328,0,collectibles,64482,"{'4wd/2wd': None, 'age': None, 'age level': No...",DESCRIPTION The item you see in the pictures i...
3,155390411573,TWV3dHdvIFZTVEFSIEdvbGQgMDkxLzA3MSBVUiBTMTBiIE...,mewtwo vstar gold 091/071 ur s10b japanese pok...,None,2023-02-02,2024-09-02,183454,0,collectibles,220,"{'4wd/2wd': None, 'age': None, 'age level': No...",Mewtwo VSTAR Gold 091/071 UR S10b Japanese Pok...
4,175597284740,PGI+PGZvbnQgZmFjZT0iQXJpYWwiIHNpemU9IjQiPkw0OD...,signature models 1937 cord 812 supercharger (1...,None,2023-02-02,2024-09-02,180506,0,collectibles,220,"{'4wd/2wd': None, 'age': None, 'age level': No...",L48
...,...,...,...,...,...,...,...,...,...,...,...,...
4995,355509021587,VGhpc8KgMjAwOMKgVG9wcHPCoEtpY2tvZmbCoHNpbmdsZS...,peyton manning 2008 topps kickoff stars of the...,None,2024-03-02,2024-09-02,261328,0,collectibles,64482,"{'4wd/2wd': None, 'age': None, 'age level': No...",This 2008 Topps Kickoff single-insert sports t...
4996,355761665374,PGRpdj48c3Ryb25nIHN0eWxlPSJjb2xvcjogcmdiKDE3My...,lego black minifig torso hogwarts robe clasp s...,None,2024-06-02,2024-09-02,183448,0,collectibles,220,"{'4wd/2wd': None, 'age': None, 'age level': '4...","100% LEGO Brand - Check out our store often, a..."
4997,375408380381,PHAgZGlyPSJsdHIiIHN0eWxlPSJtYXJnaW4tdG9wOjA7IG...,vintage 1999 hoop furby basketball very damage...,None,2024-05-02,2024-09-02,1083,0,collectibles,220,"{'4wd/2wd': None, 'age': None, 'age level': No...",This is in horrible condition. It's covered in...
4998,394974214459,PGZvbnQgc2l6ZT0iNCIgc3R5bGU9ImZvbnQtZmFtaWx5Ok...,old vintage antique photo booth photograph pre...,None,2023-11-02,2024-09-02,262421,0,collectibles,1,"{'4wd/2wd': None, 'age': None, 'age level': No...","Photograph 1-1/2x1-3/4"" This listing is for a ..."


In [106]:
tmp = items_aspects_pdf_collectb_us.loc[0, 'aspects']
tmp1 = {k:v for k,v in tmp.items() if v is not None}
tmp1

{'catrecoid_1': '21147',
 'catrecoid_2': '850',
 'catrecoid_3': '261672',
 'catrecoscore_1': '40',
 'catrecoscore_2': '25',
 'catrecoscore_3': '21',
 'color': 'multi-color',
 'core_product_type': 'advertising items',
 'core_product_type_v2': 'vintage advertisements: primary product',
 'country/region of manufacture': 'united states',
 'miscatscore': '56',
 'original/reproduction': 'original',
 'theme': 'restaurants & fast food'}

In [124]:
items_aspects_pdf_collectb_us['aspects_no_none'] = items_aspects_pdf_collectb_us['aspects'].apply(lambda d : {k:v for k,v in d.items() if v is not None})


In [114]:
items_aspects_pdf_collectb_us.rename(columns={'aspects':'aspects_with_none', 'aspects_no_none':'aspects'}, inplace=True)

In [128]:
items_aspects_pdf_collectb_us.loc[0, 'aspects_no_none']

{'catrecoid_1': '21147',
 'catrecoid_2': '850',
 'catrecoid_3': '261672',
 'catrecoscore_1': '40',
 'catrecoscore_2': '25',
 'catrecoscore_3': '21',
 'color': 'multi-color',
 'core_product_type': 'advertising items',
 'core_product_type_v2': 'vintage advertisements: primary product',
 'country/region of manufacture': 'united states',
 'miscatscore': '56',
 'original/reproduction': 'original',
 'theme': 'restaurants & fast food'}

In [121]:
items_aspects_pdf_collectb_us.loc[0, 'aspects']

{'catrecoid_1': '21147',
 'catrecoid_2': '850',
 'catrecoid_3': '261672',
 'catrecoscore_1': '40',
 'catrecoscore_2': '25',
 'catrecoscore_3': '21',
 'color': 'multi-color',
 'core_product_type': 'advertising items',
 'core_product_type_v2': 'vintage advertisements: primary product',
 'country/region of manufacture': 'united states',
 'miscatscore': '56',
 'original/reproduction': 'original',
 'theme': 'restaurants & fast food'}

In [129]:
items_aspects_pdf_collectb_us.to_parquet('data/items_aspects_pdf_collectb_us_no_none_aspects_1.parquet')

In [123]:

items_aspects_pdf_collectb_us

,item_id,item_desc,title,SUBTITLE,AUCT_START_DT,AUCT_END_DT,leaf_categ_id,ITEM_SITE_ID,bsns_vrtcl_name,meta_categ_id,aspects_with_none,desc,aspects
0,114664125925,VGhpcyB2aW50YWdlIG11ZyBpcyBhIG11c3QtaGF2ZSBmb3...,vintage hardee's rise and shine made from scra...,None,2021-02-02,2024-09-02,850,0,collectibles,1,"{'4wd/2wd': None, 'age': None, 'age level': No...",This vintage mug is a must-have for collectors...,"{'catrecoid_1': '21147', 'catrecoid_2': '850',..."
1,115755359434,PHA+VXAgZm9yIHNhbGUgaXMgYSAxOTgwcyBEb24gTGFjZW...,1980 don lacefield chevrolet license plate hil...,None,2023-04-02,2024-09-02,421,0,collectibles,1,"{'4wd/2wd': None, 'age': None, 'age level': No...",Up for sale is a 1980s Don Lacefield Chevrolet...,"{'catrecoid_1': '25432', 'catrecoid_2': '421',..."
2,155236461306,PGRpdiBpZD0iZHNfZGl2Ij48Zm9udCByd3I9IjEiIHNpem...,2020 topps opening day fernando tatis jr. #189...,None,2022-11-02,2024-09-02,261328,0,collectibles,64482,"{'4wd/2wd': None, 'age': None, 'age level': No...",DESCRIPTION The item you see in the pictures i...,"{'autographed': 'no', 'brand': 'topps', 'card ..."
3,155390411573,TWV3dHdvIFZTVEFSIEdvbGQgMDkxLzA3MSBVUiBTMTBiIE...,mewtwo vstar gold 091/071 ur s10b japanese pok...,None,2023-02-02,2024-09-02,183454,0,collectibles,220,"{'4wd/2wd': None, 'age': None, 'age level': No...",Mewtwo VSTAR Gold 091/071 UR S10b Japanese Pok...,"{'autographed': 'no', 'brand': 'nintendo', 'ca..."
4,175597284740,PGI+PGZvbnQgZmFjZT0iQXJpYWwiIHNpemU9IjQiPkw0OD...,signature models 1937 cord 812 supercharger (1...,None,2023-02-02,2024-09-02,180506,0,collectibles,220,"{'4wd/2wd': None, 'age': None, 'age level': No...",L48,"{'brand': 'cord', 'catrecoid_1': '180506', 'ca..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,355509021587,VGhpc8KgMjAwOMKgVG9wcHPCoEtpY2tvZmbCoHNpbmdsZS...,peyton manning 2008 topps kickoff stars of the...,None,2024-03-02,2024-09-02,261328,0,collectibles,64482,"{'4wd/2wd': None, 'age': None, 'age level': No...",This 2008 Topps Kickoff single-insert sports t...,"{'brand': 'topps', 'card condition': 'near min..."
4996,355761665374,PGRpdj48c3Ryb25nIHN0eWxlPSJjb2xvcjogcmdiKDE3My...,lego black minifig torso hogwarts robe clasp s...,None,2024-06-02,2024-09-02,183448,0,collectibles,220,"{'4wd/2wd': None, 'age': None, 'age level': '4...","100% LEGO Brand - Check out our store often, a...","{'age level': '4+', 'brand': 'lego', 'catrecoi..."
4997,375408380381,PHAgZGlyPSJsdHIiIHN0eWxlPSJtYXJnaW4tdG9wOjA7IG...,vintage 1999 hoop furby basketball very damage...,None,2024-05-02,2024-09-02,1083,0,collectibles,220,"{'4wd/2wd': None, 'age': None, 'age level': No...",This is in horrible condition. It's covered in...,"{'brand': 'tiger', 'catrecoid_1': '1083', 'cat..."
4998,394974214459,PGZvbnQgc2l6ZT0iNCIgc3R5bGU9ImZvbnQtZmFtaWx5Ok...,old vintage antique photo booth photograph pre...,None,2023-11-02,2024-09-02,262421,0,collectibles,1,"{'4wd/2wd': None, 'age': None, 'age level': No...","Photograph 1-1/2x1-3/4"" This listing is for a ...","{'catrecoid_1': '262421', 'catrecoid_2': '1124..."


In [109]:
items_aspects_pdf_collectb_us.loc[1, 'aspects_no_none']

{'catrecoid_1': '25432',
 'catrecoid_2': '421',
 'catrecoid_3': '9886',
 'catrecoscore_1': '54',
 'catrecoscore_2': '38',
 'catrecoscore_3': '5',
 'core_product_type': 'license plates',
 'core_product_type_v2': 'collectibles: primary product',
 'miscatscore': '62'}

## Extract P&A for site 100 (US)

In [9]:
start_date = '2024-09-25'
end_date = '2024-09-25'
business_vertical_names = ['Parts & Accessories']
site_ids = [100]
n_items = 10000
take_shortest_dup_value = False
leafcats=None
metacats=None

In [ ]:
start = time.time()
items_aspects_df_pna_us = extract_item_data(spark, start_date, end_date, business_vertical_names=business_vertical_names,
                                            site_ids=site_ids,
                                            n_items = n_items, take_shortest_dup_value=take_shortest_dup_value )
items_aspects_pdf_pna_us = items_aspects_df_pna_us.toPandas()
items_aspects_pdf_pna_us.to_parquet('data/Datasets/hag_pna_collct_09_24/RawExtracted/pna_us_09_27.parquet')
end = time.time()
print(f"elapsed time (min): {(end - start)//60}")

Adding aspects
Adding descriptions


In [15]:
items_aspects_pdf_pna_us_read = pd.read_parquet('data/Datasets/hag_pna_collct_09_24/RawExtracted/pna_us_09_27.parquet')
items_aspects_pdf_pna_us_read

,item_id,item_desc,title,SUBTITLE,AUCT_START_DT,AUCT_END_DT,leaf_categ_id,ITEM_SITE_ID,bsns_vrtcl_name,meta_categ_id,GALLERY_URL,aspects
0,116229424196,PG1ldGEgbmFtZT0ia2V5d29yZHMiIGNvbnRlbnQ9InVzZW...,00-06 mercedes w215 cl500 cl600 front right se...,None,2024-06-25,2024-09-25,262201,100,parts & accessories,6000,https://i.ebayimg.com/00/s/ODY3WDEzMDA=/z/zlgA...,"{'2-way paging': None, 'accessories': None, 'a..."
1,126028451215,PGZvbnQgcndyPSIxIiBzdHlsZT0iIj48ZGl2IHN0eWxlPS...,"blue-point tools new 3/8"" drive 6mm metric hex...",None,2023-07-25,2024-09-25,179527,100,parts & accessories,6000,https://i.ebayimg.com/00/s/MTIwMFgxNjAw/z/dtcA...,"{'2-way paging': None, 'accessories': None, 'a..."
2,134952985532,PCEtLSBEZXNjVGVtcGxhdGVJRDogRnJvbnQgQm9keSAoNj...,windshield wiper motor fits 95-96 ford e150 va...,None,2024-02-25,2024-09-25,262179,100,parts & accessories,6000,https://i.ebayimg.com/00/s/NDUwWDgwMA==/z/CEsA...,"{'2-way paging': None, 'accessories': None, 'a..."
3,135211723255,PHN0eWxlPgogICAgQG1lZGlhIHNjcmVlbiBhbmQgKG1heC...,21'' front wheel rim & dual disc wheel hub fit...,None,2024-08-25,2024-09-25,147699,100,parts & accessories,6000,https://i.ebayimg.com/00/s/MTMwMFgxMzAw/z/FxoA...,"{'2-way paging': None, 'accessories': None, 'a..."
4,144432285379,PCEtLSBCRUdJTiBWSURFTyAtLT4KPGRpdiBjbGFzcz0iZn...,hyper flash free led light 3156 amber two bulb...,None,2022-02-25,2024-09-25,172517,100,parts & accessories,6000,https://i.ebayimg.com/00/s/MTU5MlgxNjAw/z/D7oA...,"{'2-way paging': None, 'accessories': None, 'a..."
...,...,...,...,...,...,...,...,...,...,...,...,...
9981,394481992200,CiAgPCFET0NUWVBFIGh0bWw+CjxodG1sIGxhbmc9ImVuIj...,for 2001-2004 mercedes slk320 camshaft positio...,None,2023-02-25,2024-09-25,33614,100,parts & accessories,6000,https://i.ebayimg.com/00/s/NDgwWDY0MA==/z/aTwA...,"{'2-way paging': None, 'accessories': None, 'a..."
9982,395618655825,PG1ldGEgY2hhcnNldD0iVVRGLTgiPgo8bWV0YSBuYW1lPS...,"for ford maverick 2022-2024 4.5 ft 54.4"" truck...",None,2024-08-25,2024-09-25,33697,100,parts & accessories,6000,https://i.ebayimg.com/00/s/MTIwMFgxMjAw/z/FRYA...,"{'2-way paging': None, 'accessories': None, 'a..."
9983,402382727509,CgkJCQkJICA8IURPQ1RZUEUgaHRtbD4KPGh0bWwgbGFuZz...,front right anco contour wiper blade fits satu...,None,2020-08-25,2024-09-25,179852,100,parts & accessories,6000,https://i.ebayimg.com/00/s/NDgwWDY0MA==/z/ihQA...,"{'2-way paging': None, 'accessories': None, 'a..."
9984,405123346459,PCEtLXRvbmd0b29sLXJlbGF0ZWRDc3NTdGFydC0tPjxsaW...,4 gang switch box emergency strobe light contr...,None,2024-07-25,2024-09-25,50459,100,parts & accessories,6000,https://i.ebayimg.com/00/s/MTYwMFgxNjAw/z/54IA...,"{'2-way paging': None, 'accessories': None, 'a..."


In [14]:
!ls -l data/Datasets/hag_pna_collct_09_24/RawExtracted/

total 269208
-rw-rw-r-- 1 mmandelbrod mmandelbrod  33202721 Sep 15 06:10 collct_us_09_27.parquet
-rw-rw-r-- 1 mmandelbrod mmandelbrod 122304008 Sep 15 11:27 hag_us_09_27.parquet
-rw-rw-r-- 1 mmandelbrod mmandelbrod 119059290 Sep 16 20:37 pna_us_09_27.parquet


In [12]:
items_aspects_pdf_pna_us

,item_id,item_desc,title,SUBTITLE,AUCT_START_DT,AUCT_END_DT,leaf_categ_id,ITEM_SITE_ID,bsns_vrtcl_name,meta_categ_id,GALLERY_URL,aspects
0,116229424196,PG1ldGEgbmFtZT0ia2V5d29yZHMiIGNvbnRlbnQ9InVzZW...,00-06 mercedes w215 cl500 cl600 front right se...,None,2024-06-25,2024-09-25,262201,100,parts & accessories,6000,https://i.ebayimg.com/00/s/ODY3WDEzMDA=/z/zlgA...,"{'ks_classified': 'p/core_product_type', 'coun..."
1,126028451215,PGZvbnQgcndyPSIxIiBzdHlsZT0iIj48ZGl2IHN0eWxlPS...,"blue-point tools new 3/8"" drive 6mm metric hex...",None,2023-07-25,2024-09-25,179527,100,parts & accessories,6000,https://i.ebayimg.com/00/s/MTIwMFgxNjAw/z/dtcA...,"{'drive size': '3/8in.', 'ks_classified': 'p/c..."
2,134952985532,PCEtLSBEZXNjVGVtcGxhdGVJRDogRnJvbnQgQm9keSAoNj...,windshield wiper motor fits 95-96 ford e150 va...,None,2024-02-25,2024-09-25,262179,100,parts & accessories,6000,https://i.ebayimg.com/00/s/NDUwWDgwMA==/z/CEsA...,"{'mergednamespacenames': 'brand type', 'ks_cla..."
3,135211723255,PHN0eWxlPgogICAgQG1lZGlhIHNjcmVlbiBhbmQgKG1heC...,21'' front wheel rim & dual disc wheel hub fit...,None,2024-08-25,2024-09-25,147699,100,parts & accessories,6000,https://i.ebayimg.com/00/s/MTMwMFgxMzAw/z/FxoA...,"{'ks_classified': 'p/core_product_type', 'whee..."
4,144432285379,PCEtLSBCRUdJTiBWSURFTyAtLT4KPGRpdiBjbGFzcz0iZn...,hyper flash free led light 3156 amber two bulb...,None,2022-02-25,2024-09-25,172517,100,parts & accessories,6000,https://i.ebayimg.com/00/s/MTU5MlgxNjAw/z/D7oA...,"{'suitable for': 'daytime running light', 'per..."
...,...,...,...,...,...,...,...,...,...,...,...,...
9981,394481992200,CiAgPCFET0NUWVBFIGh0bWw+CjxodG1sIGxhbmc9ImVuIj...,for 2001-2004 mercedes slk320 camshaft positio...,None,2023-02-25,2024-09-25,33614,100,parts & accessories,6000,https://i.ebayimg.com/00/s/NDgwWDY0MA==/z/aTwA...,"{'ks_classified': 'p/core_product_type', 'manu..."
9982,395618655825,PG1ldGEgY2hhcnNldD0iVVRGLTgiPgo8bWV0YSBuYW1lPS...,"for ford maverick 2022-2024 4.5 ft 54.4"" truck...",None,2024-08-25,2024-09-25,33697,100,parts & accessories,6000,https://i.ebayimg.com/00/s/MTIwMFgxMjAw/z/FRYA...,"{'features': 'easy installation', 'ks_classifi..."
9983,402382727509,CgkJCQkJICA8IURPQ1RZUEUgaHRtbD4KPGh0bWwgbGFuZz...,front right anco contour wiper blade fits satu...,None,2020-08-25,2024-09-25,179852,100,parts & accessories,6000,https://i.ebayimg.com/00/s/NDgwWDY0MA==/z/ihQA...,"{'ks_classified': 'p/core_product_type', 'epti..."
9984,405123346459,PCEtLXRvbmd0b29sLXJlbGF0ZWRDc3NTdGFydC0tPjxsaW...,4 gang switch box emergency strobe light contr...,None,2024-07-25,2024-09-25,50459,100,parts & accessories,6000,https://i.ebayimg.com/00/s/MTYwMFgxNjAw/z/54IA...,"{'features': 'waterproof', 'ks_classified': 'p..."


In [91]:
# start = time.time()
# items_aspects_df_pna_us = extract_item_data(spark, start_date, end_date, business_vertical_names=business_vertical_names,
#                                             site_ids=site_ids,
#                                             n_items = n_items, take_shortest_dup_value=take_shortest_dup_value )
# items_aspects_pdf_pna_us = items_aspects_df_pna_us.toPandas()
# items_aspects_pdf_pna_us.to_parquet('data/items_aspects_pdf_pna_us.parquet')
# end = time.time()
# print(f"elapsed time (min): {(end - start)//60}")

Adding aspects
Adding descriptions


24/08/26 07:00:18 ERROR YarnScheduler: Lost executor 10814 on hdc42-mcc10-01-0110-5504-019-tess0173.stratus.rno.ebay.com: Executor decommission.
24/08/26 07:00:27 ERROR YarnScheduler: Lost executor 10572 on hdc42-mcc10-01-1110-3902-013-tess0029.stratus.rno.ebay.com: Executor decommission.
24/08/26 07:00:32 ERROR YarnScheduler: Lost executor 10093 on hdc42-mcc10-01-0110-5903-006-tess0173.stratus.rno.ebay.com: Executor decommission.
24/08/26 07:00:33 ERROR YarnScheduler: Lost executor 10228 on hdc42-mcc10-01-0110-5804-014-tess0173.stratus.rno.ebay.com: Executor decommission.
24/08/26 07:00:34 ERROR YarnScheduler: Lost executor 10877 on hdc42-mcc10-01-0110-5306-005-tess0173.stratus.rno.ebay.com: Executor decommission.
24/08/26 07:00:36 ERROR YarnScheduler: Lost executor 10416 on hdc42-mcc10-01-0710-3301-007-tess0035.stratus.rno.ebay.com: Executor decommission.
24/08/26 07:00:38 ERROR YarnScheduler: Lost executor 10088 on hdc42-mcc10-01-0210-7401-015-tess0131.stratus.rno.ebay.com: Executor

In [ ]:
items_aspects_pdf_collectb_us

In [55]:
items_aspects_df_pna_us

,ITEM_ID,item_desc,AUCT_TITL,SUBTITLE,AUCT_START_DT,AUCT_END_DT,leaf_categ_id,ITEM_SITE_ID,bsns_vrtcl_name,meta_categ_id,ASPECTS


24/08/25 13:58:20 ERROR YarnScheduler: Lost executor 7837 on hdc42-mcc10-01-0210-5401-012-tess0131.stratus.rno.ebay.com: Executor decommission.
24/08/25 13:58:20 ERROR YarnScheduler: Lost executor 7577 on hdc42-mcc10-01-0110-5107-013-tess0173.stratus.rno.ebay.com: Executor decommission.
24/08/25 13:58:20 ERROR YarnScheduler: Lost executor 7190 on hdc42-mcc10-01-0110-5902-001-tess0173.stratus.rno.ebay.com: Executor decommission.
24/08/25 13:58:20 ERROR YarnScheduler: Lost executor 7132 on hdc42-mcc10-01-0110-7305-005-tess0173.stratus.rno.ebay.com: Executor decommission.
24/08/25 13:58:20 ERROR YarnScheduler: Lost executor 7732 on hdc42-mcc10-01-0110-5103-016-tess0173.stratus.rno.ebay.com: Executor decommission.
24/08/25 13:58:20 ERROR YarnScheduler: Lost executor 6952 on hdc42-mcc10-01-0510-3401-025-tess0097.stratus.rno.ebay.com: Executor decommission.
24/08/25 13:58:20 ERROR YarnScheduler: Lost executor 7354 on hdc42-mcc10-01-0510-2104-018-tess0097.stratus.rno.ebay.com: Executor decomm

In [ ]:
## Examine None valued aspects in Collectibles 

## Extract H&G for site 0 (US)

In [8]:
start_date = '2024-09-25'
end_date = '2024-09-25'
business_vertical_names = ['Home & Garden']
site_ids = [0]
n_items = 10000
take_shortest_dup_value = False
leafcats=None
metacats=None

In [ ]:
start = time.time()
items_aspects_df_hag_us = extract_item_data(spark, start_date, end_date, business_vertical_names=business_vertical_names,
                                            site_ids=site_ids,
                                            n_items = n_items, take_shortest_dup_value=take_shortest_dup_value )
items_aspects_pdf_hag_us = items_aspects_df_hag_us.toPandas()
items_aspects_pdf_hag_us.to_parquet('data/Datasets/hag_pna_collct_09_24/RawExtracted/hag_us_09_27.parquet')
end = time.time()
print(f"elapsed time (min): {(end - start)//60}")

Adding aspects
Adding descriptions


## Add Image url

### Collectibles 

In [18]:
collectibles_pdf = pd.read_parquet('data/ExtractedNoSig/items_aspects_pdf_collectb_us_oct.parquet')
collectibles_item_ids = collectibles_pdf[['item_id']].rename(columns={'item_id': 'ITEM_ID'})
collectibles_df = spark.createDataFrame(collectibles_item_ids.to_dict(orient='records'))

24/09/03 18:19:40 ERROR YarnScheduler: Lost executor 158 on hdc42-mcc10-01-1110-1809-026-tess0029.stratus.rno.ebay.com: Executor decommission.


In [20]:
items_df = spark.table("ACCESS_VIEWS.DW_LSTG_ITEM").filter((F.col('auct_end_dt') >= start_date) &\
                                                           (F.col('auct_end_dt') <= end_date)).select('ITEM_ID', 'GALLERY_URL')
collectibles_df_with_image = collectibles_df.join(items_df, 'ITEM_ID', 'left').withColumnRenamed('GALLERY_URL', 'img_url')
collectibles_df_with_image

DataFrame[ITEM_ID: decimal(38,18), img_url: string]

In [21]:
collectibles_pdf_with_image = collectibles_df_with_image.toPandas()
collectibles_pdf_with_image

,ITEM_ID,img_url
0,116110059429.000000000000000000,https://i.ebayimg.com/00/s/MTYwMFgxMjAw/z/4N8A...
1,126630913258.000000000000000000,https://i.ebayimg.com/00/s/MTU1NVg5NDU=/z/3oAA...
2,135151947341.000000000000000000,https://i.ebayimg.com/00/s/OTA0WDEyMzc=/z/NpsA...
3,156260735987.000000000000000000,https://i.ebayimg.com/00/s/MTYwMFgxMTc4/z/Q5AA...
4,156261207194.000000000000000000,https://i.ebayimg.com/00/s/MTYwMFgxMjAw/z/1zIA...
...,...,...
4995,365015908836.000000000000000000,https://i.ebayimg.com/00/s/MTYwMFgxNjAw/z/47sA...
4996,365073049738.000000000000000000,https://i.ebayimg.com/00/s/MTYwMFgxNjAw/z/~GQA...
4997,375380530250.000000000000000000,https://i.ebayimg.com/00/s/MTYwMFgxMjAw/z/P~wA...
4998,385486869165.000000000000000000,https://i.ebayimg.com/00/s/MTYwMFg5NDM=/z/WRYA...


24/09/03 18:26:37 ERROR YarnScheduler: Lost executor 1014 on hdc42-mcc10-01-0210-5101-011-tess0131.stratus.rno.ebay.com: Executor decommission.
24/09/03 18:26:37 ERROR YarnScheduler: Lost executor 937 on hdc42-mcc10-01-0710-1708-021-tess0035.stratus.rno.ebay.com: Executor decommission.
24/09/03 18:26:37 ERROR YarnScheduler: Lost executor 1041 on hdc42-mcc10-01-1110-1708-004-tess0029.stratus.rno.ebay.com: Executor decommission.
24/09/03 18:26:37 ERROR YarnScheduler: Lost executor 862 on hdc42-mcc10-01-0110-7304-015-tess0173.stratus.rno.ebay.com: Executor decommission.
24/09/03 18:26:38 ERROR YarnScheduler: Lost executor 1123 on hdc42-mcc10-01-0110-5606-011-tess0173.stratus.rno.ebay.com: Executor decommission.
24/09/03 18:26:38 ERROR YarnScheduler: Lost executor 1276 on hdc42-mcc10-01-1110-1708-008-tess0029.stratus.rno.ebay.com: Executor decommission.
24/09/03 18:26:38 ERROR YarnScheduler: Lost executor 1365 on hdc42-mcc10-01-1110-4203-019-tess0029.stratus.rno.ebay.com: Executor decommis

In [24]:
collectibles_pdf_with_image.loc[105, 'img_url']

'https://i.ebayimg.com/00/s/Njk4WDkwMA==/z/B6YAAOSwx8RlV6Rf/$_1.JPG?set_id=880000500F'

In [26]:
collectibles_pdf_from_csv = pd.read_csv('data/DataWithSigs/gen_collct_us_oct_gpt_mistral.csv')

In [45]:
collectibles_pdf_with_image['item_id'] = collectibles_pdf_with_image['ITEM_ID'].astype(int)


In [52]:
collectibles_pdf_with_image.set_index('item_id_url', inplace=True)
collectibles_pdf_with_image

,ITEM_ID,img_url,item_id
item_id_url,,,
116110059429,116110059429.000000000000000000,https://i.ebayimg.com/00/s/MTYwMFgxMjAw/z/4N8A...,116110059429
126630913258,126630913258.000000000000000000,https://i.ebayimg.com/00/s/MTU1NVg5NDU=/z/3oAA...,126630913258
135151947341,135151947341.000000000000000000,https://i.ebayimg.com/00/s/OTA0WDEyMzc=/z/NpsA...,135151947341
156260735987,156260735987.000000000000000000,https://i.ebayimg.com/00/s/MTYwMFgxMTc4/z/Q5AA...,156260735987
156261207194,156261207194.000000000000000000,https://i.ebayimg.com/00/s/MTYwMFgxMjAw/z/1zIA...,156261207194
...,...,...,...
365015908836,365015908836.000000000000000000,https://i.ebayimg.com/00/s/MTYwMFgxNjAw/z/47sA...,365015908836
365073049738,365073049738.000000000000000000,https://i.ebayimg.com/00/s/MTYwMFgxNjAw/z/~GQA...,365073049738
375380530250,375380530250.000000000000000000,https://i.ebayimg.com/00/s/MTYwMFgxMjAw/z/P~wA...,375380530250


In [53]:
collectibles_pdf_from_csv.set_index('item_id', inplace=True)
collectibles_pdf_from_csv

,Unnamed: 0,item_desc,title,SUBTITLE,AUCT_START_DT,AUCT_END_DT,leaf_categ_id,ITEM_SITE_ID,bsns_vrtcl_name,meta_categ_id,GALLERY_URL,aspects,desc,gen_text_mistral,gen_sig_mistral,gen_text_gpt,gen_sig_gpt
item_id,,,,,,,,,,,,,,,,,
123979599749,0,PGZvbnQgcndyPSIxIiBzdHlsZT0iZm9udC1mYW1pbHk6IE...,"ripon cathedral north yorkshire, england great...",NaN,2019-11-17,2024-09-17,262042,0,collectibles,1,https://i.ebayimg.com/00/s/mtezmfgxnjaw/z/ftma...,"{'city': 'ripon', 'continent': 'europe', 'coun...","Hi, thanks for looking !!! This Postcard is co...","{""characteristic"": ""European city"", ""sale pitc...",European city,"{""characteristic"": ""unused postcard"", ""sale pi...",unused postcard
134658435740,1,PHN0eWxlPmltZ3ttYXgtd2lkdGg6MTAwJX08L3N0eWxlPj...,1971 topps football #174 adrian young nm *d3,NaN,2023-07-17,2024-09-17,261328,0,collectibles,64482,https://i.ebayimg.com/00/s/mte0nfg4ndq=/z/ey8a...,"{'autographed': 'no', 'brand': 'topps', 'card ...",1971 Topps Football #174 Adrian Young NM *d3 D...,"{""characteristic"": ""NM *d3 condition"", ""sale p...",NM *d3 condition,"{""characteristic"": ""Original 1971 Topps"", ""sal...",Original 1971 Topps
145560028063,2,PGI+UExFQVNFIExPT0sgQVQgVEhFIENPTkRJVElPTiBPRi...,#8 2000 winners circle dale earnhardt jr unmar...,NaN,2024-01-17,2024-09-17,261330,0,collectibles,64482,https://i.ebayimg.com/00/s/oty5wdc1mw==/z/axaa...,"{'core_product_type': 'sports trading cards', ...",PLEASE LOOK AT THE CONDITION OF THE CARD IN TH...,"{""characteristic"": ""sports trading cards"", ""sa...",sports trading cards,"{""characteristic"": ""2000 season"", ""sale pitch""...",2000 season
156013659941,3,PHA+MTk3MiBUb3BwcyAtIFRvbnkgUGVyZXogLSAgIzgwIC...,1972 topps - tony perez - #80 cincinnati reds,NaN,2024-01-17,2024-09-17,261328,0,collectibles,64482,https://i.ebayimg.com/00/s/mtmxm1g4mji=/z/pv0a...,"{'autographed': 'no', 'brand': 'topps', 'card ...",1972 Topps - Tony Perez - #80 - Cincinnati Red...,"{""characteristic"": ""1972 Topps"", ""sale pitch"":...",1972 Topps,"{""characteristic"": ""Card Condition: Very Good""...",Card Condition: Very Good
166873659153,4,PHAgZGlyPSJsdHIiIHN0eWxlPSJtYXJnaW4tdG9wOjA7IG...,"vintage estwing 16 oz. claw hammer 12.5"" inche...",NaN,2024-07-17,2024-09-17,39730,0,collectibles,1,https://i.ebayimg.com/00/s/mtywmfg1odu=/z/otya...,"{'brand': 'estwing', 'core_product_type': 'ham...",This vintage Estwing hammer made in Rockford I...,"{""characteristic"": ""vintage, made in Rockford""...","vintage, made in Rockford","{""characteristic"": ""vintage original"", ""sale p...",vintage original
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353816745693,4995,PHAgZGlyPSJsdHIiIHN0eWxlPSJtYXJnaW4tdG9wOjA7IG...,"daredevil #104 vol 2 (marvel, 2008) nm-",NaN,2021-12-17,2024-09-17,259104,0,collectibles,1,https://i.ebayimg.com/00/s/mtywmfgxmjaw/z/hsia...,"{'artist/writer': 'michael lark', 'character':...","Daredevil #104 Vol 2 (Marvel, 2008) NM-. Pleas...","{""characteristic"": ""michael lark art"", ""sale p...",michael lark art,"{""characteristic"": ""1st edition"", ""sale pitch""...",1st edition
354114504594,4996,PGZvbnQgcndyPSIxIiBzaXplPSI0IiBzdHlsZT0iZm9udC...,dc comics action comics #862 superman 2008 vf+...,NaN,2022-06-17,2024-09-17,259104,0,collectibles,1,https://i.ebayimg.com/00/s/mtywmfgxmjm2/z/py8a...,"{'artist/writer': 'jon sibal', 'character': 'l...","Hello , You are Bidding on the DC Comic Action...","{""characteristic"": ""Jon Sibal art"", ""sale pitc...",Jon Sibal art,"{""characteristic"": ""9.0 very fine/near mint"", ...",9.0 very fine/near mint
354593615047,4997,PCFET0NUWVBFIGh0bWw+CjxodG1sIGxhbmc9IiI+Cgo8aG...,2007-08 spx freshman tandems jerseys glen davi...,NaN,2023-02-17,2024-09-17,261328,0,collectibles,64482,https://i.ebayimg.com/00/s/oduwwdeyoda=/z/2vua...,"{'autographed': 'no', 'card condition': 'very ...",2007-08 SPx Freshman Tandems Jerseys Glen Davi...,"{""characteristic"": ""very good card condition"",...",very good card condition,"{""characteristic

In [58]:
collectibles_pdf_from_csv['img_url'] = collectibles_pdf_with_image['img_url']


GALLERY_URL    https://i.ebayimg.com/00/s/mtezmfgxnjaw/z/ftma...
img_url        https://i.ebayimg.com/00/s/MTEzMFgxNjAw/z/FtMA...
Name: 123979599749, dtype: object

In [60]:
collectibles_pdf_from_csv['GALLERY_URL'].iloc[0],\
collectibles_pdf_from_csv['img_url'].iloc[0]


('https://i.ebayimg.com/00/s/mtezmfgxnjaw/z/ftmaaosw6dtdrpdb/$_57.jpg?set_id=8800005007',
 'https://i.ebayimg.com/00/s/MTEzMFgxNjAw/z/FtMAAOSw6dtdrpDb/$_57.JPG?set_id=8800005007')

In [67]:
# collectibles_pdf_from_csv.reset_index(inplace=True)
# collectibles_pdf_from_csv.drop(columns = ['Unnamed: 0'], inplace=True)
collectibles_pdf_from_csv.index

RangeIndex(start=0, stop=4989, step=1)

In [ ]:
collectibles_pdf_from_csv.to_csv('data/DataWithSigs/gen_collct_us_oct_gpt_mistral_.csv')

In [46]:
len(set(collectibles_pdf_with_image['item_id']).intersection(set(collectibles_pdf_from_csv['item_id'])))

4989

In [47]:
collectibles_pdf_from_csv_img_url =  collectibles_pdf_from_csv.join(collectibles_pdf_with_image, 'item_id', 'left',  lsuffix='_left', rsuffix='_right')

In [48]:
collectibles_pdf_from_csv_img_url

,item_id,Unnamed: 0,item_id_left,item_desc,title,SUBTITLE,AUCT_START_DT,AUCT_END_DT,leaf_categ_id,ITEM_SITE_ID,...,aspects,desc,gen_text_mistral,gen_sig_mistral,gen_text_gpt,gen_sig_gpt,ITEM_ID,img_url,item_id_right,item_id_url
0,123979599749,0,123979599749,PGZvbnQgcndyPSIxIiBzdHlsZT0iZm9udC1mYW1pbHk6IE...,"ripon cathedral north yorkshire, england great...",NaN,2019-11-17,2024-09-17,262042,0,...,"{'city': 'ripon', 'continent': 'europe', 'coun...","Hi, thanks for looking !!! This Postcard is co...","{""characteristic"": ""European city"", ""sale pitc...",European city,"{""characteristic"": ""unused postcard"", ""sale pi...",unused postcard,NaN,NaN,NaN,NaN
1,134658435740,1,134658435740,PHN0eWxlPmltZ3ttYXgtd2lkdGg6MTAwJX08L3N0eWxlPj...,1971 topps football #174 adrian young nm *d3,NaN,2023-07-17,2024-09-17,261328,0,...,"{'autographed': 'no', 'brand': 'topps', 'card ...",1971 Topps Football #174 Adrian Young NM *d3 D...,"{""characteristic"": ""NM *d3 condition"", ""sale p...",NM *d3 condition,"{""characteristic"": ""Original 1971 Topps"", ""sal...",Original 1971 Topps,NaN,NaN,NaN,NaN
2,145560028063,2,145560028063,PGI+UExFQVNFIExPT0sgQVQgVEhFIENPTkRJVElPTiBPRi...,#8 2000 winners circle dale earnhardt jr unmar...,NaN,2024-01-17,2024-09-17,261330,0,...,"{'core_product_type': 'sports trading cards', ...",PLEASE LOOK AT THE CONDITION OF THE CARD IN TH...,"{""characteristic"": ""sports trading cards"", ""sa...",sports trading cards,"{""characteristic"": ""2000 season"", ""sale pitch""...",2000 season,NaN,NaN,NaN,NaN
3,156013659941,3,156013659941,PHA+MTk3MiBUb3BwcyAtIFRvbnkgUGVyZXogLSAgIzgwIC...,1972 topps - tony perez - #80 cincinnati reds,NaN,2024-01-17,2024-09-17,261328,0,...,"{'autographed': 'no', 'brand': 'topps', 'card ...",1972 Topps - Tony Perez - #80 - Cincinnati Red...,"{""characteristic"": ""1972 Topps"", ""sale pitch"":...",1972 Topps,"{""characteristic"": ""Card Condition: Very Good""...",Card Condition: Very Good,NaN,NaN,NaN,NaN
4,166873659153,4,166873659153,PHAgZGlyPSJsdHIiIHN0eWxlPSJtYXJnaW4tdG9wOjA7IG...,"vintage estwing 16 oz. claw hammer 12.5"" inche...",NaN,2024-07-17,2024-09-17,39730,0,...,"{'brand': 'estwing', 'core_product_type': 'ham...",This vintage Estwing hammer made in Rockford I...,"{""characteristic"": ""vintage, made in Rockford""...","vintage, made in Rockford","{""characteristic"": ""vintage original"", ""sale p...",vintage original,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4984,353816745693,4995,353816745693,PHAgZGlyPSJsdHIiIHN0eWxlPSJtYXJnaW4tdG9wOjA7IG...,"daredevil #104 vol 2 (marvel, 2008) nm-",NaN,2021-12-17,2024-09-17,259104,0,...,"{'artist/writer': 'michael lark', 'character':...","Daredevil #104 Vol 2 (Marvel, 2008) NM-. Pleas...","{""characteristic"": ""michael lark art"", ""sale p...",michael lark art,"{""characteristic"": ""1st edition"", ""sale pitch""...",1st edition,NaN,NaN,NaN,NaN
4985,354114504594,4996,354114504594,PGZvbnQgcndyPSIxIiBzaXplPSI0IiBzdHlsZT0iZm9udC...,dc comics action comics #862 superman 2008 vf+...,NaN,2022-06-17,2024-09-17,259104,0,...,"{'artist/writer': 'jon sibal', 'character': 'l...","Hello , You are Bidding on the DC Comic Action...","{""characteristic"": ""Jon Sibal art"", ""sale pitc...",Jon Sibal art,"{""characteristic"": ""9.0 very fine/near mint"", ...",9.0 very fine/near mint,NaN,NaN,NaN,NaN
4986,354593615047,4997,354593615047,PCFET0NUWVBFIGh0bWw+CjxodG1sIGxhbmc9IiI+Cgo8aG...,2007-08 spx freshman tandems jerseys glen davi...,NaN,2023-02-17,2024-09-17,261328,0,...,"{'autographed': 'no', 'card condition': 'very ...",2007-08 SPx Freshman Tandems Jerseys Glen Davi...,"{""characteristic"": ""very good card condition"",...",very good card condition,"{""characteristic"": ""black/green patch"", ""sale ...",black/green patch,NaN,NaN,NaN,NaN
4987,395538561456,4998,395538561456,SmFwYW5lc2UgYW5pbWUgS2lubmlrdW1hbiBmaWd1cmUgUm...,japanese anime kinnikuman figure rikishiman si...,NaN,2024-07-17,2024-09-17,69528,0,...,

In [ ]:
all(collectibles_pdf_from_csv['item_id'] == collectibles_pdf_with_image['item_id'])

KeyError: 'item_id'

In [10]:
start_date = '2024-09-02'
end_date = '2024-09-02'

In [11]:
collectibles_pdf = pd.read_csv('data/collectibles_us_with_sigs_mistral_gpt_lean.csv', index_col=0)
# collectibles_pdf['gen_sig_mistral'] = collectibles_pdf['gen_sig_mistral'].astype(str)
# collectibles_pdf['SUBTITLE'] = collectibles_pdf['SUBTITLE'].astype(str)
collectibles_item_ids = collectibles_pdf[['item_id']].rename(columns={'item_id': 'ITEM_ID'})

collectibles_df = spark.createDataFrame(collectibles_item_ids.to_dict(orient='records'))

In [12]:
collectibles_df

DataFrame[ITEM_ID: bigint]

In [39]:
collectibles_df_with_image

24/08/30 03:48:07 ERROR YarnScheduler: Lost executor 67950 on hdc42-mcc10-01-0210-7707-010-tess0131.stratus.rno.ebay.com: Executor decommission.
24/08/30 03:48:07 ERROR YarnScheduler: Lost executor 67936 on hdc42-mcc10-01-0910-1706-028-tess0028.stratus.rno.ebay.com: Executor decommission.
24/08/30 03:48:07 ERROR YarnScheduler: Lost executor 67913 on hdc42-mcc10-01-0110-7303-017-tess0173.stratus.rno.ebay.com: Executor decommission.


DataFrame[ITEM_ID: bigint, img_url: string]

24/08/30 03:48:07 ERROR YarnScheduler: Lost executor 67895 on hdc42-mcc10-01-0210-5903-005-tess0131.stratus.rno.ebay.com: Executor decommission.
24/08/30 03:48:07 ERROR YarnScheduler: Lost executor 67893 on hdc42-mcc10-01-0710-2108-014-tess0035.stratus.rno.ebay.com: Executor decommission.
24/08/30 03:48:07 ERROR YarnScheduler: Lost executor 67889 on hdc42-mcc10-01-1110-3305-004-tess0029.stratus.rno.ebay.com: Executor decommission.
24/08/30 03:48:07 ERROR YarnScheduler: Lost executor 67920 on hdc42-mcc10-01-0510-3401-014-tess0097.stratus.rno.ebay.com: Executor decommission.
24/08/30 03:48:07 ERROR YarnScheduler: Lost executor 67918 on hdc42-mcc10-01-0510-3602-032-tess0097.stratus.rno.ebay.com: Executor decommission.
24/08/30 03:48:07 ERROR YarnScheduler: Lost executor 67907 on hdc42-mcc10-01-1110-3609-039-tess0029.stratus.rno.ebay.com: Executor decommission.
24/08/30 03:48:07 ERROR YarnScheduler: Lost executor 67953 on hdc42-mcc10-01-1110-3608-037-tess0029.stratus.rno.ebay.com: Executor

In [13]:
items_df = spark.table("ACCESS_VIEWS.DW_LSTG_ITEM").filter((F.col('auct_end_dt') >= start_date) &\
                                                           (F.col('auct_end_dt') <= end_date)).select('ITEM_ID', 'GALLERY_URL')
collectibles_df_with_image = collectibles_df.join(items_df, 'ITEM_ID', 'left').withColumnRenamed('GALLERY_URL', 'img_url')

In [14]:
collectibles_df_with_image

DataFrame[ITEM_ID: bigint, img_url: string]

In [15]:
collectibles_pdf_with_image = collectibles_df_with_image.toPandas()
collectibles_pdf_with_image

,ITEM_ID,img_url
0,114664125925,https://i.ebayimg.com/00/s/MTYwMFgxMjAw/z/d8sA...
1,115755359434,https://i.ebayimg.com/00/s/MTYwMFgxMjAw/z/d2kA...
2,155236461306,https://i.ebayimg.com/00/s/MTU4NVg5NzQ=/z/eBAA...
3,155390411573,https://i.ebayimg.com/00/s/MTYwMFgxMjAw/z/IWIA...
4,175597284740,https://i.ebayimg.com/00/s/NDgwWDY0MA==/z/3VUA...
...,...,...
4995,355509021587,https://i.ebayimg.com/00/s/MTYwMFgxMjAw/z/UaQA...
4996,355761665374,https://i.ebayimg.com/00/s/NDQ5WDU5OQ==/z/t3AA...
4997,375408380381,https://i.ebayimg.com/00/s/MTYwMFgxNjAw/z/vpsA...
4998,394974214459,https://i.ebayimg.com/00/s/NTU3WDUzNA==/z/gsoA...


24/08/30 04:17:00 ERROR YarnScheduler: Lost executor 1082 on hdc42-mcc10-01-1110-1806-007-tess0029.stratus.rno.ebay.com: Executor decommission.
24/08/30 04:17:00 ERROR YarnScheduler: Lost executor 980 on hdc42-mcc10-01-0510-2104-034-tess0097.stratus.rno.ebay.com: Executor decommission.
24/08/30 04:17:00 ERROR YarnScheduler: Lost executor 1181 on hdc42-mcc10-01-0110-5804-006-tess0173.stratus.rno.ebay.com: Executor decommission.
24/08/30 04:17:00 ERROR YarnScheduler: Lost executor 895 on hdc42-mcc10-01-0110-7910-014-tess0173.stratus.rno.ebay.com: Executor decommission.
24/08/30 04:17:00 ERROR YarnScheduler: Lost executor 1341 on hdc42-mcc10-01-0510-3401-037-tess0097.stratus.rno.ebay.com: Executor decommission.
24/08/30 04:17:00 ERROR YarnScheduler: Lost executor 933 on hdc42-mcc10-01-0110-5504-012-tess0173.stratus.rno.ebay.com: Executor decommission.
24/08/30 04:17:00 ERROR YarnScheduler: Lost executor 975 on hdc42-mcc10-01-0210-7703-010-tess0131.stratus.rno.ebay.com: Executor decommissi

In [24]:
collectibles_pdf_with_image.rename(columns={'ITEM_ID': 'item_id'}, inplace=True)

In [20]:
all(collectibles_pdf_with_image['ITEM_ID'] == collectibles_pdf['item_id'])

True

In [25]:
collectibles_pdf['img_url'] = collectibles_pdf_with_image['img_url']

In [27]:
collectibles_pdf.to_csv('data/collectibles_us_with_sigs_mistral_gpt_lean.csv')

### P&A

In [52]:
pna_pdf = pd.read_csv('data/DataWithSigs/gen_pna_us_oct_gpt_mistral.csv')
pna_item_ids = pna_pdf[['item_id']].rename(columns={'item_id': 'ITEM_ID'})
pna_df = spark.createDataFrame(pna_item_ids.to_dict(orient='records'))

In [57]:
start_date = '2024-09-17'
end_date = '2024-09-17'
items_df = spark.table("ACCESS_VIEWS.DW_LSTG_ITEM").filter((F.col('auct_end_dt') >= start_date) &\
                                                           (F.col('auct_end_dt') <= end_date)).select('ITEM_ID', 'GALLERY_URL')
pna_df_with_image = pna_df.join(items_df, 'ITEM_ID', 'left').withColumnRenamed('GALLERY_URL', 'img_url')
pna_pdf_with_image = pna_df_with_image.toPandas()
pna_pdf_with_image

,ITEM_ID,img_url
0,133145078392,https://i.ebayimg.com/00/s/MTAwMFgxMDkx/z/ouwA...
1,145730116960,https://i.ebayimg.com/00/s/ODAwWDgwMA==/z/Aj8A...
2,145896526900,https://i.ebayimg.com/00/s/MTAwMFgxMDAw/z/BIIA...
3,154376682589,https://i.ebayimg.com/00/s/NDgwWDY0MA==/z/mOUA...
4,156171302116,https://i.ebayimg.com/00/s/NjUwWDExNTA=/z/IhEA...
...,...,...
4985,393800002843,https://i.ebayimg.com/00/s/MTEyMFgxMzUw/z/puEA...
4986,395069562156,https://i.ebayimg.com/00/s/MTAwMFgxMDAw/z/lL0A...
4987,395070130374,https://i.ebayimg.com/00/s/MTUwMFgxNTAw/z/hhUA...
4988,403415943912,https://i.ebayimg.com/00/s/NDgwWDY0MA==/z/qOYA...


24/09/04 09:33:09 ERROR YarnScheduler: Lost executor 1429 on hdc42-mcc10-01-0110-7305-021-tess0173.stratus.rno.ebay.com: Executor decommission.
24/09/04 09:33:11 ERROR YarnScheduler: Lost executor 1456 on hdc42-mcc10-01-1110-1808-018-tess0029.stratus.rno.ebay.com: Executor decommission.
24/09/04 09:33:12 ERROR YarnScheduler: Lost executor 1448 on hdc42-mcc10-01-0510-3702-024-tess0097.stratus.rno.ebay.com: Executor decommission.
24/09/04 09:33:18 ERROR YarnScheduler: Lost executor 1417 on hdc42-mcc10-01-0710-3508-009-tess0035.stratus.rno.ebay.com: Executor decommission.
24/09/04 09:33:18 ERROR YarnScheduler: Lost executor 1442 on hdc42-mcc10-01-0710-2106-020-tess0035.stratus.rno.ebay.com: Executor decommission.
24/09/04 09:33:18 ERROR YarnScheduler: Lost executor 1486 on hdc42-mcc10-01-0910-1709-014-tess0028.stratus.rno.ebay.com: Executor decommission.
24/09/04 09:33:18 ERROR YarnScheduler: Lost executor 1461 on hdc42-mcc10-01-1110-3903-001-tess0029.stratus.rno.ebay.com: Executor decomm

In [66]:
pna_pdf['item_id']
pna_pdf_with_image['ITEM_ID']

0       133145078392
1       145730116960
2       145896526900
3       154376682589
4       156171302116
            ...     
4985    393800002843
4986    395069562156
4987    395070130374
4988    403415943912
4989    404630452043
Name: ITEM_ID, Length: 4990, dtype: int64

In [67]:
all(pna_pdf['item_id'] == pna_pdf_with_image['ITEM_ID'])

True

In [68]:
pna_pdf_with_image

,ITEM_ID,img_url
0,133145078392,https://i.ebayimg.com/00/s/MTAwMFgxMDkx/z/ouwA...
1,145730116960,https://i.ebayimg.com/00/s/ODAwWDgwMA==/z/Aj8A...
2,145896526900,https://i.ebayimg.com/00/s/MTAwMFgxMDAw/z/BIIA...
3,154376682589,https://i.ebayimg.com/00/s/NDgwWDY0MA==/z/mOUA...
4,156171302116,https://i.ebayimg.com/00/s/NjUwWDExNTA=/z/IhEA...
...,...,...
4985,393800002843,https://i.ebayimg.com/00/s/MTEyMFgxMzUw/z/puEA...
4986,395069562156,https://i.ebayimg.com/00/s/MTAwMFgxMDAw/z/lL0A...
4987,395070130374,https://i.ebayimg.com/00/s/MTUwMFgxNTAw/z/hhUA...
4988,403415943912,https://i.ebayimg.com/00/s/NDgwWDY0MA==/z/qOYA...


In [69]:
pna_pdf['img_url'] = pna_pdf_with_image['img_url']

In [72]:
pna_pdf.drop(columns=['Unnamed: 0'], inplace=True)
pna_pdf

,item_id,item_desc,title,SUBTITLE,AUCT_START_DT,AUCT_END_DT,leaf_categ_id,ITEM_SITE_ID,bsns_vrtcl_name,meta_categ_id,GALLERY_URL,aspects,desc,gen_text_mistral,gen_sig_mistral,gen_text_gpt,gen_sig_gpt,img_url
0,133145078392,PGh0bWw+CiAgPGhlYWQ+CiAgICA8bWV0YSBuYW1lPSJ2aW...,jnc 026 black machined face 18x8 5x100 +35 whe...,authorized jnc dealer. 100% authentic. free sh...,2019-08-17,2024-09-17,179679,100,parts & accessories,6000,https://i.ebayimg.com/00/s/mtawmfgxmdkx/z/ouwa...,"{'bolt pattern': '5x100', 'brand': 'jnc', 'col...",Our eBay Store Contact Us Add to Favorite Sell...,"{""characteristic"": ""5x100 bolt pattern"", ""sale...",5x100 bolt pattern,"{""characteristic"": ""73.1 centerbore"", ""sale pi...",73.1 centerbore,https://i.ebayimg.com/00/s/MTAwMFgxMDkx/z/ouwA...
1,145730116960,PCEtLVBBNF9IVE1MVG9wX0JlZ2luLS0+PGRpdiBjbGFzcz...,05175684aa a/c pressure transducer switch for ...,NaN,2024-04-17,2024-09-17,33543,100,parts & accessories,6000,https://i.ebayimg.com/00/s/odawwdgwma==/z/aj8a...,"{'compressor drive type': 'belt driven', 'epti...",Part number: 05174039AB 5174039AB 05174039AA 5...,"{""characteristic"": ""belt driven"", ""sale pitch""...",belt driven,"{""characteristic"": ""Lifetime warranty"", ""sale ...",Lifetime warranty,https://i.ebayimg.com/00/s/ODAwWDgwMA==/z/Aj8A...
2,145896526900,PGZyb290ZW1wbGF0ZW5hbWUgZGF0YT0iY3VzdG9tX3RlbX...,fuel d808 hurricane wheel & nitto trail grappl...,"lt295/55r20 | 20x10 -18 et | 6x139.7, 6x5.5",2024-07-17,2024-09-17,179684,100,parts & accessories,6000,https://i.ebayimg.com/00/s/mtawmfgxmdaw/z/biia...,"{'alphanumeric tire code': 'mt', 'aspect ratio...",CLICK HERE for full item description Powered b...,"{""characteristic"": ""mt tires"", ""sale pitch"": ""...",mt tires,"{""characteristic"": ""professionally mounted"", ""...",professionally mounted,https://i.ebayimg.com/00/s/MTAwMFgxMDAw/z/BIIA...
3,154376682589,CgkJCQkJICA8IURPQ1RZUEUgaHRtbD4KPGh0bWwgbGFuZz...,front wheel stud for 1998-2002 mazda b3000 4wd...,NaN,2021-03-17,2024-09-17,33749,100,parts & accessories,6000,https://i.ebayimg.com/00/s/ndgwwdy0ma==/z/moua...,"{'brand': 'dorman', 'brand type': 'aftermarket...",Front Wheel Stud For 1998-2002 Mazda B3000 4WD...,"{""characteristic"": ""aftermarket brand: Dorman""...",aftermarket brand: Dorman,"{""characteristic"": ""carbon steel material"", ""s...",carbon steel material,https://i.ebayimg.com/00/s/NDgwWDY0MA==/z/mOUA...
4,156171302116,PCEtLSBIVE1MIC0tPgo8IS0tCkNvcHlyaWdodCDCqTIwMT...,"fuel tech beadlock black 15"" wheels 32"" motova...",this kit is ready-to-ride with free mounting &...,2024-04-17,2024-09-17,263051,100,parts & accessories,6000,https://i.ebayimg.com/00/s/njuwwdexnta=/z/ihea...,"{'ks_classified': 'p/core_product_type', 'mach...","FREE SHIPPING ON ORDERS $99 AND OVER, TO LOWER...","{""characteristic"": ""FREE SHIPPING"", ""sale pitc...",FREE SHIPPING,"{""characteristic"": ""FREE SHIPPING"", ""sale pitc...",FREE SHIPPING,https://i.ebayimg.com/00/s/NjUwWDExNTA=/z/IhEA...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4985,393800002843,CgkJCQkJICA8IURPQ1RZUEUgaHRtbD4KPGh0bWwgbGFuZz...,for 1991-1995 gmc sonoma-premium engine coolan...,NaN,2021-12-17,2024-09-17,33603,100,parts & accessories,6000,https://i.ebayimg.com/00/s/mteymfgxmzuw/z/puea...,"{'brand': 'gates', 'brand type': 'aftermarket ...",eBay Listing Template Search Required Parts Vi...,"{""characteristic"": ""aftermarket Gates brand"", ...",aftermarket Gates brand,"{""characteristic"": ""upgraded thermal valve"", ""...",upgraded thermal valve,https://i.ebayimg.com/00/s/MTEyMFgxMzUw/z/puEA...
4986,395069562156,PCEtLVBBNF9IVE1MVG9wX0JlZ2luLS0+PGRpdiBjbGFzcz...,for ford f-150 2015-2020 black wood grain cons...,NaN,2023-12-17,2024-09-17,33705,100,parts & accessories,6000,https://i.ebayimg.com/00/s/mtawmfgxmdaw/z/ll0a...,"{'brand': 'unbranded', 'brand type': 'unbrande...",Condition: 100% Brand New Color: As Pictures E...,"{""characteristic"": ""100% Brand New"", ""sale pit...",100% Bran

In [74]:
pna_pdf.to_csv('data/DataWithSigs/gen_pna_us_oct_gpt_mistral_with_img_url.csv')

In [76]:
len("country/region of manufacture: andorra")

38

In [29]:
pna_pdf = pd.read_csv('data/pna_us_with_sigs_mistral_gpt_lean.csv', index_col=0)
pna_item_ids = pna_pdf[['item_id']].rename(columns={'item_id': 'ITEM_ID'})
pna_df = spark.createDataFrame(pna_item_ids.to_dict(orient='records'))
pna_df

DataFrame[ITEM_ID: bigint]

# Breakdown

## Part 1

In [15]:

start_date = '2024-08-20'
end_date = '2024-08-20'
leafcats=[165193, 165194]
items_df = spark.table("ACCESS_VIEWS.DW_LSTG_ITEM").filter((F.col('auct_end_dt') >= start_date) &\
                                                           (F.col('auct_end_dt') <= end_date))
#TODO: there's no meta_categ_id in the table. Fix this
# if metacats is not None:
#     items_df = items_df.filter(F.col('meta_categ_id').isin(metacats))
if leafcats is not None:
    items_df = items_df.filter(F.col('leaf_categ_id').isin(leafcats))

items_df = items_df.select('ITEM_ID', 'AUCT_START_DT', 'AUCT_END_DT',  'leaf_categ_id' )#.cache()

description_df = spark.table("ACCESS_VIEWS.LSTG_ITEM_DESC_DIM").filter((F.col('auct_end_dt') >= start_date) &\
                                                            (F.col('auct_end_dt') <= end_date)).\
                                                            select('item_id', 'item_desc')
aspects_df = spark.table("ACCESS_VIEWS.ITEM_ASPCT_CLSSFCTN_SAP").filter((F.col('auct_end_dt') >= start_date) &\
                                                            (F.col('auct_end_dt') <= end_date))
print("Adding aspects")

Adding aspects


### Execution times tests 

In [16]:
start = time.time()
items_df.count()
end = time.time()
print(f"elapsed time (min): {(end - start)//60}")

elapsed time (min): 4.0


In [ ]:
start = time.time()
aspects_df.count()
end = time.time()
print(f"elapsed time (min): {(end - start)//60}")

## Part 2 - breakdwon of collect_item_aspects

In [ ]:
start = time.time()
df_items = items_df
df_aspects = aspects_df
df_item_aspects = df_aspects.join(df_items, on=['ITEM_ID', 'AUCT_END_DT'])
df_item_aspects.write.parquet('hdfs://apollo-rno/user/b_perso/mmandelbrod/df_item_aspects_from_krylov.parquet')
# print(df_item_aspects.count())
end = time.time()
print(f"elapsed time (min): {(end - start)//60}")

In [22]:
df_item_aspects

DataFrame[item_id: decimal(18,0), auct_end_dt: date, prdct_aspct_nm: string, aspct_vlu_nm: string, ns_type_cd: string, aspct_src: int, load_ts: string, key_type: tinyint, AUCT_START_DT: date, leaf_categ_id: decimal(9,0)]

In [24]:
df_item_aspects_pdf = pd.read_parquet('data/df_item_aspects_from_krylov.parquet/')
df_item_aspects_pdf

,item_id,auct_end_dt,prdct_aspct_nm,aspct_vlu_nm,ns_type_cd,aspct_src,load_ts,key_type,AUCT_START_DT,leaf_categ_id
0,334673485722,2024-08-20,Original/Reproduction,Original,umns,101,2024-07-20 20:02:45,1,2022-12-20,165194
1,334673485722,2024-08-20,type,Gas Pump,sdner,160,2024-07-20 20:02:45,1,2022-12-20,165194
2,334673485722,2024-08-20,core_product_type_v3,Collectibles,preco,228,2024-07-20 20:02:45,1,2022-12-20,165194
3,334673485722,2024-08-20,Brand,John Deere,aenv,3,2024-07-20 20:02:45,1,2022-12-20,165194
4,334673485722,2024-08-20,material,Metal Tin,dft7,197,2024-07-20 20:02:45,1,2022-12-20,165194
...,...,...,...,...,...,...,...,...,...,...
4265,266914324092,2024-08-20,coreOntologyVisited,1723686740749,sd,193,2024-08-14 18:50:14,1,2024-07-20,165194
4266,266914324092,2024-08-20,status,TRAFFIC_FILTERED_OUT,sd,268,2024-08-14 18:50:14,1,2024-07-20,165194
4267,266914324092,2024-08-20,categoryId,165194,sdner,160,2024-08-14 18:50:14,1,2024-07-20,165194
4268,266914324092,2024-08-20,productTypeModelAnnotatorVisited,1723686740311,sd,228,2024-08-14 18:50:14,1,2024-07-20,165194


### Examine specific listing (285868885432)

In [26]:
df_item_aspects_pdf['item_id'].value_counts()

item_id
364431820792    408
156130055263    395
364852534006    171
334567323247    170
276474900499    165
305363174949    161
305150783870    154
276474911494    151
387020042694    149
405173910460    147
285868885432    147
235018424150    147
334673485722    146
266914319338    146
145902363635    146
305727308320    145
266948032540    143
266914324092    142
403422040982    139
264248084497    138
114077191682    136
335450451587    136
145950472855    130
126623884354    128
226011945904    120
166834125389    110
Name: count, dtype: int64

In [28]:
df_285868885432 = df_item_aspects_pdf[df_item_aspects_pdf['item_id'] == 285868885432]
df_285868885432[['prdct_aspct_nm', 'aspct_vlu_nm']]

,prdct_aspct_nm,aspct_vlu_nm
3270,Original/Reproduction,Original
3271,productTypeAttributesVisited,1723657608427
3272,CatLogPriceSD,0.7095790387600255
3273,Original/Reproduction,Original
3274,Brand,John Deere
...,...,...
3412,DescSummary,stamps : all stamps are mint never hinged.
3413,QryItemSimEmbA,jTEOquYmn7H/Ky6wL6+YrT8x9ij/r1curDAwMPQdeyY4sa...
3414,DescSummary,the 1981 john deere employees credit union ban...
3415,ItemSimEmbA,H60JLJQx4jDrK6Cu7a+uMq8tnChGLEwphrJpG7qmoyAYsQ...


In [29]:
df_285868885432['prdct_aspct_nm'].value_counts()

prdct_aspct_nm
DescSummary              15
Brand                    11
Theme                     9
Original/Reproduction     7
skip                      6
                         ..
CatLogPriceMean           1
rlogId                    1
status                    1
descHash                  1
MarketPriceConfidence     1
Name: count, Length: 97, dtype: int64

In [30]:
[len(a) for a in df_285868885432['aspct_vlu_nm']]

[8,
 13,
 18,
 8,
 10,
 8,
 78,
 8,
 7,
 11,
 13,
 10,
 13,
 11,
 17,
 20,
 10,
 7,
 6,
 256,
 13,
 11,
 4,
 10,
 13,
 10,
 10,
 19,
 4,
 13,
 8,
 13,
 7,
 10,
 18,
 13,
 12,
 11,
 12,
 11,
 18,
 1,
 19,
 29,
 13,
 32,
 20,
 72,
 18,
 22,
 2,
 7,
 18,
 19,
 19,
 4,
 8,
 4,
 41,
 29,
 10,
 7,
 10,
 4,
 22,
 18,
 11,
 13,
 7,
 29,
 18,
 10,
 24,
 10,
 7,
 18,
 2,
 77,
 6,
 1,
 4,
 88,
 256,
 4,
 3,
 4,
 34,
 256,
 34,
 34,
 13,
 18,
 23,
 6,
 37,
 10,
 256,
 9,
 83,
 78,
 24,
 2,
 32,
 344,
 3,
 101,
 50,
 62,
 13,
 3,
 8,
 60,
 1,
 10,
 18,
 256,
 81,
 2,
 34,
 0,
 112,
 34,
 32,
 3,
 8,
 11,
 78,
 256,
 6,
 1,
 18,
 4,
 2,
 6,
 4,
 4,
 34,
 18,
 19,
 34,
 34,
 31,
 42,
 256,
 147,
 256,
 5]

In [31]:
df_285868885432[df_285868885432['prdct_aspct_nm'] == 'DescSummary']

,item_id,auct_end_dt,prdct_aspct_nm,aspct_vlu_nm,ns_type_cd,aspct_src,load_ts,key_type,AUCT_START_DT,leaf_categ_id
3347,285868885432,2024-08-20,DescSummary,the bank was created in 1981 and is a true rep...,cu,163,2024-08-16 07:57:15,2,2024-05-20,165194
3351,285868885432,2024-08-20,DescSummary,the best way to assess condition is either pro...,cu,163,2024-08-16 07:57:15,2,2024-05-20,165194
3362,285868885432,2024-08-20,DescSummary,all stamps are genuine.,cu,163,2024-08-16 07:57:15,2,2024-05-20,165194
3364,285868885432,2024-08-20,DescSummary,coins : coins are graded differently.,cu,163,2024-08-16 07:57:15,2,2024-05-20,165194
3368,285868885432,2024-08-20,DescSummary,this is a rare and unique collectible for any ...,cu,163,2024-08-16 07:57:15,2,2024-05-20,165194
3370,285868885432,2024-08-20,DescSummary,comes with original box.,cu,163,2024-08-16 07:57:15,2,2024-05-20,165194
3375,285868885432,2024-08-20,DescSummary,"challenge coins : are all metal, beautifully h...",cu,163,2024-08-16 07:57:15,2,2024-05-20,165194
3376,285868885432,2024-08-20,DescSummary,i price in accordance with the chip rack crite...,cu,163,2024-08-16 07:57:15,2,2024-05-20,165194
3377,285868885432,2024-08-20,DescSummary,the model d was the first tractor to bear the ...,cu,163,2024-08-16 07:57:15,2,2024-05-20,165194
3381,285868885432,2024-08-20,DescSummary,"the bank measures 4 1/2"" wide x 4 1/2"" tall an...",cu,163,2024-08-16 07:57:15,2,2024-05-20,165194


In [32]:
df_285868885432[df_285868885432['prdct_aspct_nm'] == 'Brand']

,item_id,auct_end_dt,prdct_aspct_nm,aspct_vlu_nm,ns_type_cd,aspct_src,load_ts,key_type,AUCT_START_DT,leaf_categ_id
3274,285868885432,2024-08-20,Brand,John Deere,sdattr,102,2024-08-14 10:32:18,1,2024-05-20,165194
3281,285868885432,2024-08-20,Brand,John Deere,df,3,2024-08-14 10:32:18,1,2024-05-20,165194
3286,285868885432,2024-08-20,Brand,John Deere,df,30,2024-08-14 10:32:18,1,2024-05-20,165194
3293,285868885432,2024-08-20,Brand,John Deere,vdf,156,2024-08-14 10:32:18,1,2024-05-20,165194
3296,285868885432,2024-08-20,Brand,John Deere,ae,3,2024-08-14 10:32:18,1,2024-05-20,165194
3330,285868885432,2024-08-20,Brand,John Deere,umns,101,2024-08-14 10:32:18,1,2024-05-20,165194
3332,285868885432,2024-08-20,Brand,john deere,da,267,2024-08-14 10:32:18,1,2024-05-20,165194
3341,285868885432,2024-08-20,Brand,John Deere,aenv,3,2024-08-14 10:32:18,1,2024-05-20,165194
3343,285868885432,2024-08-20,Brand,John Deere,df,101,2024-08-14 10:32:18,1,2024-05-20,165194
3346,285868885432,2024-08-20,Brand,jd,da,240,2024-08-14 10:32:18,1,2024-05-20,165194


In [33]:
df_285868885432[df_285868885432['prdct_aspct_nm'] == 'Theme']

,item_id,auct_end_dt,prdct_aspct_nm,aspct_vlu_nm,ns_type_cd,aspct_src,load_ts,key_type,AUCT_START_DT,leaf_categ_id
3278,285868885432,2024-08-20,Theme,farming,da,240,2024-08-14 10:32:18,1,2024-05-20,165194
3279,285868885432,2024-08-20,Theme,Agriculture,umns,101,2024-08-14 10:32:18,1,2024-05-20,165194
3283,285868885432,2024-08-20,Theme,Agriculture,df,101,2024-08-14 10:32:18,1,2024-05-20,165194
3291,285868885432,2024-08-20,Theme,Agriculture,sdattr,102,2024-08-14 10:32:18,1,2024-05-20,165194
3306,285868885432,2024-08-20,Theme,agricultural,da,240,2024-08-14 10:32:18,1,2024-05-20,165194
3307,285868885432,2024-08-20,Theme,Agriculture,aenv,3,2024-08-14 10:32:18,1,2024-05-20,165194
3309,285868885432,2024-08-20,Theme,Agriculture,df,3,2024-08-14 10:32:18,1,2024-05-20,165194
3336,285868885432,2024-08-20,Theme,Agriculture,vdf,156,2024-08-14 10:32:18,1,2024-05-20,165194
3395,285868885432,2024-08-20,Theme,Agriculture,vdf,185,2024-08-16 07:57:15,2,2024-05-20,165194


In [34]:
valid_df = df_285868885432[df_285868885432['ns_type_cd'] == 'df']
len(valid_df)

16

In [36]:
valid_df['prdct_aspct_nm'].value_counts()

prdct_aspct_nm
Brand                    3
Original/Reproduction    2
Theme                    2
core_product_type_v2     2
CatRecoID_1              1
CatRecoScore_1           1
MiscatScore              1
CatRecoID_2              1
CatRecoScore_3           1
CatRecoID_3              1
CatRecoScore_2           1
Name: count, dtype: int64

In [37]:
valid_df[valid_df['prdct_aspct_nm'] == 'core_product_type_v2']

,item_id,auct_end_dt,prdct_aspct_nm,aspct_vlu_nm,ns_type_cd,aspct_src,load_ts,key_type,AUCT_START_DT,leaf_categ_id
3328,285868885432,2024-08-20,core_product_type_v2,Piggy Banks & Money Jars: primary product,df,228,2024-08-14 10:32:18,1,2024-05-20,165194
3339,285868885432,2024-08-20,core_product_type_v2,Collectibles: primary product,df,228,2024-08-14 10:32:18,1,2024-05-20,165194


In [45]:
items_df = spark.table("ACCESS_VIEWS.DW_LSTG_ITEM")
items_df.columns
df_items = df_items.select('ITEM_ID', "AUCT_END_DT").dropDuplicates()
df_items = df_items.select('ITEM_ID', "AUCT_END_DT").dropDuplicates()
df_items.count()

24/08/17 15:18:28 ERROR TaskSetManager: Task 3 in stage 17.0 failed 4 times; aborting job


Py4JJavaError: An error occurred while calling o879.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 17.0 failed 4 times, most recent failure: Lost task 3.3 in stage 17.0 (TID 102010) (hdc42-mcc10-01-1110-3305-034-tess0029.stratus.rno.ebay.com executor 764): java.io.FileNotFoundException: File does not exist: /sys/edw/gdw_tables/lstg/dw_lstg_item/snapshot/dt=20240814/part-00003-e4ebfdf4-8f65-48b4-b7e0-63c54849bd24_00003.c000.snappy.parquet
It is possible the underlying files have been updated. You can explicitly invalidate the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by recreating the Dataset/DataFrame involved.
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:132)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:177)
	at org.apache.spark.sql.execution.da

In [35]:
[f for f in items_df.columns if 'meta' in f]

[]

In [ ]:
.filter((F.col(filter_colname) >= start_date) &\
                                                               (F.col(filter_colname) <= end_date))

In [12]:
items_df.columns

['ITEM_ID', 'AUCT_START_DT', 'AUCT_END_DT']

In [16]:
items_df_cat.select('leaf_categ_id').dtypes

[('leaf_categ_id', 'decimal(9,0)')]

In [18]:
items_df_cat = items_df.filter((F.col(filter_colname) >= start_date) &\
                (F.col(filter_colname) <= end_date) &\
                (F.col('leaf_categ_id') == 165193)
               )
items_df_cat.count()




24/08/15 17:09:40 ERROR YarnScheduler: Lost executor 144 on hdc42-mcc10-01-1110-4205-037-tess0029.stratus.rno.ebay.com: Container from a bad node: container_e4084_1723088988367_616827_02_000047 on host: hdc42-mcc10-01-1110-4205-037-tess0029.stratus.rno.ebay.com. Exit status: 143. Diagnostics: [2024-08-15 10:09:38.251]Container killed on request. Exit code is 143
[2024-08-15 10:09:38.251]Container exited with a non-zero exit code 143. 
[2024-08-15 10:09:38.256]Killed by external signal
.
24/08/15 17:12:56 ERROR YarnScheduler: Lost executor 260 on hdc42-mcc10-01-1110-1902-033-tess0029.stratus.rno.ebay.com: Executor heartbeat timed out after 641648 ms


1

In [24]:
items_df_cat = items_df.filter((F.col(filter_colname) >= start_date) &\
                (F.col(filter_colname) <= end_date) &\
                (F.col('leaf_categ_id').isin([165193, 165194]))
               )
items_df_cat.count()


7

24/08/16 08:18:21 ERROR YarnScheduler: Lost executor 659 on hdc42-mcc10-01-1110-1808-013-tess0029.stratus.rno.ebay.com: Executor decommission.
24/08/16 08:18:22 ERROR YarnScheduler: Lost executor 686 on hdc42-mcc10-01-1110-2104-018-tess0029.stratus.rno.ebay.com: Executor decommission.
24/08/16 08:18:22 ERROR YarnScheduler: Lost executor 681 on hdc42-mcc10-01-0510-3708-034-tess0097.stratus.rno.ebay.com: Executor decommission.
24/08/16 08:18:22 ERROR YarnScheduler: Lost executor 732 on hdc42-mcc10-01-0110-7504-002-tess0173.stratus.rno.ebay.com: Executor decommission.
24/08/16 08:18:22 ERROR YarnScheduler: Lost executor 678 on hdc42-mcc10-01-0210-6002-027-tess0131.stratus.rno.ebay.com: Executor decommission.
24/08/16 08:18:23 ERROR YarnScheduler: Lost executor 712 on hdc42-mcc10-01-0510-1703-033-tess0097.stratus.rno.ebay.com: Executor decommission.
24/08/16 08:18:23 ERROR YarnScheduler: Lost executor 710 on hdc42-mcc10-01-0210-5405-021-tess0131.stratus.rno.ebay.com: Executor decommission.

In [22]:
items_df_cat_pdf = items_df_cat.select('ITEM_ID', 'AUCT_START_DT', 'AUCT_END_DT', 'leaf_categ_id' ).toPandas()
items_df_cat_pdf

,ITEM_ID,AUCT_START_DT,AUCT_END_DT,leaf_categ_id
0,116241526281,2024-07-05,2024-07-15,165193


In [23]:
items_df_cat_pdf

,ITEM_ID,AUCT_START_DT,AUCT_END_DT,leaf_categ_id
0,116241526281,2024-07-05,2024-07-15,165193


24/08/15 19:28:24 ERROR YarnScheduler: Lost executor 654 on hdc42-mcc10-01-1110-3404-028-tess0029.stratus.rno.ebay.com: Executor decommission.
24/08/15 19:28:26 ERROR YarnScheduler: Lost executor 611 on hdc42-mcc10-01-0110-7102-022-tess0173.stratus.rno.ebay.com: Executor decommission.
24/08/15 19:28:27 ERROR YarnScheduler: Lost executor 644 on hdc42-mcc10-01-0710-3607-010-tess0035.stratus.rno.ebay.com: Executor decommission.
24/08/15 19:28:27 ERROR YarnScheduler: Lost executor 591 on hdc42-mcc10-01-1110-1904-019-tess0029.stratus.rno.ebay.com: Executor decommission.
24/08/15 19:28:27 ERROR YarnScheduler: Lost executor 602 on hdc42-mcc10-01-0110-7704-030-tess0173.stratus.rno.ebay.com: Executor decommission.
24/08/15 19:28:28 ERROR YarnScheduler: Lost executor 615 on hdc42-mcc10-01-0210-7401-018-tess0131.stratus.rno.ebay.com: Executor decommission.
24/08/15 19:28:28 ERROR YarnScheduler: Lost executor 646 on hdc42-mcc10-01-1110-4104-025-tess0029.stratus.rno.ebay.com: Executor decommission.

In [10]:
# items_df.agg({"auct_start_dt": "max"}).collect()[0][0]
# items_df.select('auct_start_dt')

datetime.date(2024, 7, 15)

24/08/13 12:17:14 ERROR YarnScheduler: Lost executor 172 on hdc42-mcc10-01-0510-3807-018-tess0097.stratus.rno.ebay.com: Executor decommission.
24/08/13 12:17:14 ERROR YarnScheduler: Lost executor 105 on hdc42-mcc10-01-0210-7702-022-tess0131.stratus.rno.ebay.com: Executor decommission.
24/08/13 12:17:15 ERROR YarnScheduler: Lost executor 149 on hdc42-mcc10-01-0110-7305-014-tess0173.stratus.rno.ebay.com: Executor decommission.
24/08/13 12:17:15 ERROR YarnScheduler: Lost executor 94 on hdc42-mcc10-01-1110-3803-017-tess0029.stratus.rno.ebay.com: Executor decommission.
24/08/13 12:17:15 ERROR YarnScheduler: Lost executor 131 on hdc42-mcc10-01-1110-4206-034-tess0029.stratus.rno.ebay.com: Executor decommission.
24/08/13 12:17:16 ERROR YarnScheduler: Lost executor 148 on hdc42-mcc10-01-0710-4201-012-tess0035.stratus.rno.ebay.com: Executor decommission.
24/08/13 12:17:16 ERROR YarnScheduler: Lost executor 122 on hdc42-mcc10-01-0710-3606-017-tess0035.stratus.rno.ebay.com: Executor decommission.


In [ ]:
df_items_aspects = collect_item_aspects(spark, items_df, aspects_df)
pdf = df_items_aspects.limit(10).toPandas()
pdf

24/08/13 13:10:48 ERROR YarnScheduler: Lost executor 67 on hdc42-mcc10-01-0510-3902-028-tess0097.stratus.rno.ebay.com: Container container_e4084_1723088988367_504684_01_000121 on host: hdc42-mcc10-01-0510-3902-028-tess0097.stratus.rno.ebay.com was preempted.
24/08/13 13:10:48 ERROR YarnScheduler: Lost executor 58 on hdc42-mcc10-01-0510-1702-027-tess0097.stratus.rno.ebay.com: Container container_e4084_1723088988367_504684_01_000098 on host: hdc42-mcc10-01-0510-1702-027-tess0097.stratus.rno.ebay.com was preempted.
24/08/13 13:10:48 ERROR YarnScheduler: Lost executor 76 on hdc42-mcc10-01-1110-3701-006-tess0029.stratus.rno.ebay.com: Container container_e4084_1723088988367_504684_01_000130 on host: hdc42-mcc10-01-1110-3701-006-tess0029.stratus.rno.ebay.com was preempted.
24/08/13 13:10:48 ERROR YarnScheduler: Lost executor 71 on hdc42-mcc10-01-0510-1904-036-tess0097.stratus.rno.ebay.com: Container container_e4084_1723088988367_504684_01_000125 on host: hdc42-mcc10-01-0510-1904-036-tess0097.

In [15]:
pdf

NameError: name 'pdf' is not defined

In [17]:
items_df_small = items_df.limit(100)
items_df_small.show()

+------------+-------------+-----------+
|     ITEM_ID|AUCT_START_DT|AUCT_END_DT|
+------------+-------------+-----------+
|114171507568|   2020-04-04| 2024-07-15|
|264737223329|   2020-05-31| 2024-07-15|
|114253382847|   2020-06-08| 2024-07-15|
|265992204016|   2022-11-15| 2024-07-15|
|114556217875|   2020-12-03| 2024-07-15|
|266354362529|   2023-07-31| 2024-07-15|
|114939239865|   2021-08-16| 2024-07-15|
|266413441323|   2023-09-15| 2024-07-15|
|115282510241|   2022-03-06| 2024-07-15|
|266428280489|   2023-09-27| 2024-07-15|
|115286808023|   2022-03-09| 2024-07-15|
|266470702445|   2023-10-24| 2024-07-15|
|115316392788|   2022-03-28| 2024-07-15|
|266481299944|   2023-10-30| 2024-07-15|
|115350724452|   2022-04-19| 2024-07-15|
|266568565468|   2023-12-15| 2024-07-15|
|115370939363|   2022-05-04| 2024-07-15|
|266568566045|   2023-12-15| 2024-07-15|
|115434223495|   2022-06-20| 2024-07-15|
|266619729680|   2024-01-15| 2024-07-15|
+------------+-------------+-----------+
only showing top

24/08/14 10:14:56 ERROR YarnScheduler: Lost executor 354 on hdc42-mcc10-01-0110-7802-015-tess0173.stratus.rno.ebay.com: Executor decommission.
24/08/14 10:14:56 ERROR YarnScheduler: Lost executor 265 on hdc42-mcc10-01-0210-5302-040-tess0131.stratus.rno.ebay.com: Executor decommission.
24/08/14 10:14:56 ERROR YarnScheduler: Lost executor 330 on hdc42-mcc10-01-0510-1702-023-tess0097.stratus.rno.ebay.com: Executor decommission.
24/08/14 10:14:57 ERROR YarnScheduler: Lost executor 342 on hdc42-mcc10-01-1110-3908-040-tess0029.stratus.rno.ebay.com: Executor decommission.
24/08/14 10:14:57 ERROR YarnScheduler: Lost executor 306 on hdc42-mcc10-01-0210-7707-019-tess0131.stratus.rno.ebay.com: Executor decommission.
24/08/14 10:14:57 ERROR YarnScheduler: Lost executor 293 on hdc42-mcc10-01-0210-6002-002-tess0131.stratus.rno.ebay.com: Executor decommission.
24/08/14 10:14:57 ERROR YarnScheduler: Lost executor 334 on hdc42-mcc10-01-0510-3902-019-tess0097.stratus.rno.ebay.com: Executor decommission.

## Create a small dataset to verify the flow

# Listings Description

In [4]:
desc_table = spark.table('ACCESS_VIEWS.LSTG_ITEM_DESC_DIM')


In [5]:
desc_table.columns

['item_id',
 'site_id',
 'auct_end_date',
 'auct_end_dt',
 'auct_start_date',
 'auct_start_dt',
 'last_mdfd_date',
 'slr_id',
 'auct_type_cd',
 'item_desc_lgth_num',
 'item_desc']

In [6]:
df_desc = desc_table.limit(10)
df_desc.show()


+------------+-------+-------------------+-----------+-------------------+-------------+-------------------+----------+------------+------------------+--------------------+
|     item_id|site_id|      auct_end_date|auct_end_dt|    auct_start_date|auct_start_dt|     last_mdfd_date|    slr_id|auct_type_cd|item_desc_lgth_num|           item_desc|
+------------+-------+-------------------+-----------+-------------------+-------------+-------------------+----------+------------+------------------+--------------------+
|113461454118|     77|2020-02-10 17:59:10| 2020-02-10|2018-12-17 20:22:40|   2018-12-17|2020-02-10 17:59:13| 699778905|           9|               182|PGZvbnQgcndyPScxJ...|
|392494212579|      3|2020-02-10 12:38:51| 2020-02-10|2019-10-22 15:01:14|   2019-10-22|2020-02-10 12:38:55| 110322925|           9|              3439|Cjx0YWJsZSBzdHlsZ...|
|113492150933|      3|2020-02-10 23:34:12| 2020-02-10|2018-12-26 16:57:38|   2018-12-26|2020-02-10 23:34:15|1144129463|           7|   

24/08/11 15:32:10 ERROR YarnScheduler: Lost executor 7 on hdc42-mcc10-01-1110-3706-030-tess0029.stratus.rno.ebay.com: Executor decommission.
24/08/11 15:32:11 ERROR YarnScheduler: Lost executor 18 on hdc42-mcc10-01-0210-7401-001-tess0131.stratus.rno.ebay.com: Executor decommission.
24/08/11 15:32:11 ERROR YarnScheduler: Lost executor 48 on hdc42-mcc10-01-0110-7701-008-tess0173.stratus.rno.ebay.com: Executor decommission.
24/08/11 15:32:11 ERROR YarnScheduler: Lost executor 47 on hdc42-mcc10-01-0110-7905-019-tess0173.stratus.rno.ebay.com: Executor decommission.
24/08/11 15:32:11 ERROR YarnScheduler: Lost executor 9 on hdc42-mcc10-01-0710-2106-012-tess0035.stratus.rno.ebay.com: Executor decommission.
24/08/11 15:32:11 ERROR YarnScheduler: Lost executor 8 on hdc42-mcc10-01-0910-1909-012-tess0028.stratus.rno.ebay.com: Executor decommission.
24/08/11 15:32:11 ERROR YarnScheduler: Lost executor 56 on hdc42-mcc10-01-0110-7810-007-tess0173.stratus.rno.ebay.com: Executor decommission.
24/08/11 

In [8]:
pdf_desc = df_desc.select('item_id', 'item_desc').toPandas()
pdf_desc

,item_id,item_desc
0,113673839609,PGZvbnQgcndyPSIxIiBzaXplPSI0IiBzdHlsZT0iZm9udC...
1,392274802748,PGRpdiBzdHlsZT0iY2xlYXI6Ym90aDttYXJnaW46NXB4ID...
2,113686208582,PGZvbnQgcndyPScxJyBzaXplPSc0JyBzdHlsZT0nZm9udC...
3,392321880284,PGRpdiBzdHlsZT0ibWFyZ2luOjBweDtwYWRkaW5nOjBweD...
4,113776653894,PG1ldGEgaHR0cC1lcXVpdj0iQ29udGVudC1UeXBlIiBjb2...
5,392337142501,Cjx0YWJsZSBzdHlsZT0id2lkdGg6IDEwMCU7IiBib3JkZX...
6,113798915150,PG1ldGEgY29udGVudD0idGV4dC9odG1sOyBjaGFyc2V0PX...
7,392337183163,Cjx0YWJsZSBzdHlsZT0id2lkdGg6IDEwMCU7IiBib3JkZX...
8,113866414224,PGZvbnQgcndyPSIxIiBzaXplPSI0IiBzdHlsZT0iZm9udC...
9,392372612344,PCFkb2N0eXBlIGh0bWw+PCEtLSBPcmlnaW5hbCBkZXNpZ2...


In [20]:
import base64

encoded_str = pdf_desc.iloc[0,1]
print(encoded_str)

print(html_to_plain(encoded_str))


# print(encoded_str)
# print()
# # Decode the Base64 string
# decoded_bytes = base64.b64decode(encoded_str)

# # Convert the decoded bytes to a string
# decoded_str = decoded_bytes.decode('utf-8')

# # Print the decoded string
# print(decoded_str)

PGZvbnQgcndyPSIxIiBzaXplPSI0IiBzdHlsZT0iZm9udC1mYW1pbHk6QXJpYWwiPjxmb250IGZhY2U9IkFyaWFsIj48c3BhbiBzdHlsZT0iZm9udC1zaXplOiAxOC42NjY3cHg7Ij48Yj5HZW51aW5lIEhQIDUzQSBCbGFjayBMYXNlckpldCBUb25lciBDYXJ0cmlkZ2UgKFE3NTUzQSk8L2I+PC9zcGFuPjwvZm9udD48ZGl2Pjxmb250IGZhY2U9IkFyaWFsIj48c3BhbiBzdHlsZT0iZm9udC1zaXplOiAxOC42NjY3cHg7Ij48Yj48YnI+PC9iPjwvc3Bhbj48L2ZvbnQ+PC9kaXY+PGRpdj48Zm9udCBmYWNlPSJBcmlhbCI+PHNwYW4gc3R5bGU9ImZvbnQtc2l6ZTogMTguNjY2N3B4OyI+PGI+T3JpZ2luYWwgUGFja2FnaW5nLiBTZWFsZWQ8L2I+PC9zcGFuPjwvZm9udD48L2Rpdj48ZGl2Pjxmb250IGZhY2U9IkFyaWFsIj48c3BhbiBzdHlsZT0iZm9udC1zaXplOiAxOC42NjY3cHg7Ij48Yj48YnI+PC9iPjwvc3Bhbj48L2ZvbnQ+PC9kaXY+PGRpdj48Zm9udCBmYWNlPSJBcmlhbCI+PHNwYW4gc3R5bGU9ImZvbnQtc2l6ZTogMTguNjY2N3B4OyI+PGI+U3VycGx1cyZuYnNwOzwvYj48L3NwYW4+PC9mb250PjwvZGl2PjwvZm9udD4=



In [22]:
html = encoded_str
parsed = ''
if html is None or len(html) == 0:
    print(1)
html = base64.b64decode(html).decode('utf-8')
try:
    bs = BeautifulSoup(html, 'html.parser')
    plain = bs.get_text(separator=' ', strip=True)
    parsed =  re.sub(r"\s+", " ", plain)
except:
    print(2)

2


In [23]:
html

'<font rwr="1" size="4" style="font-family:Arial"><font face="Arial"><span style="font-size: 18.6667px;"><b>Genuine HP 53A Black LaserJet Toner Cartridge (Q7553A)</b></span></font><div><font face="Arial"><span style="font-size: 18.6667px;"><b><br></b></span></font></div><div><font face="Arial"><span style="font-size: 18.6667px;"><b>Original Packaging. Sealed</b></span></font></div><div><font face="Arial"><span style="font-size: 18.6667px;"><b><br></b></span></font></div><div><font face="Arial"><span style="font-size: 18.6667px;"><b>Surplus&nbsp;</b></span></font></div></font>'

In [24]:
bs = BeautifulSoup(html, 'html.parser')
bs

<font rwr="1" size="4" style="font-family:Arial"><font face="Arial"><span style="font-size: 18.6667px;"><b>Genuine HP 53A Black LaserJet Toner Cartridge (Q7553A)</b></span></font><div><font face="Arial"><span style="font-size: 18.6667px;"><b><br/></b></span></font></div><div><font face="Arial"><span style="font-size: 18.6667px;"><b>Original Packaging. Sealed</b></span></font></div><div><font face="Arial"><span style="font-size: 18.6667px;"><b><br/></b></span></font></div><div><font face="Arial"><span style="font-size: 18.6667px;"><b>Surplus </b></span></font></div></font>

In [27]:
plain = bs.get_text(separator=' ', strip=True)
plain

'Genuine HP 53A Black LaserJet Toner Cartridge (Q7553A) Original Packaging. Sealed Surplus'

In [30]:
parsed =  re.sub(r"\s+", " ", plain)
parsed

'Genuine HP 53A Black LaserJet Toner Cartridge (Q7553A) Original Packaging. Sealed Surplus'

24/08/12 10:09:22 ERROR YarnScheduler: Lost executor 343 on hdc42-mcc10-01-0510-2203-015-tess0097.stratus.rno.ebay.com: Executor decommission.
24/08/12 10:09:33 ERROR YarnScheduler: Lost executor 283 on hdc42-mcc10-01-0910-2208-005-tess0028.stratus.rno.ebay.com: Executor decommission.
24/08/12 10:09:59 ERROR YarnScheduler: Lost executor 320 on hdc42-mcc10-01-1110-4105-007-tess0029.stratus.rno.ebay.com: Executor decommission.
24/08/12 10:10:17 ERROR YarnScheduler: Lost executor 329 on hdc42-mcc10-01-0110-7103-011-tess0173.stratus.rno.ebay.com: Executor decommission.
24/08/12 10:10:33 ERROR YarnScheduler: Lost executor 321 on hdc42-mcc10-01-0510-3401-026-tess0097.stratus.rno.ebay.com: Executor decommission.


In [ ]:
df_desc.select('').unbase64(DESCRIPTION)

# Aspects

In [28]:
df_aspects = spark.table('ACCESS_VIEWS.ITEM_ASPCT_CLSSFCTN_SAP')
# df_aspects_394275915987 = df_aspects.filter(F.col('item_id') == '394275915987')
# df_aspects_394275915987.show()


In [30]:
aspects_items_table = collect_item_aspects(spark, lst_table)

NameError: name 'Window' is not defined

In [ ]:
aspects_items_table

24/08/12 13:21:09 ERROR YarnScheduler: Lost executor 95 on hdc42-mcc10-01-0210-5405-014-tess0131.stratus.rno.ebay.com: Executor decommission.
24/08/12 13:21:10 ERROR YarnScheduler: Lost executor 64 on hdc42-mcc10-01-0210-6002-021-tess0131.stratus.rno.ebay.com: Executor decommission.
24/08/12 13:21:11 ERROR YarnScheduler: Lost executor 94 on hdc42-mcc10-01-0710-4201-019-tess0035.stratus.rno.ebay.com: Executor decommission.
24/08/12 13:21:11 ERROR YarnScheduler: Lost executor 48 on hdc42-mcc10-01-0210-7405-036-tess0131.stratus.rno.ebay.com: Executor decommission.
24/08/12 13:21:11 ERROR YarnScheduler: Lost executor 101 on hdc42-mcc10-01-0110-5210-014-tess0173.stratus.rno.ebay.com: Executor decommission.
24/08/12 13:21:11 ERROR YarnScheduler: Lost executor 57 on hdc42-mcc10-01-0110-7910-009-tess0173.stratus.rno.ebay.com: Executor decommission.
24/08/12 13:21:11 ERROR YarnScheduler: Lost executor 69 on hdc42-mcc10-01-0910-1706-019-tess0028.stratus.rno.ebay.com: Executor decommission.
24/08

In [29]:
df_aspects.columns

['item_id',
 'prdct_aspct_nm',
 'aspct_vlu_nm',
 'ns_type_cd',
 'aspct_src',
 'auct_end_dt',
 'load_ts',
 'key_type']

In [ ]:
df_aspects_394275915987.show()

24/08/12 11:34:34 ERROR YarnScheduler: Lost executor 326 on hdc42-mcc10-01-1110-3503-005-tess0029.stratus.rno.ebay.com: Executor decommission.
24/08/12 11:34:39 ERROR YarnScheduler: Lost executor 344 on hdc42-mcc10-01-1110-1903-012-tess0029.stratus.rno.ebay.com: Executor decommission.
24/08/12 11:34:43 ERROR YarnScheduler: Lost executor 385 on hdc42-mcc10-01-0210-8007-005-tess0131.stratus.rno.ebay.com: Executor decommission.
24/08/12 11:34:44 ERROR YarnScheduler: Lost executor 347 on hdc42-mcc10-01-1110-1808-007-tess0029.stratus.rno.ebay.com: Executor decommission.
24/08/12 11:34:48 ERROR YarnScheduler: Lost executor 375 on hdc42-mcc10-01-0510-3903-009-tess0097.stratus.rno.ebay.com: Executor decommission.
24/08/12 11:34:53 ERROR YarnScheduler: Lost executor 371 on hdc42-mcc10-01-1110-1707-040-tess0029.stratus.rno.ebay.com: Executor decommission.
24/08/12 11:34:59 ERROR YarnScheduler: Lost executor 324 on hdc42-mcc10-01-0210-5405-008-tess0131.stratus.rno.ebay.com: Executor decommission.

# DW_LSTG_ITEM - getting to Know

In [84]:
lst_table = spark.sql("SELECT * from ACCESS_VIEWS.DW_LSTG_ITEM  where AUCT_END_DT == '2024-09-15' limit 100")

In [27]:
lst_table.columns

['item_id',
 'auct_end_dt',
 'auct_start_dt',
 'sale_sched_end_dt',
 'auct_type_code',
 'item_site_id',
 'item_cntry_id',
 'lstg_curncy_id',
 'lstg_status_id',
 'icon_flags',
 'slr_id',
 'slr_site_id',
 'slr_cntry_id',
 'high_bdr_id',
 'leaf_categ_id',
 'leaf_categ_id2',
 'auct_start_exchng_rate',
 'auct_end_exchng_rate',
 'start_price_lstg_curncy',
 'rsrv_price_list_crncy',
 'bin_price_lstg_curncy',
 'curnt_price_lstg_curncy',
 'gms_lstg_curncy',
 'gms_lstg_curncy_est',
 'qty_sold',
 'qty_avail',
 'bidcount',
 'wacko_yn',
 'success_yn',
 'stor_lstg_yn_flag_id',
 'ebx_elgbl_item_yn_id',
 'mr_lister_flag',
 'slng_mgr_sbscrbr_flag',
 'slng_mgr_pro_sbscrbr_flag',
 'slng_mgr_pro_used_up_flag',
 'bin_sold_up_flag',
 'gtc_up_flag',
 'paypal_accepted_up_flag',
 'ck_fetr_up_flag',
 'mrch_tool_up_flag',
 'syi_2_up_flag',
 'charity_giving_tool_up_flag',
 'b2b_up_flag',
 'slr_paypal_prefd_up_flag',
 'categ_slng_guide_used_up_flag',
 'relist_up_flag',
 'relist_crd_up_flag',
 'listed_with_autopay_u

In [86]:
urls = lst_table.select('ITEM_ID', 'GALLERY_URL').limit(10).toPandas()
urls

,ITEM_ID,GALLERY_URL
0,404924405113,https://i.ebayimg.com/00/s/MTYwMFgxNjAw/z/Cz8A...
1,260769676787,http://i.ebayimg.com/00/s/MTAyNFg3Njg=/z/jl4AA...
2,404924448917,https://i.ebayimg.com/00/s/NjAwWDYwMA==/z/WzsA...
3,261113935609,https://i.ebayimg.com/00/s/MFgyMTQ3NDgzNjQ3/z/...
4,404924488088,https://i.ebayimg.com/00/s/MTIwMFgxNjAw/z/aFUA...
5,261354957995,https://i.ebayimg.com/00/s/MTA2NlgxNjAw/z/evEA...
6,404924621871,https://i.ebayimg.com/00/s/ODAwWDgwMA==/z/G6QA...
7,261663045415,https://i.ebayimg.com/00/s/MTYwMFg5Mzk=/z/m0MA...
8,404924623869,https://i.ebayimg.com/00/s/ODAwWDgwMA==/z/dagA...
9,262094772364,https://i.ebayimg.com/00/s/MTIwMFgxNjAw/z/lzkA...


In [88]:
urls.loc[7, 'GALLERY_URL']

'https://i.ebayimg.com/00/s/MTYwMFg5Mzk=/z/m0MAAOSw-jhUAziB/$_1.JPG?set_id=880000500F'

In [36]:
max_auct_end_dt = lst_table.agg({"AUCT_END_DT": "max"}).collect()[0][0]
min_auct_end_dt = lst_table.agg({"AUCT_END_DT": "min"}).collect()[0][0]
print(max_auct_end_dt), print(min_auct_end_dt)

2024-07-15
2024-07-15


(None, None)

In [37]:
lst_table.select('item_id', 'CATEG_KEY').show()

+------------+---------+
|     item_id|CATEG_KEY|
+------------+---------+
|115062604995| 21376871|
|263328937392| 21894383|
|115198771881| 19844981|
|265714156006| 22133436|
|115286991200| 19844984|
|265990369782| 22949709|
|115370820019| 21149898|
|266340888328| 17763061|
|115434786256| 21149898|
|266354360149| 21116838|
|115476830873| 19844984|
|266428257407| 22670824|
|115479808484| 21099585|
|266428257932| 20683186|
|115588365228| 21149900|
|266569179989| 22382538|
|115612426318| 21099585|
|266697422027| 12212173|
|115646140332| 21375886|
|266715576516| 22382477|
+------------+---------+
only showing top 20 rows



In [ ]:
# lst_data = lst_table.select('AUCT_TITL', 'SUBTITLE', 

In [4]:
lst_table.columns

['item_id',
 'auct_end_dt',
 'auct_start_dt',
 'sale_sched_end_dt',
 'auct_type_code',
 'item_site_id',
 'item_cntry_id',
 'lstg_curncy_id',
 'lstg_status_id',
 'icon_flags',
 'slr_id',
 'slr_site_id',
 'slr_cntry_id',
 'high_bdr_id',
 'leaf_categ_id',
 'leaf_categ_id2',
 'auct_start_exchng_rate',
 'auct_end_exchng_rate',
 'start_price_lstg_curncy',
 'rsrv_price_list_crncy',
 'bin_price_lstg_curncy',
 'curnt_price_lstg_curncy',
 'gms_lstg_curncy',
 'gms_lstg_curncy_est',
 'qty_sold',
 'qty_avail',
 'bidcount',
 'wacko_yn',
 'success_yn',
 'stor_lstg_yn_flag_id',
 'ebx_elgbl_item_yn_id',
 'mr_lister_flag',
 'slng_mgr_sbscrbr_flag',
 'slng_mgr_pro_sbscrbr_flag',
 'slng_mgr_pro_used_up_flag',
 'bin_sold_up_flag',
 'gtc_up_flag',
 'paypal_accepted_up_flag',
 'ck_fetr_up_flag',
 'mrch_tool_up_flag',
 'syi_2_up_flag',
 'charity_giving_tool_up_flag',
 'b2b_up_flag',
 'slr_paypal_prefd_up_flag',
 'categ_slng_guide_used_up_flag',
 'relist_up_flag',
 'relist_crd_up_flag',
 'listed_with_autopay_u

24/08/07 13:04:11 ERROR YarnScheduler: Lost executor 2 on hdc42-mcc10-01-0910-2007-002-tess0028.stratus.rno.ebay.com: Container container_e4083_1722988178968_47304_01_000007 on host: hdc42-mcc10-01-0910-2007-002-tess0028.stratus.rno.ebay.com was preempted.


In [32]:
lst_table.count()

NameError: name 'lst_table' is not defined

# Categories

In [69]:
cats_table = spark.table('ACCESS_VIEWS.DW_CATEGORY_GROUPINGS')
cats_table = cats_table.withColumn('upd_date', F.col('upd_date').cast("string"))
cats_table

DataFrame[leaf_categ_id: decimal(9,0), leaf_categ_name: string, site_id: decimal(9,0), categ_busn_mgr: string, categ_busn_unit: string, regn_categ: string, user_defined_field1: string, user_defined_field3: string, cre_date: date, upd_date: string, cre_user: string, upd_user: string, meta_categ_id: decimal(9,0), meta_categ_name: string, categ_lvl2_id: decimal(9,0), categ_lvl3_id: decimal(9,0), categ_lvl4_id: decimal(9,0), categ_lvl5_id: decimal(9,0), categ_lvl6_id: decimal(9,0), categ_lvl7_id: decimal(9,0), categ_lvl2_name: string, categ_lvl3_name: string, categ_lvl4_name: string, categ_lvl5_name: string, categ_lvl6_name: string, categ_lvl7_name: string, categ_flags: decimal(12,0), adult_categ_yn: string, domain_id: decimal(9,0), user_defined_field5: string, vcs_id: decimal(12,0), gcs_id: decimal(12,0), move_to: decimal(9,0), sap_category_id: decimal(9,0), src_id: int, bsns_vrtcl_name: string]

In [52]:
cats_table.count()

711113

In [72]:
cats_table.select('bsns_vrtcl_name').distinct().show()

+--------------------+
|     bsns_vrtcl_name|
+--------------------+
|            Vehicles|
|             Fashion|
|Business & Indust...|
|         Real Estate|
|         Electronics|
|       Home & Garden|
|               Media|
|             Unknown|
|        Collectibles|
|           Lifestyle|
| Parts & Accessories|
+--------------------+



In [75]:
cats_table.filter(F.col('bsns_vrtcl_name') == 'Vehicles').select('site_id').distinct().show()

+-------+
|site_id|
+-------+
|      0|
|     -3|
|    196|
|     77|
|    212|
|    201|
|    203|
|    -15|
|     71|
|    193|
|    215|
|    146|
|    -16|
|    207|
|     -2|
|    -17|
|    224|
|    100|
|    123|
|    210|
+-------+
only showing top 20 rows



24/08/25 18:50:55 ERROR YarnScheduler: Lost executor 7881 on hdc42-mcc10-01-0710-1708-008-tess0035.stratus.rno.ebay.com: Executor decommission.
24/08/25 18:50:55 ERROR YarnScheduler: Lost executor 7539 on hdc42-mcc10-01-1110-3708-013-tess0029.stratus.rno.ebay.com: Executor decommission.


## Examine meta_categ_id == 131090

In [77]:
metacat_pdf = cats_table.filter(F.col('meta_categ_id') == 131090).toPandas()
metacat_pdf

,leaf_categ_id,leaf_categ_name,site_id,categ_busn_mgr,categ_busn_unit,regn_categ,user_defined_field1,user_defined_field3,cre_date,upd_date,...,categ_flags,adult_categ_yn,domain_id,user_defined_field5,vcs_id,gcs_id,move_to,sap_category_id,src_id,bsns_vrtcl_name
0,75208,Auto e moto: ricambi e accessori:Golf car: ric...,101,None,None,None,24,Oggetti sportivi,2008-06-03,2018-08-01 01:44:26,...,262144,N,-999,Sporting Goods,-999,-999,75208,24,1,Lifestyle
1,34210,Auto's en motoren: onderdelen en accessoires:A...,123,None,None,None,3,Boeken en tijdschriften,2018-07-31,2020-10-06 23:46:17,...,262144,N,-999,Books,-999,-999,34210,3,1,Media
2,14778,Auto's & Motoren:Handleidingen & Tijdschriften...,123,None,6000,None,3,Boeken en tijdschriften,2008-06-03,2020-10-06 23:46:17,...,0,N,-999,Books,-999,-999,34210,3,1,Media
3,180496,"Auto, moto - pièces, accessoires:Tuning, styli...",23,None,None,None,24,Sports dequipment,2016-07-11,2019-06-11 04:29:27,...,262144,N,-999,Sporting Goods,-999,-999,180496,24,1,Lifestyle
4,9825,Auto's & Motoren:Automobilia & Fansport:Formule 1,123,None,None,None,11,Verzamelen,2008-06-03,2020-10-06 23:46:17,...,0,N,-999,Collectibles,-999,-999,91985,11,1,Collectibles
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44419,263517,Auto & Motorrad: Teile:Kfz-Services & -Reparat...,77,None,None,None,8,Teile & Zubehör für Autos,2024-07-16,2024-07-16 10:02:34.643,...,0,N,-999,Auto - Parts,-999,-999,263517,8,1,Parts & Accessories
44420,263516,Auto & Motorrad: Teile:Kfz-Services & -Reparat...,77,None,None,None,8,Teile & Zubehör für Autos,2024-07-16,2024-07-16 10:02:34.643,...,0,N,-999,Auto - Parts,-999,-999,263516,8,1,Parts & Accessories
44421,263514,Auto & Motorrad: Teile:Kfz-Services & -Reparat...,77,None,None,None,8,Teile & Zubehör für Autos,2024-07-16,2024-07-16 10:02:34.643,...,0,N,-999,Auto - Parts,-999,-999,263514,8,1,Parts & Accessories
44422,263522,Auto & Motorrad: Teile:Kfz-Services & -Reparat...,77,None,None,None,8,Teile & Zubehör für Autos,2024-07-16,2024-07-16 10:02:34.643,...,0,N,-999,Auto - Parts,-999,-999,263522,8,1,Parts & Accessories


In [80]:
metacat_pdf['meta_categ_name'].value_counts()

meta_categ_name
Auto & Motorrad: Teile                          19242
Vehicle Parts & Accessories                      8532
Motoryzacja: Części                              4541
Auto, moto - pièces, accessoires                 4456
Auto's en motoren: onderdelen en accessoires     3767
Auto e moto: ricambi e accessori                 2021
Motor: piezas y accesorios                       1720
Automotive                                        139
Véhicules: pièces, accessoires                      4
Voertuigonderdelen                                  2
Name: count, dtype: int64

In [78]:
metacat_pdf['site_id'].value_counts()

site_id
77     6464
193    6389
16     6389
212    4541
3      3037
205    3016
71     2581
15     2479
101    2021
146    1892
23     1879
123    1877
186    1720
203     139
Name: count, dtype: int64

In [79]:
metacat_pdf['bsns_vrtcl_name'].value_counts()

bsns_vrtcl_name
Parts & Accessories      43950
Electronics                165
Vehicles                   152
Fashion                     66
Business & Industrial       49
Collectibles                20
Lifestyle                    9
Media                        7
Home & Garden                6
Name: count, dtype: int64

## Examine site_id 0

In [40]:
cats_pdf_site_0 = cats_table.filter(F.col('site_id') == 0).toPandas()
cats_pdf_site_0

,leaf_categ_id,leaf_categ_name,site_id,categ_busn_mgr,categ_busn_unit,regn_categ,user_defined_field1,user_defined_field3,cre_date,upd_date,...,categ_flags,adult_categ_yn,domain_id,user_defined_field5,vcs_id,gcs_id,move_to,sap_category_id,src_id,bsns_vrtcl_name
0,11430,Books:Children:Activity Books:Coloring Books,0,None,267,None,3,Books,2008-06-03,2021-05-12 00:38:41.875,...,0,N,-999,Books,-999,-999,261186,3,1,Media
1,29345,Books:Nonfiction:Military & War:Specific Wars:...,0,None,None,None,3,Books,2008-06-03,2021-05-12 00:38:41.875,...,0,N,-999,Books,-999,-999,261186,3,1,Media
2,29347,Books:Nonfiction:Military & War:Specific Wars:...,0,None,None,None,3,Books,2008-06-03,2021-05-12 00:38:41.875,...,0,N,-999,Books,-999,-999,261186,3,1,Media
3,46337,Books:Magazines & Catalogs:Magazine Back Issue...,0,None,None,None,3,Books,2008-06-03,2021-05-12 00:38:41.875,...,0,N,-999,Books,-999,-999,280,3,1,Media
4,378,Books:Nonfiction,0,None,267,Other,3,Books,2008-06-03,2021-05-12 00:38:41.875,...,262144,N,-999,Books,1388,1127,261186,3,1,Media
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45660,108575,"Cell Phones & PDAs:Accessories, Parts:Batterie...",0,None,None,None,40,Cell Phones & Portable Electro,2008-06-03,2015-06-02 02:46:53,...,262144,N,-999,Cell Phones & Accessories,1786,1441,42425,40,1,Electronics
45661,263510,Sporting Goods:Hunting:Gun Parts:Firearm Magaz...,0,None,None,None,24,Sporting Goods,2024-07-16,2024-07-16 10:02:34.643,...,262144,N,-999,Sporting Goods,-999,-999,263510,24,1,Lifestyle
45662,263527,Toys & Hobbies:Outdoor Toys & Structures:Gel B...,0,None,None,None,28,Toys,2024-07-16,2024-07-16 10:02:34.643,...,262144,N,-999,Toys,-999,-999,263527,28,1,Collectibles
45663,263528,Toys & Hobbies:Outdoor Toys & Structures:Gel B...,0,None,None,None,28,Toys,2024-07-16,2024-07-16 10:02:34.643,...,262144,N,-999,Toys,-999,-999,263528,28,1,Collectibles


In [44]:
cats_pdf_site_0.select('bsns_vrtcl_name').unique()

AttributeError: 'DataFrame' object has no attribute 'select'

In [55]:
cats_pdf_site_0_pna = cats_pdf_site_0[cats_pdf_site_0['bsns_vrtcl_name'] == "Parts & Accessories"]
cats_pdf_site_0_pna

,leaf_categ_id,leaf_categ_name,site_id,categ_busn_mgr,categ_busn_unit,regn_categ,user_defined_field1,user_defined_field3,cre_date,upd_date,...,categ_flags,adult_categ_yn,domain_id,user_defined_field5,vcs_id,gcs_id,move_to,sap_category_id,src_id,bsns_vrtcl_name
4476,111125,eBay Motors:Parts & Accessories:Boat Parts:Mot...,0,None,None,None,8,Auto - Parts,2014-09-14,2014-09-23 03:22:08,...,262144,N,-999,Auto - Parts,-999,-999,-99,8,1,Parts & Accessories
4477,133202,eBay Motors:Parts & Accessories:Performance & ...,0,None,None,None,8,Auto - Parts,2014-09-14,2014-09-23 03:22:08,...,262144,N,-999,Auto - Parts,-999,-999,-99,8,1,Parts & Accessories
4478,34236,eBay Motors:Parts & Accessories:Manuals & Lite...,0,None,None,None,8,Auto - Parts,2014-09-14,2014-09-23 03:22:08,...,262144,N,-999,Auto - Parts,-999,-999,-99,8,1,Parts & Accessories
4479,43976,eBay Motors:Parts & Accessories:ATV Parts:Elec...,0,None,None,None,8,Auto - Parts,2014-09-14,2014-09-23 03:22:08,...,262144,N,-999,Auto - Parts,-999,-999,-99,8,1,Parts & Accessories
4480,35560,eBay Motors:Parts & Accessories:Motorcycle Par...,0,None,None,None,8,Auto - Parts,2014-09-14,2014-09-23 03:22:08,...,262144,N,-999,Auto - Parts,-999,-999,-99,8,1,Parts & Accessories
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25849,50073,"eBay Motors:Parts & Accessories:RV, Trailer & ...",0,None,None,None,8,Auto - Parts,2014-09-14,2014-09-23 03:22:08,...,262144,N,-999,Auto - Parts,-999,-999,-99,8,1,Parts & Accessories
25850,38663,eBay Motors:Parts & Accessories:Manuals & Lite...,0,None,None,None,8,Auto - Parts,2014-09-14,2014-09-23 03:22:08,...,262144,N,-999,Auto - Parts,-999,-999,-99,8,1,Parts & Accessories
25851,26442,eBay Motors:Parts & Accessories:Aviation Parts...,0,None,6001,None,8,Auto - Parts,2014-09-14,2014-09-23 03:22:08,...,262144,N,-999,Auto - Parts,-999,-999,-99,8,1,Parts & Accessories
25852,33557,eBay Motors:Parts & Accessories:Car & Truck Pa...,0,None,None,None,8,Auto - Parts,2014-09-14,2014-09-23 03:22:08,...,262144,N,-999,Auto - Parts,-999,-999,-99,8,1,Parts & Accessories


24/08/25 18:20:57 ERROR YarnScheduler: Lost executor 6421 on hdc42-mcc10-01-0510-4004-014-tess0097.stratus.rno.ebay.com: Executor decommission.
24/08/25 18:20:57 ERROR YarnScheduler: Lost executor 6634 on hdc42-mcc10-01-0210-5506-020-tess0131.stratus.rno.ebay.com: Executor decommission.
24/08/25 18:20:58 ERROR YarnScheduler: Lost executor 6680 on hdc42-mcc10-01-0510-3401-025-tess0097.stratus.rno.ebay.com: Executor decommission.
24/08/25 18:20:58 ERROR YarnScheduler: Lost executor 6870 on hdc42-mcc10-01-1110-1709-003-tess0029.stratus.rno.ebay.com: Executor decommission.
24/08/25 18:20:58 ERROR YarnScheduler: Lost executor 6556 on hdc42-mcc10-01-1110-3302-027-tess0029.stratus.rno.ebay.com: Executor decommission.
24/08/25 18:20:58 ERROR YarnScheduler: Lost executor 6687 on hdc42-mcc10-01-1110-1806-018-tess0029.stratus.rno.ebay.com: Executor decommission.
24/08/25 18:20:58 ERROR YarnScheduler: Lost executor 6695 on hdc42-mcc10-01-0710-2106-018-tess0035.stratus.rno.ebay.com: Executor decomm

In [57]:
cats_pdf_site_0_pna['leaf_categ_id'].value_counts()

leaf_categ_id
111125    1
175614    1
43977     1
170577    1
124104    1
         ..
33601     1
169344    1
43983     1
33656     1
36474     1
Name: count, Length: 516, dtype: int64

24/08/25 18:22:04 ERROR YarnScheduler: Lost executor 5911 on hdc42-mcc10-01-1110-3406-032-tess0029.stratus.rno.ebay.com: Executor decommission.
24/08/25 18:22:05 ERROR YarnScheduler: Lost executor 5978 on hdc42-mcc10-01-0710-3605-005-tess0035.stratus.rno.ebay.com: Executor decommission.
24/08/25 18:22:05 ERROR YarnScheduler: Lost executor 5989 on hdc42-mcc10-01-1110-3404-011-tess0029.stratus.rno.ebay.com: Executor decommission.
24/08/25 18:22:05 ERROR YarnScheduler: Lost executor 6579 on hdc42-mcc10-01-1110-4203-017-tess0029.stratus.rno.ebay.com: Executor decommission.
24/08/25 18:22:05 ERROR YarnScheduler: Lost executor 6004 on hdc42-mcc10-01-0510-3401-028-tess0097.stratus.rno.ebay.com: Executor decommission.
24/08/25 18:22:06 ERROR YarnScheduler: Lost executor 6146 on hdc42-mcc10-01-1110-3706-005-tess0029.stratus.rno.ebay.com: Executor decommission.
24/08/25 18:22:06 ERROR YarnScheduler: Lost executor 6800 on hdc42-mcc10-01-0210-5502-071-tess0131.stratus.rno.ebay.com: Executor decomm

In [62]:
site_0_leafcats_from_cats_table = set(cats_pdf_site_0_pna['leaf_categ_id'].unique())
site_0_leafcats_from_cats_table

{Decimal('6032'),
 Decimal('6749'),
 Decimal('6752'),
 Decimal('6779'),
 Decimal('6783'),
 Decimal('26436'),
 Decimal('26439'),
 Decimal('26440'),
 Decimal('26442'),
 Decimal('26444'),
 Decimal('26446'),
 Decimal('26447'),
 Decimal('26448'),
 Decimal('26449'),
 Decimal('26450'),
 Decimal('26451'),
 Decimal('26453'),
 Decimal('26455'),
 Decimal('26456'),
 Decimal('31276'),
 Decimal('31278'),
 Decimal('31280'),
 Decimal('31281'),
 Decimal('31283'),
 Decimal('31284'),
 Decimal('31285'),
 Decimal('31286'),
 Decimal('33543'),
 Decimal('33544'),
 Decimal('33545'),
 Decimal('33546'),
 Decimal('33547'),
 Decimal('33548'),
 Decimal('33550'),
 Decimal('33551'),
 Decimal('33553'),
 Decimal('33554'),
 Decimal('33555'),
 Decimal('33556'),
 Decimal('33557'),
 Decimal('33558'),
 Decimal('33560'),
 Decimal('33563'),
 Decimal('33564'),
 Decimal('33565'),
 Decimal('33566'),
 Decimal('33567'),
 Decimal('33569'),
 Decimal('33570'),
 Decimal('33573'),
 Decimal('33574'),
 Decimal('33575'),
 Decimal('33576')

In [67]:
site_0_leafcats_from_cats_table_int = set([int(x) for x in site_0_leafcats_from_cats_table])
site_0_leafcats_from_cats_table_int

{6032,
 6749,
 6752,
 6779,
 6783,
 26436,
 26439,
 26440,
 26442,
 26444,
 26446,
 26447,
 26448,
 26449,
 26450,
 26451,
 26453,
 26455,
 26456,
 31276,
 31278,
 31280,
 31281,
 31283,
 31284,
 31285,
 31286,
 33543,
 33544,
 33545,
 33546,
 33547,
 33548,
 33550,
 33551,
 33553,
 33554,
 33555,
 33556,
 33557,
 33558,
 33560,
 33563,
 33564,
 33565,
 33566,
 33567,
 33569,
 33570,
 33573,
 33574,
 33575,
 33576,
 33577,
 33578,
 33580,
 33581,
 33582,
 33583,
 33584,
 33585,
 33586,
 33588,
 33589,
 33590,
 33591,
 33592,
 33593,
 33595,
 33596,
 33597,
 33598,
 33600,
 33601,
 33602,
 33603,
 33604,
 33607,
 33609,
 33610,
 33613,
 33614,
 33615,
 33616,
 33617,
 33619,
 33621,
 33622,
 33623,
 33625,
 33627,
 33629,
 33630,
 33631,
 33632,
 33633,
 33634,
 33636,
 33638,
 33639,
 33640,
 33642,
 33643,
 33644,
 33645,
 33646,
 33647,
 33648,
 33649,
 33650,
 33651,
 33652,
 33653,
 33654,
 33655,
 33656,
 33657,
 33659,
 33660,
 33661,
 33662,
 33663,
 33665,
 33666,
 33667,
 3366

In [61]:
items_df_site_0 = spark.table("ACCESS_VIEWS.DW_LSTG_ITEM").filter((F.col('auct_end_dt') >= start_date) &\
                                                           (F.col('auct_end_dt') <= end_date)).filter(F.col('ITEM_SITE_ID').isin([0]))
site_0_leafcats = set(items_df_site_0.select('LEAF_CATEG_ID').distinct().rdd.map(lambda row: row[0]).collect())
site_0_leafcats

{Decimal('32770'),
 Decimal('32771'),
 Decimal('32772'),
 Decimal('131081'),
 Decimal('131082'),
 Decimal('163851'),
 Decimal('12'),
 Decimal('163853'),
 Decimal('131084'),
 Decimal('131093'),
 Decimal('163862'),
 Decimal('28'),
 Decimal('163868'),
 Decimal('163870'),
 Decimal('163871'),
 Decimal('163872'),
 Decimal('163873'),
 Decimal('163874'),
 Decimal('163875'),
 Decimal('163869'),
 Decimal('37'),
 Decimal('163878'),
 Decimal('163879'),
 Decimal('35'),
 Decimal('41'),
 Decimal('32810'),
 Decimal('163882'),
 Decimal('163884'),
 Decimal('163885'),
 Decimal('44'),
 Decimal('163887'),
 Decimal('163888'),
 Decimal('163883'),
 Decimal('163886'),
 Decimal('42'),
 Decimal('52'),
 Decimal('163893'),
 Decimal('53'),
 Decimal('163895'),
 Decimal('163896'),
 Decimal('56'),
 Decimal('58'),
 Decimal('59'),
 Decimal('60'),
 Decimal('61'),
 Decimal('32834'),
 Decimal('260529'),
 Decimal('156325'),
 Decimal('80'),
 Decimal('156326'),
 Decimal('32866'),
 Decimal('32872'),
 Decimal('32874'),
 Decimal

24/08/25 18:31:47 ERROR YarnScheduler: Lost executor 7190 on hdc42-mcc10-01-1110-3508-022-tess0029.stratus.rno.ebay.com: Executor decommission.
24/08/25 18:31:50 ERROR YarnScheduler: Lost executor 7111 on hdc42-mcc10-01-0510-3608-004-tess0097.stratus.rno.ebay.com: Executor decommission.
24/08/25 18:31:50 ERROR YarnScheduler: Lost executor 7609 on hdc42-mcc10-01-1110-3406-038-tess0029.stratus.rno.ebay.com: Executor decommission.
24/08/25 18:31:51 ERROR YarnScheduler: Lost executor 7768 on hdc42-mcc10-01-0510-2002-036-tess0097.stratus.rno.ebay.com: Executor decommission.
24/08/25 18:31:51 ERROR YarnScheduler: Lost executor 7256 on hdc42-mcc10-01-1110-1708-019-tess0029.stratus.rno.ebay.com: Executor decommission.
24/08/25 18:31:51 ERROR YarnScheduler: Lost executor 7242 on hdc42-mcc10-01-0110-7305-013-tess0173.stratus.rno.ebay.com: Executor decommission.
24/08/25 18:31:51 ERROR YarnScheduler: Lost executor 7081 on hdc42-mcc10-01-0210-7709-012-tess0131.stratus.rno.ebay.com: Executor decomm

In [66]:
site_0_leafcats_int = set([int(x) for x in site_0_leafcats])
site_0_leafcats_int

{32770,
 32771,
 32772,
 131081,
 131082,
 163851,
 12,
 163853,
 131084,
 131093,
 163862,
 163868,
 28,
 163870,
 163871,
 163872,
 163873,
 163874,
 163875,
 163869,
 37,
 163878,
 163879,
 35,
 41,
 32810,
 163882,
 163884,
 163885,
 44,
 163887,
 163888,
 163883,
 163886,
 42,
 52,
 163893,
 53,
 163895,
 163896,
 56,
 58,
 59,
 60,
 61,
 32834,
 260530,
 80,
 32866,
 32872,
 32874,
 65643,
 32878,
 163951,
 163952,
 163953,
 114,
 32883,
 116,
 32885,
 115,
 32886,
 163960,
 163961,
 163962,
 163963,
 163964,
 163965,
 163966,
 126,
 131200,
 163969,
 130,
 163968,
 128,
 163973,
 132,
 163975,
 163976,
 136,
 163978,
 163979,
 138,
 163981,
 131207,
 163983,
 144,
 143,
 98448,
 163984,
 163988,
 142,
 163990,
 150,
 163992,
 153,
 163994,
 155,
 163993,
 163997,
 262302,
 262303,
 158,
 163999,
 98466,
 262305,
 262308,
 164005,
 164003,
 167,
 164008,
 164007,
 262311,
 170,
 262313,
 98477,
 98478,
 262319,
 164016,
 262321,
 262317,
 164019,
 164013,
 164014,
 164022,
 16402

In [68]:
site_0_leafcats_int.intersection(site_0_leafcats_from_cats_table_int)

set()

In [57]:
cats_pdf['bsns_vrtcl_name'].value_counts()

bsns_vrtcl_name
Collectibles             14275
Fashion                   7891
Electronics               5775
Lifestyle                 5685
Home & Garden             5062
Business & Industrial     4829
Media                     1172
Parts & Accessories        516
Unknown                    286
Real Estate                166
Vehicles                     8
Name: count, dtype: int64

### Parts & Accessories 

In [68]:
pna_pdf = cats_pdf[cats_pdf['bsns_vrtcl_name'] == 'Parts & Accessories']
pna_pdf

,leaf_categ_id,leaf_categ_name,site_id,categ_busn_mgr,categ_busn_unit,regn_categ,user_defined_field1,user_defined_field3,cre_date,upd_date,...,categ_flags,adult_categ_yn,domain_id,user_defined_field5,vcs_id,gcs_id,move_to,sap_category_id,src_id,bsns_vrtcl_name
4476,111125,eBay Motors:Parts & Accessories:Boat Parts:Mot...,0,None,None,None,8,Auto - Parts,2014-09-14,2014-09-23 03:22:08,...,262144,N,-999,Auto - Parts,-999,-999,-99,8,1,Parts & Accessories
4477,133202,eBay Motors:Parts & Accessories:Performance & ...,0,None,None,None,8,Auto - Parts,2014-09-14,2014-09-23 03:22:08,...,262144,N,-999,Auto - Parts,-999,-999,-99,8,1,Parts & Accessories
4478,34236,eBay Motors:Parts & Accessories:Manuals & Lite...,0,None,None,None,8,Auto - Parts,2014-09-14,2014-09-23 03:22:08,...,262144,N,-999,Auto - Parts,-999,-999,-99,8,1,Parts & Accessories
4479,43976,eBay Motors:Parts & Accessories:ATV Parts:Elec...,0,None,None,None,8,Auto - Parts,2014-09-14,2014-09-23 03:22:08,...,262144,N,-999,Auto - Parts,-999,-999,-99,8,1,Parts & Accessories
4480,35560,eBay Motors:Parts & Accessories:Motorcycle Par...,0,None,None,None,8,Auto - Parts,2014-09-14,2014-09-23 03:22:08,...,262144,N,-999,Auto - Parts,-999,-999,-99,8,1,Parts & Accessories
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25849,50073,"eBay Motors:Parts & Accessories:RV, Trailer & ...",0,None,None,None,8,Auto - Parts,2014-09-14,2014-09-23 03:22:08,...,262144,N,-999,Auto - Parts,-999,-999,-99,8,1,Parts & Accessories
25850,38663,eBay Motors:Parts & Accessories:Manuals & Lite...,0,None,None,None,8,Auto - Parts,2014-09-14,2014-09-23 03:22:08,...,262144,N,-999,Auto - Parts,-999,-999,-99,8,1,Parts & Accessories
25851,26442,eBay Motors:Parts & Accessories:Aviation Parts...,0,None,6001,None,8,Auto - Parts,2014-09-14,2014-09-23 03:22:08,...,262144,N,-999,Auto - Parts,-999,-999,-99,8,1,Parts & Accessories
25852,33557,eBay Motors:Parts & Accessories:Car & Truck Pa...,0,None,None,None,8,Auto - Parts,2014-09-14,2014-09-23 03:22:08,...,262144,N,-999,Auto - Parts,-999,-999,-99,8,1,Parts & Accessories


In [74]:
pna_pdf['meta_categ_name'].unique()

array(['eBay Motors'], dtype=object)

### Collectibles 

In [81]:
cats_pdf[cats_pdf['bsns_vrtcl_name'] == 'Collectibles']

,leaf_categ_id,leaf_categ_name,site_id,categ_busn_mgr,categ_busn_unit,regn_categ,user_defined_field1,user_defined_field3,cre_date,upd_date,...,categ_flags,adult_categ_yn,domain_id,user_defined_field5,vcs_id,gcs_id,move_to,sap_category_id,src_id,bsns_vrtcl_name
77,50725,Collectibles:Animals:Dogs:Shiba Inu,0,None,None,None,11,Collectibles,2008-06-03,2012-09-12 03:26:04,...,262144,N,-999,Collectibles,-999,-999,50725,11,1,Collectibles
78,78000,Collectibles:Holiday & Seasonal:Incense Smokers,0,None,None,None,11,Collectibles,2021-05-11,2021-05-12 00:41:45.601,...,262144,N,-999,Collectibles,-999,-999,78000,11,1,Collectibles
79,158429,Collectibles:Militaria:WW I (1914-18):Original...,0,None,None,None,11,Collectibles,2008-06-03,2008-06-03 20:36:00,...,262144,N,-999,Collectibles,-999,-999,158429,11,1,Collectibles
80,13942,Collectibles:Kitchen & Home:Tableware:Pitchers,0,None,1,None,11,Collectibles,2008-06-03,2021-05-12 00:38:41.875,...,262144,N,-999,Collectibles,-999,-999,261671,11,1,Collectibles
81,37835,Collectibles:Advertising:Merchandise & Memorab...,0,None,None,None,11,Collectibles,2008-06-03,2012-09-12 03:26:04,...,262144,N,-999,Collectibles,-999,-999,37835,11,1,Collectibles
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44792,73416,"Sports Mem, Cards & Fan Shop:Autographs-Origin...",0,None,None,None,25,Sports Memorabilia,2008-06-03,2013-04-29 03:37:19,...,262144,N,-999,Sports Memorabilia,-999,-999,73416,25,1,Collectibles
44793,86853,"Sports Mem, Cards & Fan Shop:Cards:Baseball-ML...",0,None,None,None,25,Sports Memorabilia,2008-06-03,2021-05-12 00:38:41.875,...,262144,N,-999,Sports Memorabilia,3636,2152,261328,25,1,Collectibles
45662,263527,Toys & Hobbies:Outdoor Toys & Structures:Gel B...,0,None,None,None,28,Toys,2024-07-16,2024-07-16 10:02:34.643,...,262144,N,-999,Toys,-999,-999,263527,28,1,Collectibles
45663,263528,Toys & Hobbies:Outdoor Toys & Structures:Gel B...,0,None,None,None,28,Toys,2024-07-16,2024-07-16 10:02:34.643,...,262144,N,-999,Toys,-999,-999,263528,28,1,Collectibles


In [45]:
# That's for historical categories 
# cats = spark.table('ACCESS_VIEWS.DW_CATEGORY_HIERARCHY')

In [18]:
[ s for s in lst_table.columns if 'site' in s ]

['item_site_id', 'slr_site_id', 'high_bdr_site_id', 'site_create_date']

In [23]:
lst_table.columns


['item_id',
 'auct_end_dt',
 'auct_start_dt',
 'sale_sched_end_dt',
 'auct_type_code',
 'item_site_id',
 'item_cntry_id',
 'lstg_curncy_id',
 'lstg_status_id',
 'icon_flags',
 'slr_id',
 'slr_site_id',
 'slr_cntry_id',
 'high_bdr_id',
 'leaf_categ_id',
 'leaf_categ_id2',
 'auct_start_exchng_rate',
 'auct_end_exchng_rate',
 'start_price_lstg_curncy',
 'rsrv_price_list_crncy',
 'bin_price_lstg_curncy',
 'curnt_price_lstg_curncy',
 'gms_lstg_curncy',
 'gms_lstg_curncy_est',
 'qty_sold',
 'qty_avail',
 'bidcount',
 'wacko_yn',
 'success_yn',
 'stor_lstg_yn_flag_id',
 'ebx_elgbl_item_yn_id',
 'mr_lister_flag',
 'slng_mgr_sbscrbr_flag',
 'slng_mgr_pro_sbscrbr_flag',
 'slng_mgr_pro_used_up_flag',
 'bin_sold_up_flag',
 'gtc_up_flag',
 'paypal_accepted_up_flag',
 'ck_fetr_up_flag',
 'mrch_tool_up_flag',
 'syi_2_up_flag',
 'charity_giving_tool_up_flag',
 'b2b_up_flag',
 'slr_paypal_prefd_up_flag',
 'categ_slng_guide_used_up_flag',
 'relist_up_flag',
 'relist_crd_up_flag',
 'listed_with_autopay_u

In [24]:
[ s for s in lst_table.columns if 'end' in s]

['auct_end_dt',
 'sale_sched_end_dt',
 'auct_end_exchng_rate',
 'auct_end_date',
 'auction_end_reason_code',
 'sale_sched_end_date',
 'prmtn_sale_end_dt',
 'prmtn_sale_end_tm']

## Examine leaf_cat == 181708

In [33]:
cats_df_181708 = cats_table.filter(F.col('leaf_categ_id') == 181708)
cats_df_181708.count()

21

In [36]:
cats_df_181708.select('leaf_categ_id', 'leaf_categ_name', 'site_id', 'bsns_vrtcl_name').show()

+-------------+--------------------+-------+--------------------+
|leaf_categ_id|     leaf_categ_name|site_id|     bsns_vrtcl_name|
+-------------+--------------------+-------+--------------------+
|       181708|Business, Office ...|    205|Business & Indust...|
|       181708|Business & Indust...|      2|Business & Indust...|
|       181708|Professioneel mat...|    123|Business & Indust...|
|       181708|Business & Indust...|    215|Business & Indust...|
|       181708|Equipamiento y ma...|    186|Business & Indust...|
|       181708|Business & Indust...|    207|Business & Indust...|
|       181708|Business & Indust...|    193|Business & Indust...|
|       181708|Équipements profe...|     71|Business & Indust...|
|       181708|Business & Indust...|    216|Business & Indust...|
|       181708|Business, Office ...|      3|Business & Indust...|
|       181708|Industrial:Indust...|     15|Business & Indust...|
|       181708|Équipements profe...|     23|Business & Indust...|
|       18

In [8]:
cats_1 = spark.table('ACCESS_VIEWS.DW_CATEGORY_GROUPINGS')
cats_1.columns

['leaf_categ_id',
 'leaf_categ_name',
 'site_id',
 'categ_busn_mgr',
 'categ_busn_unit',
 'regn_categ',
 'user_defined_field1',
 'user_defined_field3',
 'cre_date',
 'upd_date',
 'cre_user',
 'upd_user',
 'meta_categ_id',
 'meta_categ_name',
 'categ_lvl2_id',
 'categ_lvl3_id',
 'categ_lvl4_id',
 'categ_lvl5_id',
 'categ_lvl6_id',
 'categ_lvl7_id',
 'categ_lvl2_name',
 'categ_lvl3_name',
 'categ_lvl4_name',
 'categ_lvl5_name',
 'categ_lvl6_name',
 'categ_lvl7_name',
 'categ_flags',
 'adult_categ_yn',
 'domain_id',
 'user_defined_field5',
 'vcs_id',
 'gcs_id',
 'move_to',
 'sap_category_id',
 'src_id',
 'bsns_vrtcl_name']

24/08/12 12:07:39 ERROR YarnScheduler: Lost executor 12 on hdc42-mcc10-01-0910-1907-018-tess0028.stratus.rno.ebay.com: Executor decommission.
24/08/12 12:07:39 ERROR YarnScheduler: Lost executor 19 on hdc42-mcc10-01-0110-7402-007-tess0173.stratus.rno.ebay.com: Executor decommission.
24/08/12 12:07:39 ERROR YarnScheduler: Lost executor 8 on hdc42-mcc10-01-0110-5609-009-tess0173.stratus.rno.ebay.com: Executor decommission.
24/08/12 12:07:39 ERROR YarnScheduler: Lost executor 2 on hdc42-mcc10-01-0110-7501-003-tess0173.stratus.rno.ebay.com: Executor decommission.
24/08/12 12:07:40 ERROR YarnScheduler: Lost executor 13 on hdc42-mcc10-01-0910-1706-023-tess0028.stratus.rno.ebay.com: Executor decommission.
24/08/12 12:07:40 ERROR YarnScheduler: Lost executor 17 on hdc42-mcc10-01-0510-3903-034-tess0097.stratus.rno.ebay.com: Executor decommission.
24/08/12 12:07:41 ERROR YarnScheduler: Lost executor 9 on hdc42-mcc10-01-1110-1708-004-tess0029.stratus.rno.ebay.com: Executor decommission.
24/08/12 

## Examine site_id 100 = (US P&A)

In [79]:
cats_pdf_site_100 = cats_table.filter(F.col('site_id') == 100).toPandas()
cats_pdf_site_100

,leaf_categ_id,leaf_categ_name,site_id,categ_busn_mgr,categ_busn_unit,regn_categ,user_defined_field1,user_defined_field3,cre_date,upd_date,...,categ_flags,adult_categ_yn,domain_id,user_defined_field5,vcs_id,gcs_id,move_to,sap_category_id,src_id,bsns_vrtcl_name
0,73362,Consumer Electronics:Vehicle Electronics & GPS...,100,None,None,None,13,Consumer Electronics - Other,2014-09-14,2014-09-23 03:22:08,...,262144,N,-999,Consumer Electronics - Other,-999,-999,-99,13,1,Electronics
1,11332,Sporting Goods:Outdoor Sports:Scooters:Parts &...,100,None,888,None,24,Sporting Goods,2014-09-14,2014-09-23 03:22:08,...,262144,N,-999,Sporting Goods,-999,-999,-99,24,1,Lifestyle
2,33758,Collectibles:Transportation:Automobilia:Licens...,100,None,None,None,11,Collectibles,2014-09-14,2014-09-23 03:22:08,...,262144,N,-999,Collectibles,-999,-999,-99,11,1,Collectibles
3,71536,Consumer Electronics:Vehicle Electronics & GPS...,100,None,None,None,13,Consumer Electronics - Other,2014-09-14,2014-09-23 03:22:08,...,262144,N,-999,Consumer Electronics - Other,-999,-999,-99,13,1,Electronics
4,378,Books:Nonfiction,100,None,267,Other,3,Books,2014-09-14,2014-09-23 03:22:08,...,262144,N,-999,Books,-999,-999,-99,3,1,Media
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5275,79845,Consumer Electronics:Multipurpose Batteries & ...,100,None,None,None,40,Cell Phones & Portable Electro,2014-09-14,2014-09-23 03:22:08,...,262144,N,-999,Cell Phones & Accessories,-999,-999,-99,40,1,Electronics
5276,9355,Cell Phones & Accessories:Cell Phones & Smartp...,100,None,None,None,40,Cell Phones & Portable Electro,2014-09-14,2014-09-23 03:22:08,...,262144,N,-999,Cell Phones & Accessories,-999,-999,-99,40,1,Electronics
5277,35190,Cell Phones & Accessories:Cell Phone Accessori...,100,None,None,None,40,Cell Phones & Portable Electro,2014-09-14,2014-09-23 03:22:08,...,262144,N,-999,Cell Phones & Accessories,-999,-999,-99,40,1,Electronics
5278,20380,"Cell Phones & PDAs:Accessories, Parts:Antennas...",100,None,293,None,40,Cell Phones & Portable Electro,2014-09-14,2014-09-23 03:22:08,...,0,N,-999,Cell Phones & Accessories,-999,-999,-99,40,1,Electronics


In [81]:
site_100_leafcats_from_cats_table = set(cats_pdf_site_100['leaf_categ_id'])
site_100_leafcats_from_cats_table

{Decimal('262144'),
 Decimal('0'),
 Decimal('262146'),
 Decimal('3'),
 Decimal('262148'),
 Decimal('5'),
 Decimal('262149'),
 Decimal('262150'),
 Decimal('8'),
 Decimal('262152'),
 Decimal('262154'),
 Decimal('2'),
 Decimal('262156'),
 Decimal('262157'),
 Decimal('262158'),
 Decimal('262159'),
 Decimal('262161'),
 Decimal('262162'),
 Decimal('262163'),
 Decimal('262164'),
 Decimal('262166'),
 Decimal('262167'),
 Decimal('262168'),
 Decimal('262170'),
 Decimal('262171'),
 Decimal('262172'),
 Decimal('262173'),
 Decimal('262174'),
 Decimal('30'),
 Decimal('262176'),
 Decimal('32'),
 Decimal('262177'),
 Decimal('262179'),
 Decimal('262178'),
 Decimal('262181'),
 Decimal('262182'),
 Decimal('262183'),
 Decimal('262184'),
 Decimal('262185'),
 Decimal('42'),
 Decimal('262186'),
 Decimal('262188'),
 Decimal('38'),
 Decimal('32810'),
 Decimal('262191'),
 Decimal('262192'),
 Decimal('262193'),
 Decimal('262194'),
 Decimal('262189'),
 Decimal('262196'),
 Decimal('262197'),
 Decimal('262198'),
 D

24/08/26 06:21:41 ERROR YarnScheduler: Lost executor 9535 on hdc42-mcc10-01-1110-4105-036-tess0029.stratus.rno.ebay.com: Executor decommission.
24/08/26 06:21:44 ERROR YarnScheduler: Lost executor 9839 on hdc42-mcc10-01-1110-3303-032-tess0029.stratus.rno.ebay.com: Executor decommission.
24/08/26 06:21:44 ERROR YarnScheduler: Lost executor 9688 on hdc42-mcc10-01-1110-3609-032-tess0029.stratus.rno.ebay.com: Executor decommission.
24/08/26 06:21:45 ERROR YarnScheduler: Lost executor 9557 on hdc42-mcc10-01-1110-3609-022-tess0029.stratus.rno.ebay.com: Executor decommission.
24/08/26 06:21:51 ERROR YarnScheduler: Lost executor 9656 on hdc42-mcc10-01-0510-3502-028-tess0097.stratus.rno.ebay.com: Executor decommission.


In [80]:
items_df_site_100 = spark.table("ACCESS_VIEWS.DW_LSTG_ITEM").filter((F.col('auct_end_dt') >= start_date) &\
                                                           (F.col('auct_end_dt') <= end_date)).filter(F.col('ITEM_SITE_ID').isin([100]))
site_100_leafcats = set(items_df_site_100.select('LEAF_CATEG_ID').distinct().rdd.map(lambda row: row[0]).collect())
site_100_leafcats

{Decimal('262144'),
 Decimal('262146'),
 Decimal('262148'),
 Decimal('262149'),
 Decimal('262150'),
 Decimal('262152'),
 Decimal('262154'),
 Decimal('262156'),
 Decimal('262157'),
 Decimal('57357'),
 Decimal('262159'),
 Decimal('262158'),
 Decimal('262161'),
 Decimal('262162'),
 Decimal('262163'),
 Decimal('180243'),
 Decimal('180241'),
 Decimal('262166'),
 Decimal('262167'),
 Decimal('262168'),
 Decimal('262164'),
 Decimal('262170'),
 Decimal('262171'),
 Decimal('262172'),
 Decimal('262173'),
 Decimal('262174'),
 Decimal('262176'),
 Decimal('262177'),
 Decimal('262178'),
 Decimal('262179'),
 Decimal('262180'),
 Decimal('262181'),
 Decimal('262182'),
 Decimal('262183'),
 Decimal('262184'),
 Decimal('262185'),
 Decimal('262186'),
 Decimal('262188'),
 Decimal('262189'),
 Decimal('182370'),
 Decimal('262191'),
 Decimal('262192'),
 Decimal('262193'),
 Decimal('262194'),
 Decimal('262196'),
 Decimal('262197'),
 Decimal('262198'),
 Decimal('262200'),
 Decimal('262201'),
 Decimal('262203'),
 

24/08/26 06:20:25 ERROR YarnScheduler: Lost executor 9293 on hdc42-mcc10-01-0110-7810-018-tess0173.stratus.rno.ebay.com: Executor decommission.
24/08/26 06:20:25 ERROR YarnScheduler: Lost executor 9705 on hdc42-mcc10-01-0210-7707-011-tess0131.stratus.rno.ebay.com: Executor decommission.
24/08/26 06:20:26 ERROR YarnScheduler: Lost executor 9418 on hdc42-mcc10-01-0110-5103-019-tess0173.stratus.rno.ebay.com: Executor decommission.
24/08/26 06:20:26 ERROR YarnScheduler: Lost executor 9594 on hdc42-mcc10-01-1110-2504-035-tess0029.stratus.rno.ebay.com: Executor decommission.
24/08/26 06:20:27 ERROR YarnScheduler: Lost executor 9249 on hdc42-mcc10-01-0110-7910-004-tess0173.stratus.rno.ebay.com: Executor decommission.
24/08/26 06:20:27 ERROR YarnScheduler: Lost executor 9209 on hdc42-mcc10-01-1110-2101-001-tess0029.stratus.rno.ebay.com: Executor decommission.
24/08/26 06:20:27 ERROR YarnScheduler: Lost executor 9509 on hdc42-mcc10-01-0210-7702-013-tess0131.stratus.rno.ebay.com: Executor decomm

In [83]:
len(site_100_leafcats_from_cats_table.intersection(site_100_leafcats)), len(site_100_leafcats), len(site_100_leafcats_from_cats_table)

(2191, 2191, 5280)

24/08/26 06:22:25 ERROR YarnScheduler: Lost executor 9393 on hdc42-mcc10-01-1110-2602-017-tess0029.stratus.rno.ebay.com: Executor decommission.
24/08/26 06:22:47 ERROR YarnScheduler: Lost executor 9737 on hdc42-mcc10-01-1110-3403-031-tess0029.stratus.rno.ebay.com: Executor decommission.
24/08/26 06:22:49 ERROR YarnScheduler: Lost executor 9188 on hdc42-mcc10-01-1110-1709-015-tess0029.stratus.rno.ebay.com: Executor decommission.


In [84]:
site_100_leafcats_from_cats_table.difference(site_100_leafcats)

{Decimal('0'),
 Decimal('182361'),
 Decimal('2'),
 Decimal('3'),
 Decimal('5'),
 Decimal('182362'),
 Decimal('8'),
 Decimal('182364'),
 Decimal('30'),
 Decimal('32'),
 Decimal('56994'),
 Decimal('38'),
 Decimal('42'),
 Decimal('32810'),
 Decimal('71'),
 Decimal('77'),
 Decimal('32874'),
 Decimal('32884'),
 Decimal('162'),
 Decimal('219'),
 Decimal('223'),
 Decimal('82182'),
 Decimal('82183'),
 Decimal('268'),
 Decimal('82191'),
 Decimal('280'),
 Decimal('82205'),
 Decimal('82212'),
 Decimal('180517'),
 Decimal('180520'),
 Decimal('180522'),
 Decimal('180523'),
 Decimal('303'),
 Decimal('82226'),
 Decimal('310'),
 Decimal('312'),
 Decimal('8504'),
 Decimal('317'),
 Decimal('318'),
 Decimal('320'),
 Decimal('324'),
 Decimal('57054'),
 Decimal('82255'),
 Decimal('378'),
 Decimal('82321'),
 Decimal('418'),
 Decimal('421'),
 Decimal('147900'),
 Decimal('147901'),
 Decimal('147902'),
 Decimal('147903'),
 Decimal('147904'),
 Decimal('449'),
 Decimal('147905'),
 Decimal('42425'),
 Decimal('467

24/08/26 06:23:04 ERROR YarnScheduler: Lost executor 9628 on hdc42-mcc10-01-1110-3304-024-tess0029.stratus.rno.ebay.com: Executor decommission.
24/08/26 06:23:25 ERROR YarnScheduler: Lost executor 9681 on hdc42-mcc10-01-1110-2103-013-tess0029.stratus.rno.ebay.com: Executor decommission.


In [89]:
cats_pdf_site_100[cats_pdf_site_100['leaf_categ_id'] == 182361]

,leaf_categ_id,leaf_categ_name,site_id,categ_busn_mgr,categ_busn_unit,regn_categ,user_defined_field1,user_defined_field3,cre_date,upd_date,...,categ_flags,adult_categ_yn,domain_id,user_defined_field5,vcs_id,gcs_id,move_to,sap_category_id,src_id,bsns_vrtcl_name
2580,182361,eBay Motors:Parts & Accessories:Scooter Parts:...,100,None,None,None,8,Auto - Parts,2016-05-16,2022-09-20 02:12:07.725,...,262144,N,-999,Auto - Parts,-999,-999,178041,8,1,Parts & Accessories


# Aspects - deep dive 

In [8]:
df_collectibles = pd.read_parquet('data/items_aspects_pdf_collectb_us_with_sigs.parquet')
df_collectibles

,item_id,item_desc,title,SUBTITLE,AUCT_START_DT,AUCT_END_DT,leaf_categ_id,ITEM_SITE_ID,bsns_vrtcl_name,meta_categ_id,aspects_with_none,desc,aspects,aspects_no_none,gen_text_mistral,gen_sig_mistral
0,114664125925,VGhpcyB2aW50YWdlIG11ZyBpcyBhIG11c3QtaGF2ZSBmb3...,vintage hardee's rise and shine made from scra...,None,2021-02-02,2024-09-02,850,0,collectibles,1,"{'4wd/2wd': None, 'age': None, 'age level': No...",This vintage mug is a must-have for collectors...,"{'4wd/2wd': None, 'age': None, 'age level': No...","{'4wd/2wd': None, 'age': None, 'age level': No...","{""characteristic"": ""vintage, original"", ""sale ...","vintage, original"
1,115755359434,PHA+VXAgZm9yIHNhbGUgaXMgYSAxOTgwcyBEb24gTGFjZW...,1980 don lacefield chevrolet license plate hil...,None,2023-04-02,2024-09-02,421,0,collectibles,1,"{'4wd/2wd': None, 'age': None, 'age level': No...",Up for sale is a 1980s Don Lacefield Chevrolet...,"{'4wd/2wd': None, 'age': None, 'age level': No...","{'4wd/2wd': None, 'age': None, 'age level': No...","{""characteristic"": ""collectibles: primary prod...",collectibles: primary product
2,155236461306,PGRpdiBpZD0iZHNfZGl2Ij48Zm9udCByd3I9IjEiIHNpem...,2020 topps opening day fernando tatis jr. #189...,None,2022-11-02,2024-09-02,261328,0,collectibles,64482,"{'4wd/2wd': None, 'age': None, 'age level': No...",DESCRIPTION The item you see in the pictures i...,"{'4wd/2wd': None, 'age': None, 'age level': No...","{'4wd/2wd': None, 'age': None, 'age level': No...","{""characteristic"": ""Graded 9"", ""sale pitch"": ""...",Graded 9
3,155390411573,TWV3dHdvIFZTVEFSIEdvbGQgMDkxLzA3MSBVUiBTMTBiIE...,mewtwo vstar gold 091/071 ur s10b japanese pok...,None,2023-02-02,2024-09-02,183454,0,collectibles,220,"{'4wd/2wd': None, 'age': None, 'age level': No...",Mewtwo VSTAR Gold 091/071 UR S10b Japanese Pok...,"{'4wd/2wd': None, 'age': None, 'age level': No...","{'4wd/2wd': None, 'age': None, 'age level': No...","{""characteristic"": ""gold rarity"", ""sale pitch""...",gold rarity
4,175597284740,PGI+PGZvbnQgZmFjZT0iQXJpYWwiIHNpemU9IjQiPkw0OD...,signature models 1937 cord 812 supercharger (1...,None,2023-02-02,2024-09-02,180506,0,collectibles,220,"{'4wd/2wd': None, 'age': None, 'age level': No...",L48,"{'4wd/2wd': None, 'age': None, 'age level': No...","{'4wd/2wd': None, 'age': None, 'age level': No...","{""characteristic"": ""1:32 scale"", ""sale pitch"":...",1:32 scale
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,355509021587,VGhpc8KgMjAwOMKgVG9wcHPCoEtpY2tvZmbCoHNpbmdsZS...,peyton manning 2008 topps kickoff stars of the...,None,2024-03-02,2024-09-02,261328,0,collectibles,64482,"{'4wd/2wd': None, 'age': None, 'age level': No...",This 2008 Topps Kickoff single-insert sports t...,"{'4wd/2wd': None, 'age': None, 'age level': No...","{'4wd/2wd': None, 'age': None, 'age level': No...","{""characteristic"": ""legendary quarterback"", ""s...",legendary quarterback
4996,355761665374,PGRpdj48c3Ryb25nIHN0eWxlPSJjb2xvcjogcmdiKDE3My...,lego black minifig torso hogwarts robe clasp s...,None,2024-06-02,2024-09-02,183448,0,collectibles,220,"{'4wd/2wd': None, 'age': None, 'age level': '4...","100% LEGO Brand - Check out our store often, a...","{'4wd/2wd': None, 'age': None, 'age level': '4...","{'4wd/2wd': None, 'age': None, 'age level': '4...","{""characteristic"": ""LEGO theme: Harry Potter"",...",LEGO theme: Harry Potter
4997,375408380381,PHAgZGlyPSJsdHIiIHN0eWxlPSJtYXJnaW4tdG9wOjA7IG...,vintage 1999 hoop furby basketball very damage...,None,2024-05-02,2024-09-02,1083,0,collectibles,220,"{'4wd/2wd': None, 'age': None, 'age level': No...",This is in horrible condition. It's covered in...,"{'4wd/2wd': None, 'age': None, 'age level': No...","{'4wd/2wd': None, 'age': None, 'age level': No...","{""characteristic"": ""vintage year: 1999"", ""sale...",vintage year: 1999
4998,394974214459,PGZvbnQgc2l6ZT0iNCIgc3R5bGU9ImZvbnQtZmFtaWx5Ok...,old vintage antique photo booth photograph pre...,None,2023-11-02,2024-09-02,262421,0,collectibles,1,"{'4wd/2wd':

In [12]:
items_aspects_pdf_collectb_us_oct = pd.read_parquet('data/items_aspects_pdf_collectb_us_oct.parquet')
items_aspects_pdf_collectb_us_oct

,item_id,item_desc,title,SUBTITLE,AUCT_START_DT,AUCT_END_DT,leaf_categ_id,ITEM_SITE_ID,bsns_vrtcl_name,meta_categ_id,GALLERY_URL,aspects
0,123979599749,PGZvbnQgcndyPSIxIiBzdHlsZT0iZm9udC1mYW1pbHk6IE...,"ripon cathedral north yorkshire, england great...",None,2019-11-17,2024-09-17,262042,0,collectibles,1,https://i.ebayimg.com/00/s/mtezmfgxnjaw/z/ftma...,"{'4wd/2wd': None, 'active/retired': None, 'act..."
1,134658435740,PHN0eWxlPmltZ3ttYXgtd2lkdGg6MTAwJX08L3N0eWxlPj...,1971 topps football #174 adrian young nm *d3,None,2023-07-17,2024-09-17,261328,0,collectibles,64482,https://i.ebayimg.com/00/s/mte0nfg4ndq=/z/ey8a...,"{'4wd/2wd': None, 'active/retired': None, 'act..."
2,145560028063,PGI+UExFQVNFIExPT0sgQVQgVEhFIENPTkRJVElPTiBPRi...,#8 2000 winners circle dale earnhardt jr unmar...,None,2024-01-17,2024-09-17,261330,0,collectibles,64482,https://i.ebayimg.com/00/s/oty5wdc1mw==/z/axaa...,"{'4wd/2wd': None, 'active/retired': None, 'act..."
3,156013659941,PHA+MTk3MiBUb3BwcyAtIFRvbnkgUGVyZXogLSAgIzgwIC...,1972 topps - tony perez - #80 cincinnati reds,None,2024-01-17,2024-09-17,261328,0,collectibles,64482,https://i.ebayimg.com/00/s/mtmxm1g4mji=/z/pv0a...,"{'4wd/2wd': None, 'active/retired': None, 'act..."
4,166873659153,PHAgZGlyPSJsdHIiIHN0eWxlPSJtYXJnaW4tdG9wOjA7IG...,"vintage estwing 16 oz. claw hammer 12.5"" inche...",None,2024-07-17,2024-09-17,39730,0,collectibles,1,https://i.ebayimg.com/00/s/mtywmfg1odu=/z/otya...,"{'4wd/2wd': None, 'active/retired': None, 'act..."
...,...,...,...,...,...,...,...,...,...,...,...,...
4995,353816745693,PHAgZGlyPSJsdHIiIHN0eWxlPSJtYXJnaW4tdG9wOjA7IG...,"daredevil #104 vol 2 (marvel, 2008) nm-",None,2021-12-17,2024-09-17,259104,0,collectibles,1,https://i.ebayimg.com/00/s/mtywmfgxmjaw/z/hsia...,"{'4wd/2wd': None, 'active/retired': None, 'act..."
4996,354114504594,PGZvbnQgcndyPSIxIiBzaXplPSI0IiBzdHlsZT0iZm9udC...,dc comics action comics #862 superman 2008 vf+...,None,2022-06-17,2024-09-17,259104,0,collectibles,1,https://i.ebayimg.com/00/s/mtywmfgxmjm2/z/py8a...,"{'4wd/2wd': None, 'active/retired': None, 'act..."
4997,354593615047,PCFET0NUWVBFIGh0bWw+CjxodG1sIGxhbmc9IiI+Cgo8aG...,2007-08 spx freshman tandems jerseys glen davi...,None,2023-02-17,2024-09-17,261328,0,collectibles,64482,https://i.ebayimg.com/00/s/oduwwdeyoda=/z/2vua...,"{'4wd/2wd': None, 'active/retired': None, 'act..."
4998,395538561456,SmFwYW5lc2UgYW5pbWUgS2lubmlrdW1hbiBmaWd1cmUgUm...,japanese anime kinnikuman figure rikishiman si...,None,2024-07-17,2024-09-17,69528,0,collectibles,1,https://i.ebayimg.com/00/s/mta4mfgxmdgw/z/qfwa...,"{'4wd/2wd': None, 'active/retired': None, 'act..."


## Setup

In [23]:
# start_date = '2024-09-17'
# end_date = '2024-09-17'
start_date = '2024-09-02'
end_date = '2024-09-02'

aspects_df = spark.table("ACCESS_VIEWS.ITEM_ASPCT_CLSSFCTN_SAP").filter((F.col('auct_end_dt') >= start_date)&\
                                                                (F.col('auct_end_dt') <= end_date)                                                                
                                                                )


## item 123979599749

### Extracted from extraction pipeline 

In [19]:
item_df_extracted = items_aspects_pdf_collectb_us_oct[items_aspects_pdf_collectb_us_oct['item_id'] == 123979599749]
item_df_extracted

,item_id,item_desc,title,SUBTITLE,AUCT_START_DT,AUCT_END_DT,leaf_categ_id,ITEM_SITE_ID,bsns_vrtcl_name,meta_categ_id,GALLERY_URL,aspects
0,123979599749,PGZvbnQgcndyPSIxIiBzdHlsZT0iZm9udC1mYW1pbHk6IE...,"ripon cathedral north yorkshire, england great...",None,2019-11-17,2024-09-17,262042,0,collectibles,1,https://i.ebayimg.com/00/s/mtezmfgxnjaw/z/ftma...,"{'4wd/2wd': None, 'active/retired': None, 'act..."


In [25]:
len(item_df_extracted['aspects'][0].keys()), 


436

In [28]:
aspects_no_none = {k:v for k,v in item_df_extracted['aspects'][0].items() if v is not None }
aspects_no_none

{'catrecoid_1': '262042',
 'catrecoid_2': '262043',
 'catrecoid_3': '41511',
 'catrecoscore_1': '98',
 'catrecoscore_2': '0',
 'catrecoscore_3': '0',
 'city': 'ripon',
 'continent': 'europe',
 'core_product_type_v2': 'post cards: primary product',
 'country': 'england',
 'mergednamespacenames': 'continent',
 'miscatscore': '41',
 'region': 'yorkshire',
 'subject': 'church',
 'theme': 'cities & towns'}

In [33]:
aspects_names_no_none = set(aspects_no_none.keys())
aspects_names_no_none
                        

{'catrecoid_1',
 'catrecoid_2',
 'catrecoid_3',
 'catrecoscore_1',
 'catrecoscore_2',
 'catrecoscore_3',
 'city',
 'continent',
 'core_product_type_v2',
 'country',
 'mergednamespacenames',
 'miscatscore',
 'region',
 'subject',
 'theme'}

In [23]:
len(item_df_extracted)

1

In [16]:
item_df = aspects_df.filter(F.col('item_id') == 123979599749)
item_pdf = item_df.toPandas()
item_pdf

,item_id,prdct_aspct_nm,aspct_vlu_nm,ns_type_cd,aspct_src,auct_end_dt,load_ts,key_type
0,123979599749,MiscatScore,41,df,6,2024-09-17,2024-08-30 18:20:59,2
1,123979599749,CatRecoID_2,262043,df,6,2024-09-17,2024-08-30 18:20:59,2
2,123979599749,_qryitemsimembeddingFP,1:e0995e06cfa9687291be443e35f53c77,_,258,2024-09-17,2024-08-30 18:20:59,2
3,123979599749,KeyValues_Hash,29771278913123398,vis,98,2024-09-17,2024-08-30 18:20:59,2
4,123979599749,Country,England,vdf,185,2024-09-17,2024-08-30 18:20:59,2
...,...,...,...,...,...,...,...,...
145,123979599749,Subject,Church,vdf,156,2024-09-17,2024-08-17 21:32:30,1
146,123979599749,skip,noMatch,sd,190,2024-09-17,2024-08-17 21:32:30,1
147,123979599749,Continent,Europe,vdf,156,2024-09-17,2024-08-17 21:32:30,1
148,123979599749,Country,England,ae,3,2024-09-17,2024-08-17 21:32:30,1


24/09/02 19:01:30 ERROR YarnScheduler: Lost executor 434 on hdc42-mcc10-01-0110-7802-003-tess0173.stratus.rno.ebay.com: Executor decommission.
24/09/02 19:01:30 ERROR YarnScheduler: Lost executor 445 on hdc42-mcc10-01-0210-7601-022-tess0131.stratus.rno.ebay.com: Executor decommission.
24/09/02 19:01:30 ERROR YarnScheduler: Lost executor 444 on hdc42-mcc10-01-0110-7501-010-tess0173.stratus.rno.ebay.com: Executor decommission.
24/09/02 19:01:30 ERROR YarnScheduler: Lost executor 416 on hdc42-mcc10-01-0110-5103-019-tess0173.stratus.rno.ebay.com: Executor decommission.
24/09/02 19:01:30 ERROR YarnScheduler: Lost executor 406 on hdc42-mcc10-01-0110-7602-014-tess0173.stratus.rno.ebay.com: Executor decommission.
24/09/02 19:01:30 ERROR YarnScheduler: Lost executor 422 on hdc42-mcc10-01-0710-3403-021-tess0035.stratus.rno.ebay.com: Executor decommission.
24/09/02 19:01:30 ERROR YarnScheduler: Lost executor 455 on hdc42-mcc10-01-0210-5401-010-tess0131.stratus.rno.ebay.com: Executor decommission.

In [17]:
item_pdf.columns

Index(['item_id', 'prdct_aspct_nm', 'aspct_vlu_nm', 'ns_type_cd', 'aspct_src',
       'auct_end_dt', 'load_ts', 'key_type'],
      dtype='object')

In [31]:
item_aspects = item_pdf[item_pdf['ns_type_cd'] == 'df'][['prdct_aspct_nm', 'aspct_vlu_nm']]
item_aspects

,prdct_aspct_nm,aspct_vlu_nm
0,MiscatScore,41
1,CatRecoID_2,262043
14,CatRecoID_1,262042
15,CatRecoScore_2,0
35,CatRecoID_3,41511
53,CatRecoScore_3,0
56,CatRecoScore_1,98
81,Theme,Cities & Towns
82,Subject,Church
95,Region,Yorkshire


In [32]:
len(item_aspects)

17

In [36]:
aspects_names_direct = set([i.lower() for i in item_aspects['prdct_aspct_nm']])

In [39]:
aspects_names_no_none.intersection(aspects_names_direct), aspects_names_no_none.difference(aspects_names_direct), aspects_names_direct.difference(aspects_names_no_none)

({'catrecoid_1',
  'catrecoid_2',
  'catrecoid_3',
  'catrecoscore_1',
  'catrecoscore_2',
  'catrecoscore_3',
  'city',
  'continent',
  'core_product_type_v2',
  'country',
  'mergednamespacenames',
  'miscatscore',
  'region',
  'subject',
  'theme'},
 set(),
 set())

In [40]:
len(aspects_names_no_none)

15

## item 175597284740

In [9]:
df_item = df_collectibles[df_collectibles['item_id'] == 175597284740]

NameError: name 'df_collectibles' is not defined

In [75]:
df_item

,item_id,item_desc,title,SUBTITLE,AUCT_START_DT,AUCT_END_DT,leaf_categ_id,ITEM_SITE_ID,bsns_vrtcl_name,meta_categ_id,aspects_with_none,desc,aspects,aspects_no_none,gen_text_mistral,gen_sig_mistral
4,175597284740,PGI+PGZvbnQgZmFjZT0iQXJpYWwiIHNpemU9IjQiPkw0OD...,signature models 1937 cord 812 supercharger (1...,None,2023-02-02,2024-09-02,180506,0,collectibles,220,"{'4wd/2wd': None, 'age': None, 'age level': No...",L48,"{'4wd/2wd': None, 'age': None, 'age level': No...","{'4wd/2wd': None, 'age': None, 'age level': No...","{""characteristic"": ""1:32 scale"", ""sale pitch"":...",1:32 scale


In [31]:
d = df_item['aspects']
d

NameError: name 'df_item' is not defined

In [24]:
len(list(d.items()))

1

In [28]:
d = d[4]

In [33]:
d['vehicle year']

'1937'

In [34]:
d['brand']

'cord'

In [35]:
len(d)

434

In [37]:
dd = df_item['aspects_with_none']
dd

4    {'4wd/2wd': None, 'age': None, 'age level': No...
Name: aspects_with_none, dtype: object

In [38]:
len(dd[4])

434

In [39]:
d['condition']

KeyError: 'condition'

In [40]:
d['vehicle make']

'cord'

In [42]:
d['model']

'812'

In [45]:
[k for k in d.keys() if 'cond' in k]

['card condition',
 'miniature condition',
 'postage condition',
 'sticker condition']

In [49]:
d['card condition'], d['miniature condition'], d['postage condition'], d['sticker condition']

(None, None, None, None)

In [80]:
aspects_no_none = {k:v for k,v in d.items() if v is not None}
aspects_no_none

{'brand': 'cord',
 'catrecoid_1': '180506',
 'catrecoid_2': '33741',
 'catrecoid_3': '180507',
 'catrecoscore_1': '70',
 'catrecoscore_2': '19',
 'catrecoscore_3': '4',
 'core_product_type': 'model vehicles',
 'core_product_type_v2': 'toy airplanes: part',
 'miscatscore': '49',
 'model': '812',
 'scale': '1:32',
 'shop for': 'collectors & hobbyists',
 'vehicle make': 'cord',
 'vehicle year': '1937'}

### Extract full aspects for item from HDM table

In [35]:
aspects_df = spark.table("ACCESS_VIEWS.ITEM_ASPCT_CLSSFCTN_SAP").filter((F.col('auct_end_dt') >= '2024-09-02')&\
                                                                (F.col('auct_end_dt') <= '2024-09-02')                                                                
                                                                )

In [17]:
aspects_df = spark.table("ACCESS_VIEWS.ITEM_ASPCT_CLSSFCTN_SAP").filter((F.col('auct_end_dt') >= '2024-09-02')&\
                                                                (F.col('auct_end_dt') <= '2024-09-02')&\
                                                                (F.col('item_id') == 175597284740)
                                                                )


                                                                    

In [18]:
aspects_df.count()

161

In [21]:
aspects_df.select('aspct_vlu_nm').distinct().show()

+--------------------+
|        aspct_vlu_nm|
+--------------------+
|  2.6311546952555407|
|  0.5354901146377823|
|rh9jre+s360wKtOoO...|
|   0.341935014915535|
|                1:32|
|           688730428|
| Toy Airplanes: part|
|                  64|
|                4015|
|FAQQGVrn5+XGu0OWT...|
| 1227725756169339168|
|1:1c54edda5da0cbd...|
|  721072956814406998|
|              180507|
|       1722626426794|
|  2.8869266871088057|
|Toy Motorcycles: ...|
|  1:3VUAAOSwUFBj3Bqv|
|                   0|
|pointintime/calen...|
+--------------------+
only showing top 20 rows



In [24]:
aspects_df.filter(F.col('aspct_vlu_nm').isNull()).count()

TypeError: 'Column' object is not callable

In [25]:
aspects_pdf = aspects_df.toPandas()
aspects_pdf['aspct_vlu_nm']

24/09/01 12:18:49 ERROR YarnScheduler: Lost executor 366 on hdc42-mcc10-01-0210-6001-009-tess0131.stratus.rno.ebay.com: Executor decommission.
24/09/01 12:18:49 ERROR YarnScheduler: Lost executor 373 on hdc42-mcc10-01-0510-3602-012-tess0097.stratus.rno.ebay.com: Executor decommission.
24/09/01 12:18:49 ERROR YarnScheduler: Lost executor 645 on hdc42-mcc10-01-0110-5509-004-tess0173.stratus.rno.ebay.com: Executor decommission.
24/09/01 12:18:49 ERROR YarnScheduler: Lost executor 523 on hdc42-mcc10-01-0210-7802-011-tess0131.stratus.rno.ebay.com: Executor decommission.
24/09/01 12:18:49 ERROR YarnScheduler: Lost executor 536 on hdc42-mcc10-01-0110-7303-002-tess0173.stratus.rno.ebay.com: Executor decommission.
24/09/01 12:18:49 ERROR YarnScheduler: Lost executor 392 on hdc42-mcc10-01-0510-3702-010-tess0097.stratus.rno.ebay.com: Executor decommission.
24/09/01 12:18:49 ERROR YarnScheduler: Lost executor 519 on hdc42-mcc10-01-1110-3707-007-tess0029.stratus.rno.ebay.com: Executor decommission.

0                                     2.6311546952555407
1                                       Signature Models
2                                                      0
3                                          1719948005225
4                                                   1937
                             ...                        
156                                                   64
157                   1:1c54edda5da0cbd09295cea8c0c6108d
158    rh9jre+s360wKtOoOSuRMoClkiqHILinmKZMpmWkfzIKKz...
159                                                 2893
160                                                    0
Name: aspct_vlu_nm, Length: 161, dtype: object

In [33]:
aspects_pdf

,item_id,prdct_aspct_nm,aspct_vlu_nm,ns_type_cd,aspct_src,auct_end_dt,load_ts,key_type
0,175597284740,LeafCatLogSalePriceMean,2.6311546952555407,anf,209,2024-09-02,2024-08-13 23:13:40,1
1,175597284740,Brand,Signature Models,sdattr,102,2024-09-02,2024-08-13 23:13:40,1
2,175597284740,ListQuality,0,lsqa,21,2024-09-02,2024-08-13 23:13:40,1
3,175597284740,buyerTaxonomyVisited,1719948005225,bt,187,2024-09-02,2024-08-13 23:13:40,1
4,175597284740,Year of Manufacture,1937,nf,160,2024-09-02,2024-08-13 23:13:40,1
...,...,...,...,...,...,...,...,...
156,175597284740,IQMedianDecayed7IOS1,64,,236,2024-09-02,2024-08-31 22:52:24,2
157,175597284740,_qryitemsimembeddingFP,1:1c54edda5da0cbd09295cea8c0c6108d,_,260,2024-09-02,2024-08-31 22:52:24,2
158,175597284740,QryItemSimEmbC,rh9jre+s360wKtOoOSuRMoClkiqHILinmKZMpmWkfzIKKz...,cub,232,2024-09-02,2024-08-31 22:52:24,2
159,175597284740,IQMedianNTotalCost1,2893,,236,2024-09-02,2024-08-31 22:52:24,2


24/09/01 12:22:35 ERROR YarnScheduler: Lost executor 548 on hdc42-mcc10-01-0110-7810-019-tess0173.stratus.rno.ebay.com: Executor decommission.


In [32]:
aspects_pdf['aspct_vlu_nm'].value_counts()

aspct_vlu_nm
Cord                                                                                                                                                                                                                                                                11
1937                                                                                                                                                                                                                                                                 9
1:32                                                                                                                                                                                                                                                                 8
Signature Models                                                                                                                                                                                      

In [28]:
len(aspects_pdf['aspct_vlu_nm'].unique())

104

In [29]:
len(aspects_pdf['aspct_vlu_nm'])

161

In [61]:
aspects = {k:v for k,v in zip( aspects_pdf['prdct_aspct_nm'], aspects_pdf['aspct_vlu_nm'])}

In [74]:
 aspects['Vehicle Make'], aspects['Vehicle Year'], aspects['Brand'], aspects['Model'], aspects['Scale']

('Cord', '1937', 'Signature Models', '812', '1:32')

In [72]:
aspects['condition'], aspects['UPC']

KeyError: 'condition'

In [76]:
[v for v in aspects_pdf['aspct_vlu_nm'] if 'new' in v]

[]

In [77]:
aspects_pdf[aspects_pdf['prdct_aspct_nm'] == 'Brand']

,item_id,prdct_aspct_nm,aspct_vlu_nm,ns_type_cd,aspct_src,auct_end_dt,load_ts,key_type
19,175597284740,Brand,Signature Models,vdf,185,2024-09-02,2024-08-11 09:32:38,2
72,175597284740,Brand,Signature Models,sdattr,102,2024-09-02,2024-08-13 23:13:40,1
84,175597284740,Brand,Cord,df,30,2024-09-02,2024-08-13 23:13:40,1
101,175597284740,Brand,Signature Models,umns,101,2024-09-02,2024-08-13 23:13:40,1
111,175597284740,Brand,Signature Models,df,3,2024-09-02,2024-08-13 23:13:40,1
113,175597284740,Brand,Signature Models,aenv,3,2024-09-02,2024-08-13 23:13:40,1
127,175597284740,Brand,Signature Models,df,101,2024-09-02,2024-08-13 23:13:40,1
129,175597284740,Brand,sm,da,240,2024-09-02,2024-08-13 23:13:40,1
152,175597284740,Brand,Signature Models,vdf,156,2024-09-02,2024-08-13 23:13:40,1
157,175597284740,Brand,Signature Models,ae,3,2024-09-02,2024-08-13 23:13:40,1


In [82]:
!cat ../signals/data/RawSqlData/ep_2MLs_Aug24/data_no_features/


cat: ../signals/data/RawSqlData/ep_2MLs_Aug24/data_no_features/: Is a directory


## Item 114664125925

In [37]:
df_item = aspects_df.filter(F.col('ITEM_ID') == 114664125925)
df_item.count()

173

In [38]:
df_item.filter(F.col('aspct_vlu_nm').isNull()).count()

0

24/09/01 12:46:38 ERROR YarnScheduler: Lost executor 701 on hdc42-mcc10-01-0110-5902-019-tess0173.stratus.rno.ebay.com: Executor decommission.
24/09/01 12:46:38 ERROR YarnScheduler: Lost executor 709 on hdc42-mcc10-01-0910-2007-010-tess0028.stratus.rno.ebay.com: Executor decommission.
24/09/01 12:46:38 ERROR YarnScheduler: Lost executor 699 on hdc42-mcc10-01-0710-3301-014-tess0035.stratus.rno.ebay.com: Executor decommission.
24/09/01 12:46:38 ERROR YarnScheduler: Lost executor 724 on hdc42-mcc10-01-0110-7602-012-tess0173.stratus.rno.ebay.com: Executor decommission.
24/09/01 12:46:38 ERROR YarnScheduler: Lost executor 707 on hdc42-mcc10-01-0110-7701-019-tess0173.stratus.rno.ebay.com: Executor decommission.
24/09/01 12:46:38 ERROR YarnScheduler: Lost executor 735 on hdc42-mcc10-01-0210-7102-004-tess0131.stratus.rno.ebay.com: Executor decommission.
24/09/01 12:46:38 ERROR YarnScheduler: Lost executor 691 on hdc42-mcc10-01-0510-4004-034-tess0097.stratus.rno.ebay.com: Executor decommission.

In [40]:
pdf_item = df_item.toPandas()
pdf_item

,item_id,prdct_aspct_nm,aspct_vlu_nm,ns_type_cd,aspct_src,auct_end_dt,load_ts,key_type
0,114664125925,Color,Multi-color,vdf,185,2024-09-02,2024-08-30 01:21:26,2
1,114664125925,MarketPriceUSD,708,cat,94,2024-09-02,2024-08-30 01:21:26,2
2,114664125925,Original/Reproduction,Original,vdf,185,2024-09-02,2024-08-30 01:21:26,2
3,114664125925,KeyValues_Hash3,576460786663162402,vis,98,2024-09-02,2024-08-30 01:21:26,2
4,114664125925,_aspectextractionFP,5:79f5805d0925bd40c3aa2733bd3a52c0,_,250,2024-09-02,2024-08-30 01:21:26,2
...,...,...,...,...,...,...,...,...
168,114664125925,ListQuality,75,lsqa,21,2024-09-02,2024-08-02 17:08:43,1
169,114664125925,GradedRelevanceA,rK6BsL0wQTCNMFss0y3EIQcsQiyXMK6meBI+qI2kIajJsX...,embf,202,2024-09-02,2024-08-02 17:08:43,1
170,114664125925,Theme,Restaurants & Fast Food,df,3,2024-09-02,2024-08-02 17:08:43,1
171,114664125925,Theme,Restaurants & Fast Food,umns,101,2024-09-02,2024-08-02 17:08:43,1


24/09/01 13:00:07 ERROR YarnScheduler: Lost executor 1133 on hdc42-mcc10-01-1110-3707-007-tess0029.stratus.rno.ebay.com: Executor decommission.
24/09/01 13:00:07 ERROR YarnScheduler: Lost executor 1160 on hdc42-mcc10-01-0210-7401-004-tess0131.stratus.rno.ebay.com: Executor decommission.
24/09/01 13:00:07 ERROR YarnScheduler: Lost executor 801 on hdc42-mcc10-01-0110-7910-008-tess0173.stratus.rno.ebay.com: Executor decommission.
24/09/01 13:00:07 ERROR YarnScheduler: Lost executor 1009 on hdc42-mcc10-01-0910-1704-009-tess0028.stratus.rno.ebay.com: Executor decommission.
24/09/01 13:00:07 ERROR YarnScheduler: Lost executor 961 on hdc42-mcc10-01-0210-5401-016-tess0131.stratus.rno.ebay.com: Executor decommission.
24/09/01 13:00:07 ERROR YarnScheduler: Lost executor 1299 on hdc42-mcc10-01-0910-2206-003-tess0028.stratus.rno.ebay.com: Executor decommission.
24/09/01 13:00:07 ERROR YarnScheduler: Lost executor 1144 on hdc42-mcc10-01-0110-5306-019-tess0173.stratus.rno.ebay.com: Executor decommis

In [41]:
pdf_item.columns

Index(['item_id', 'prdct_aspct_nm', 'aspct_vlu_nm', 'ns_type_cd', 'aspct_src',
       'auct_end_dt', 'load_ts', 'key_type'],
      dtype='object')

24/09/01 13:01:16 ERROR YarnScheduler: Lost executor 794 on hdc42-mcc10-01-1110-2102-022-tess0029.stratus.rno.ebay.com: Executor decommission.


In [42]:
pdf_item[pdf_item['prdct_aspct_nm'] == '4wd/2wd']

,item_id,prdct_aspct_nm,aspct_vlu_nm,ns_type_cd,aspct_src,auct_end_dt,load_ts,key_type


In [46]:
pdf_item[pdf_item['ns_type_cd'] == 'df']

,item_id,prdct_aspct_nm,aspct_vlu_nm,ns_type_cd,aspct_src,auct_end_dt,load_ts,key_type
23,114664125925,CatRecoScore_3,21,df,6,2024-09-02,2024-08-30 01:21:26,2
34,114664125925,MiscatScore,56,df,6,2024-09-02,2024-08-30 01:21:26,2
36,114664125925,CatRecoID_3,261672,df,6,2024-09-02,2024-08-30 01:21:26,2
41,114664125925,CatRecoID_2,850,df,6,2024-09-02,2024-08-30 01:21:26,2
59,114664125925,CatRecoScore_2,25,df,6,2024-09-02,2024-08-30 01:21:26,2
62,114664125925,CatRecoID_1,21147,df,6,2024-09-02,2024-08-30 01:21:26,2
74,114664125925,CatRecoScore_1,40,df,6,2024-09-02,2024-08-30 01:21:26,2
81,114664125925,core_product_type_v2,Vintage Advertisements: primary product,df,228,2024-09-02,2024-08-02 17:08:43,1
83,114664125925,Country/Region of Manufacture,United States,df,101,2024-09-02,2024-08-02 17:08:43,1
85,114664125925,Original/Reproduction,Original,df,101,2024-09-02,2024-08-02 17:08:43,1


24/09/01 13:03:05 ERROR YarnScheduler: Lost executor 1024 on hdc42-mcc10-01-0110-6004-031-tess0173.stratus.rno.ebay.com: Executor decommission.


## item 165964697050

In [19]:
aspects_df.select('auct_end_dt').distinct().show()

24/09/07 10:43:48 ERROR YarnScheduler: Lost executor 5776 on hdc42-mcc10-01-1110-2603-033-tess0029.stratus.rno.ebay.com: Executor decommission.
24/09/07 10:43:49 ERROR YarnScheduler: Lost executor 6040 on hdc42-mcc10-01-0910-2206-019-tess0028.stratus.rno.ebay.com: Executor decommission.
24/09/07 10:43:49 ERROR YarnScheduler: Lost executor 6111 on hdc42-mcc10-01-1110-3503-005-tess0029.stratus.rno.ebay.com: Executor decommission.
24/09/07 10:43:50 ERROR YarnScheduler: Lost executor 5794 on hdc42-mcc10-01-0210-5901-009-tess0131.stratus.rno.ebay.com: Executor decommission.
24/09/07 10:43:51 ERROR YarnScheduler: Lost executor 5916 on hdc42-mcc10-01-0710-3507-012-tess0035.stratus.rno.ebay.com: Executor decommission.
24/09/07 10:43:52 ERROR YarnScheduler: Lost executor 5994 on hdc42-mcc10-01-0110-7503-014-tess0173.stratus.rno.ebay.com: Executor decommission.
24/09/07 10:43:52 ERROR YarnScheduler: Lost executor 5934 on hdc42-mcc10-01-0210-5102-007-tess0131.stratus.rno.ebay.com: Executor decomm

+-----------+
|auct_end_dt|
+-----------+
| 2024-09-17|
+-----------+



24/09/07 10:47:35 ERROR YarnScheduler: Lost executor 6018 on hdc42-mcc10-01-0210-6002-028-tess0131.stratus.rno.ebay.com: Executor decommission.
24/09/07 10:47:48 ERROR YarnScheduler: Lost executor 5937 on hdc42-mcc10-01-0210-7703-004-tess0131.stratus.rno.ebay.com: Executor decommission.
24/09/07 10:47:56 ERROR YarnScheduler: Lost executor 6236 on hdc42-mcc10-01-0210-5102-016-tess0131.stratus.rno.ebay.com: Executor decommission.
24/09/07 10:47:57 ERROR YarnScheduler: Lost executor 6253 on hdc42-mcc10-01-0110-5609-008-tess0173.stratus.rno.ebay.com: Executor decommission.
24/09/07 10:47:57 ERROR YarnScheduler: Lost executor 6248 on hdc42-mcc10-01-0710-3508-020-tess0035.stratus.rno.ebay.com: Executor decommission.
24/09/07 10:47:58 ERROR YarnScheduler: Lost executor 6232 on hdc42-mcc10-01-0210-5101-018-tess0131.stratus.rno.ebay.com: Executor decommission.
24/09/07 10:47:58 ERROR YarnScheduler: Lost executor 6261 on hdc42-mcc10-01-0110-7602-017-tess0173.stratus.rno.ebay.com: Executor decomm

In [24]:
df_item = aspects_df.filter(F.col('ITEM_ID') == 165964697050)
# df_item.count()

In [25]:
pdf_item = df_item.toPandas()
pdf_item

,item_id,prdct_aspct_nm,aspct_vlu_nm,ns_type_cd,aspct_src,auct_end_dt,load_ts,key_type


24/09/07 10:58:19 ERROR YarnScheduler: Lost executor 6731 on hdc42-mcc10-01-0110-7402-012-tess0173.stratus.rno.ebay.com: Executor decommission.
24/09/07 10:58:19 ERROR YarnScheduler: Lost executor 6734 on hdc42-mcc10-01-0110-5803-019-tess0173.stratus.rno.ebay.com: Executor decommission.
24/09/07 10:58:19 ERROR YarnScheduler: Lost executor 6737 on hdc42-mcc10-01-1110-3407-008-tess0029.stratus.rno.ebay.com: Executor decommission.
24/09/07 10:58:19 ERROR YarnScheduler: Lost executor 6670 on hdc42-mcc10-01-0110-7102-011-tess0173.stratus.rno.ebay.com: Executor decommission.
24/09/07 10:58:19 ERROR YarnScheduler: Lost executor 6652 on hdc42-mcc10-01-0910-1705-001-tess0028.stratus.rno.ebay.com: Executor decommission.
24/09/07 10:58:19 ERROR YarnScheduler: Lost executor 6669 on hdc42-mcc10-01-0110-7501-011-tess0173.stratus.rno.ebay.com: Executor decommission.
24/09/07 10:58:19 ERROR YarnScheduler: Lost executor 6650 on hdc42-mcc10-01-1110-2104-021-tess0029.stratus.rno.ebay.com: Executor decomm

# Debug URL lowercase 

In [15]:
start_date = '2024-09-17'
end_date = '2024-09-17'
business_vertical_names = ['Collectibles']
site_ids = [0]
n_items = 5000
take_shortest_dup_value = False
leafcats=None
metacats=None

In [16]:


items_df = spark.table("ACCESS_VIEWS.DW_LSTG_ITEM").filter((F.col('auct_end_dt') >= start_date) &\
                                                           (F.col('auct_end_dt') <= end_date))
cats_table = spark.table('ACCESS_VIEWS.DW_CATEGORY_GROUPINGS').withColumnRenamed('leaf_categ_id',
                                                                                 'leaf_categ_id_cat_table')

if leafcats is not None:
    items_df = items_df.filter(F.col('leaf_categ_id').isin(leafcats))

if site_ids is not None:
    items_df = items_df.filter(F.col('ITEM_SITE_ID').isin(site_ids))

if (business_vertical_names is not None) or (metacats is not None):
    if metacats is not None:
        cats_table = cats_table.filter(F.col('meta_categ_id').isin(metacats))
    if business_vertical_names is not None:
        cats_table = cats_table.filter(F.col('bsns_vrtcl_name').isin(business_vertical_names))
    items_df = items_df.join(
        broadcast(cats_table),
        (items_df['leaf_categ_id'] == cats_table['leaf_categ_id_cat_table']) &
        (items_df['ITEM_SITE_ID'] == cats_table['SITE_ID']),
        how='inner'
    )
else:
    items_df = items_df.join(
        broadcast(cats_table),
        (items_df['leaf_categ_id'] == cats_table['leaf_categ_id_cat_table']) &
        (items_df['ITEM_SITE_ID'] == cats_table['SITE_ID']),
        how='left'
    )

if n_items is not None:
    # select n_items random items
    items_df = items_df.sample(False, 0.01).limit(n_items)

outcols = ['ITEM_ID', 'AUCT_TITL', 'SUBTITLE', 'AUCT_START_DT', 'AUCT_END_DT',  'leaf_categ_id',\
                           'ITEM_SITE_ID', 'bsns_vrtcl_name', 'meta_categ_id', 'GALLERY_URL']
items_df = items_df.select( *outcols )#.cache()


24/09/14 16:42:00 ERROR YarnScheduler: Lost executor 53 on hdc42-mcc10-01-0210-6001-011-tess0131.stratus.rno.ebay.com: Executor decommission.
24/09/14 16:42:01 ERROR YarnScheduler: Lost executor 20 on hdc42-mcc10-01-1110-1906-009-tess0029.stratus.rno.ebay.com: Executor decommission.
24/09/14 16:42:01 ERROR YarnScheduler: Lost executor 29 on hdc42-mcc10-01-0210-5102-005-tess0131.stratus.rno.ebay.com: Executor decommission.
24/09/14 16:42:02 ERROR YarnScheduler: Lost executor 27 on hdc42-mcc10-01-1110-3708-031-tess0029.stratus.rno.ebay.com: Executor decommission.
24/09/14 16:42:02 ERROR YarnScheduler: Lost executor 35 on hdc42-mcc10-01-0910-1909-040-tess0028.stratus.rno.ebay.com: Executor decommission.
24/09/14 16:42:03 ERROR YarnScheduler: Lost executor 28 on hdc42-mcc10-01-0110-7703-020-tess0173.stratus.rno.ebay.com: Executor decommission.
24/09/14 16:42:04 ERROR YarnScheduler: Lost executor 17 on hdc42-mcc10-01-0110-7501-010-tess0173.stratus.rno.ebay.com: Executor decommission.
24/09/

In [18]:
items_pdf = items_df.limit(3).toPandas()
items_pdf

,ITEM_ID,AUCT_TITL,SUBTITLE,AUCT_START_DT,AUCT_END_DT,leaf_categ_id,ITEM_SITE_ID,bsns_vrtcl_name,meta_categ_id,GALLERY_URL
0,266954539035,2023 Panini Donruss Red Press Proof Jaxon Smit...,None,2024-08-17,2024-09-17,261328,0,Collectibles,64482,https://i.ebayimg.com/00/s/MTYwMFgxMTc2/z/9aYA...
1,126581483193,Vintage Green Bay Packers NFL No Huddle 1921 J...,None,2024-07-17,2024-09-17,206,0,Collectibles,64482,https://i.ebayimg.com/00/s/MTYwMFgxNjAw/z/RekA...
2,132747789577,"Eusebio ""The Black Panther"" - Benfica 8x10 Col...",None,2018-08-17,2024-09-17,2887,0,Collectibles,64482,https://i.ebayimg.com/00/s/MTYwMFgxMjgw/z/mukA...


24/09/14 17:26:22 ERROR YarnScheduler: Lost executor 876 on hdc42-mcc10-01-0710-4201-005-tess0035.stratus.rno.ebay.com: Executor decommission.
24/09/14 17:26:22 ERROR YarnScheduler: Lost executor 947 on hdc42-mcc10-01-1110-1904-031-tess0029.stratus.rno.ebay.com: Executor decommission.
24/09/14 17:26:25 ERROR YarnScheduler: Lost executor 801 on hdc42-mcc10-01-0910-1709-021-tess0028.stratus.rno.ebay.com: Executor decommission.
24/09/14 17:26:27 ERROR YarnScheduler: Lost executor 921 on hdc42-mcc10-01-1110-1808-008-tess0029.stratus.rno.ebay.com: Executor decommission.
24/09/14 17:26:38 ERROR YarnScheduler: Lost executor 839 on hdc42-mcc10-01-0910-1708-012-tess0028.stratus.rno.ebay.com: Executor decommission.
24/09/14 17:26:43 ERROR YarnScheduler: Lost executor 784 on hdc42-mcc10-01-1110-3304-001-tess0029.stratus.rno.ebay.com: Executor decommission.
24/09/14 17:26:44 ERROR YarnScheduler: Lost executor 888 on hdc42-mcc10-01-0710-2108-021-tess0035.stratus.rno.ebay.com: Executor decommission.